In [176]:
import random
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np
import math
import random
import unidecode
import datetime
import time
from time import strptime
import math
from IPython.core.display import display, HTML
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
display(HTML("<style>.container { width:80% !important; }</style>"))

# Below are dicts, lists, and strings that will be used in later functions
header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinnati',
                        'PIT': 'Pittsburgh',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SF': 'San Francisco',
                        'COL': 'Colorado',
                        'SD': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TB': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KC': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
#acronym_to_city_dict = {'NYY': 'New York Yankees', 'ATL': 'Atlanta'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinnati',
                        'pirates': 'Pittsburgh',
                        'dodgers': 'Los Angeles Dodgers',
                        'diamondbacks': 'Arizona',
                        'd-backs': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}
league = {'Atlanta': 'NL',
          'Washington': 'NL',
          'New York Mets': 'NL',
          'Philadelphia': 'NL',
          'Miami': 'NL',
          'St. Louis': 'NL',
          'Milwaukee': 'NL',
          'Chicago Cubs': 'NL',
          'Cincinnati': 'NL',
          'Pittsburgh': 'NL',
          'Los Angeles Dodgers': 'NL',
          'Arizona': 'NL',
          'San Francisco': 'NL',
          'Colorado': 'NL',
          'San Diego': 'NL',
          'New York Yankees': 'AL',
          'Tampa Bay': 'AL',
          'Boston': 'AL',
          'Toronto': 'AL',
          'Baltimore': 'AL',
          'Minnesota': 'AL',
          'Cleveland': 'AL',
          'Chicago White Sox': 'AL',
          'Kansas City': 'AL',
          'Detroit': 'AL',
          'Houston': 'AL',
          'Oakland': 'AL',
          'Texas': 'AL',
          'Los Angeles Angels': 'AL',
          'Seattle': 'AL'}
team_num_dict = {'Atlanta': 16,
          'Washington': 24,
          'New York Mets': 25,
          'Philadelphia': 26,
          'Miami': 20,
          'St. Louis': 28,
          'Milwaukee': 23,
          'Chicago Cubs': 17,
          'Cincinnati': 18,
          'Pittsburgh': 27,
          'Los Angeles Dodgers': 22,
          'Arizona': 15,
          'San Francisco': 30,
          'Colorado': 19,
          'San Diego': 29,
          'New York Yankees': 9,
          'Tampa Bay': 12,
          'Boston': 3,
          'Toronto': 14,
          'Baltimore': 2,
          'Minnesota': 8,
          'Cleveland': 5,
          'Chicago White Sox': 4,
          'Kansas City': 7,
          'Detroit': 6,
          'Houston': 21,
          'Oakland': 10,
          'Texas': 13,
          'Los Angeles Angels': 1,
          'Seattle': 11}
mlb_teams = ['braves', 'nationals', 'mets', 'phillies', 'marlins', 
         'cardinals', 'brewers', 'cubs', 'reds', 'pirates', 
         'dodgers', 'd-backs', 'giants', 'rockies', 'padres', 
         'yankees', 'rays', 'red sox', 'blue jays', 'orioles', 'twins',
         'indians', 'white sox', 'royals', 'tigers', 'astros', 'athletics',
         'rangers', 'angels', 'mariners']
new_player_id = {'Anthony Bemboom': '13854',
 'Jared Walsh': '18607',
 'Elliot Soto': 'sa390666',
 'Michael Hermosillo': '16285',
 'Taylor Ward': '17548',
 'Jo Adell': 'sa3004837',
 'Austin Wynns': '15271',
 'Ryan Mountcastle': 'sa874734',
 'Andrew Velazquez': '14196',
 'Ramon Urias': 'sa597169',
 'Rylan Bannon': 'sa3004162',
 'Yusniel Diaz': 'sa912848',
 'Ryan McKenna': 'sa875055',
 'Mason Williams': '11859',
 'Juan Centeno\xa0\xa0': '7168',
 'Bobby Dalbec': 'sa857701',
 'Josh Ockimey': 'sa829375',
 'Jonathan Arauz': 'sa877498',
 'C.J. Chatham': 'sa873984',
 'Dustin Pedroia\xa0\xa0': '8370',
 'Tzu-Wei Lin': '14678',
 'Seby Zavala': '18887',
 'Nick Madrigal': 'sa3008139',
 'Danny Mendick': '18889',
 'Nicky Delmonico': '13157',
 'Daniel Palka': '14897',
 'Luis Robert': 'sa3003396',
 'Luis Alexander Basabe': 'sa736915',
 'Beau Taylor': '12384',
 'Bobby Bradley': '17278',
 'Daniel Johnson': 'sa918018',
 'Bradley Zimmer': '16221',
 'Eric Haase': '14111',
 'Sergio Alcantara': 'sa739641',
 'Isaac Paredes': 'sa920275',
 'Jorge Bonifacio': '12174',
 'Daz Cameron': 'sa874736',
 'Troy Stokes Jr.': 'sa828871',
 'Salvador Perez': '7304',
 'Matt Reynolds': '13788',
 'Erick Mejia': '15817',
 'Nick Heath': 'sa918355',
 'Nick Gordon': 'sa828662',
 'Luke Raley': 'sa918048',
 'Brent Rooker': 'sa3003172',
 'Alex Kirilloff': 'sa917929',
 'Kyle Higashioka': '5517',
 'Erik Kratz': '4403',
 'Miguel Andujar': '15878',
 'Austin Allen': '18083',
 'Jonah Heim': 'sa737843',
 'Vimael Machin': 'sa602488',
 'Jorge Mateo': 'sa657910',
 'Dustin Fowler': '17098',
 'Buddy Reed': 'sa738683',
 'Skye Bolt': '17723',
 'Joseph Odom': 'sa738310',
 'Joe Hudson': '14412',
 'Evan White': 'sa915774',
 'Patrick Wisdom': '13602',
 'Sam Haggerty\xa0\xa0': '18054',
 'Donnie Walton': '19314',
 'Jake Fraley': '19260',
 'Eric Filia': 'sa827359',
 'Michael Perez': '12977',
 'Chris Herrmann': '9284',
 'Yoshitomo Tsutsugo': '27459',
 'Esteban Quiroz': 'sa596560',
 'Randy Arozarena': '19290',
 "Brian O'Grady": '16729',
 'Brendan McKay': '20186',
 'Greg Bird': '14131',
 'Eli White': 'sa738478',
 'Adolis Garcia': '19287',
 'Caleb Joseph': '7087',
 'Santiago Espinal': 'sa918188',
 'Anthony Alford': '14329',
 'Daulton Varsho': 'sa3002838',
 'Domingo Leyba\xa0\xa0': '16404',
 'Andy Young': 'sa919031',
 'Wyatt Mathisen': 'sa657937',
 'Alex Jackson': '17276',
 'Cristian Pache': 'sa916722',
 'Nico Hoerner': '21479',
 'Steven Souza Jr.': '5667',
 'Ian Miller': '15156',
 'Alex Blandino': '16271',
 'Shogo Akiyama': '27461',
 'Travis Jankowski\xa0\xa0': '13768',
 'Drew Butera': '3411',
 'Dom Nunez': '15055',
 'Chad Wallach': '17161',
 'Lewin Diaz': 'sa828420',
 'Christian Lopes': 'sa601044',
 'Magneuris Sierra': '17023',
 'Matt Kemp': '5631',
 'Monte Harrison': 'sa828691',
 'Jesus Sanchez': 'sa872787',
 'Garrett Stubbs': '18067',
 'Taylor Jones': 'sa876202',
 'Ronnie Dawson': 'sa873674',
 'Rocky Gale': '10533',
 'Gavin Lux': '19955',
 'Zach McKinstry': 'sa918920',
 'Edwin Rios': '18316',
 'David Freitas': '10619',
 'Jacob Nottingham': '16448',
 'Logan Morrison': '9205',
 'Mark Mathias': 'sa875006',
 'Corey Ray': 'sa738510',
 'Tyrone Taylor\xa0\xa0': '13675',
 'Raudy Read': '15766',
 'Tres Barrera': '19977',
 'Carter Kieboom': '19958',
 'Adrian Sanchez': '11338',
 'Andrew Stevenson': '17932',
 'Rene Rivera': '3648',
 'Jed Lowrie\xa0\xa0': '4418',
 'Luis Guillorme': '16451',
 'Yoenis Cespedes\xa0\xa0': '13110',
 'Deivy Grullon': '15988',
 'Arquimedes Gamboa': 'sa877497',
 'Alec Bohm': 'sa3007284',
 'Kyle Garlick': '18063',
 'Mickey Moniak': 'sa917920',
 'Kevin Kramer': '17689',
 "Ke'Bryan Hayes": 'sa874730',
 'Jason Martin': '16429',
 'Socrates Brito': '12944',
 'Andrew Knizner': '19514',
 'Rangel Ravelo': '10839',
 'Edmundo Sosa': '17022',
 'Max Schrock': 'sa658670',
 'Dylan Carlson': 'sa917940',
 'Justin Williams': '15490',
 'Lane Thomas': '16939',
 'Luis Torrens': '15905',
 'Jake Cronenworth': 'sa857694',
 'Abraham Almonte': '5486',
 'Franchy Cordero': '14567',
 'Edward Olivares': 'sa832015',
 'Rob Brantly': '10655',
 'Tyler Heineman': '13897',
 'Aramis Garcia\xa0\xa0': '16925',
 'Kean Wong': '15994',
 'Cristhian Adames': '6013',
 'Jaylin Davis': '19552',
 'Chris Shaw': '17738',
 'Drew Robinson': '11761',
 'Jose Siri': 'sa736940',
 'Joe McCarthy': 'sa858039'}
id_bats_dict = {'13854': 'L',
 'sa828871': 'R',
 '5667': 'R',
 '18607': 'L',
 'sa390666': 'R',
 '16285': 'R',
 '17548': 'R',
 'sa3004837': 'R',
 '15271': 'R',
 'sa874734': 'R',
 '14196': 'S',
 'sa597169': 'R',
 'sa3004162': 'R',
 'sa912848': 'R',
 'sa875055': 'R',
 '11859': 'L',
 '7168': 'L',
 'sa857701': 'R',
 'sa829375': 'L',
 'sa877498': 'S',
 'sa873984': 'R',
 '8370': 'R',
 '14678': 'L',
 '18887': 'R',
 'sa3008139': 'R',
 '18889': 'R',
 '13157': 'L',
 '14897': 'L',
 'sa3003396': 'R',
 'sa736915': 'S',
 '12384': 'L',
 '17278': 'L',
 'sa918018': 'L',
 '16221': 'L',
 '14111': 'R',
 'sa739641': 'S',
 'sa920275': 'R',
 '12174': 'R',
 'sa874736': 'R',
 '7304': 'R',
 '13788': 'R',
 '15817': 'S',
 'sa918355': 'L',
 'sa828662': 'L',
 'sa918048': 'L',
 'sa3003172': 'R',
 'sa917929': 'L',
 '5517': 'R',
 '4403': 'R',
 '15878': 'R',
 '18083': 'L',
 'sa737843': 'S',
 'sa602488': 'L',
 'sa657910': 'R',
 '17098': 'L',
 'sa738683': 'S',
 '17723': 'S',
 'sa738310': 'R',
 '14412': 'R',
 'sa915774': 'R',
 '13602': 'R',
 '18054': 'S',
 '19314': 'L',
 '19260': 'L',
 'sa827359': 'L',
 '12977': 'L',
 '9284': 'L',
 '27459': 'L',
 'sa596560': 'L',
 '19290': 'R',
 '16729': 'L',
 '20186': 'L',
 '14131': 'L',
 'sa738478': 'R',
 '19287': 'R',
 '7087': 'R',
 'sa918188': 'R',
 '14329': 'R',
 'sa3002838': 'L',
 '16404': 'S',
 'sa919031': 'R',
 'sa657937': 'R',
 '17276': 'R',
 'sa916722': 'R',
 '21479': 'R',
 '15156': 'L',
 '16271': 'R',
 '27461': 'L',
 '13768': 'L',
 '3411': 'R',
 '15055': 'L',
 '17161': 'R',
 'sa828420': 'L',
 'sa601044': 'R',
 '17023': 'L',
 '5631': 'R',
 'sa828691': 'R',
 'sa872787': 'L',
 '18067': 'L',
 'sa876202': 'R',
 'sa873674': 'L',
 '10533': 'R',
 '19955': 'L',
 'sa918920': 'L',
 '18316': 'L',
 '10619': 'R',
 '16448': 'R',
 '9205': 'L',
 'sa875006': 'R',
 'sa738510': 'L',
 '13675': 'R',
 '15766': 'R',
 '19977': 'R',
 '19958': 'R',
 '11338': 'R',
 '17932': 'L',
 '3648': 'R',
 '4418': 'S',
 '16451': 'L',
 '13110': 'R',
 '15988': 'R',
 'sa877497': 'S',
 'sa3007284': 'R',
 '18063': 'R',
 'sa917920': 'L',
 '17689': 'L',
 'sa874730': 'R',
 '16429': 'L',
 '12944': 'L',
 '19514': 'R',
 '10839': 'R',
 '17022': 'R',
 'sa658670': 'L',
 'sa917940': 'S',
 '15490': 'L',
 '16939': 'R',
 '15905': 'R',
 'sa857694': 'L',
 '5486': 'S',
 '14567': 'L',
 'sa832015': 'R',
 '10655': 'L',
 '13897': 'S',
 '16925': 'R',
 '15994': 'L',
 '6013': 'S',
 '19552': 'R',
 '17738': 'L',
 '11761': 'L',
 'sa736940': 'R',
 'sa858039': 'L',
 '14335': 'R',
 '20275': 'S',
 'sa597096': 'R',
 '20043': 'R',
 '19643': 'L',
 '13490': 'R',
 'sa874969': 'L',
 '19938': 'R',
 '17684': 'L',
 '20078': 'R',
 '15767': 'L',
 '13770': 'R',
 'sa914201': 'R',
 '18900': 'S',
 '19997': 'R',
 '20503': 'R',
 '13288': 'R',
 '17988': 'R',
 '7226': 'R',
 '11145': 'R',
 'sa874752': 'R',
 '19533': 'R',
 '19703': 'R',
 '4054': 'S',
 'sa724030': 'L',
 '19950': 'L',
 '11400': 'R',
 '13130': 'R',
 '8722': 'L',
 '10059': 'R',
 '1177': 'R',
 '12371': 'L',
 '19318': 'L',
 '17992': 'R',
 '19858': 'S',
 '10847': 'R',
 '12861': 'R',
 '5222': 'R',
 '12858': 'L',
 '10155': 'R',
 '19755': 'L',
 '14523': 'R',
 '15694': 'L',
 '11287': 'R',
 '9272': 'L',
 '14885': 'R',
 '14503': 'R',
 '11902': 'R',
 '16041': 'S',
 '10231': 'R',
 '14103': 'L',
 '17766': 'L',
 '13473': 'L',
 '14551': 'S',
 '17929': 'S',
 '19363': 'R',
 '9774': 'R',
 '13807': 'R',
 '7870': 'R',
 '3086': 'L',
 '17321': 'R',
 '13593': 'R',
 '12161': 'R',
 '17350': 'L',
 '16313': 'R',
 '17901': 'L',
 '12434': 'R',
 '6184': 'R',
 '17027': 'L',
 '7216': 'R',
 '12984': 'L',
 '11368': 'S',
 '12859': 'R',
 '19181': 'L',
 '15676': 'R',
 '2151': 'R',
 '10473': 'R',
 '5913': 'S',
 '9807': 'L',
 '15172': 'R',
 '17232': 'S',
 '17484': 'R',
 '15082': 'R',
 '14553': 'L',
 '2900': 'R',
 '5273': 'S',
 '2396': 'S',
 '15194': 'L',
 '11241': 'L',
 '10556': 'S',
 '16434': 'R',
 '14773': 'R',
 '12916': 'S',
 '13510': 'S',
 '16424': 'R',
 '10348': 'R',
 '16623': 'S',
 '14566': 'R',
 '13359': 'L',
 '16375': 'R',
 '11379': 'R',
 '5491': 'R',
 '17062': 'R',
 '19452': 'R',
 '12546': 'R',
 '13621': 'S',
 '6547': 'R',
 '1744': 'R',
 '12092': 'S',
 '11265': 'R',
 '14691': 'L',
 '17338': 'S',
 '14713': 'R',
 '15214': 'R',
 '17714': 'L',
 '5223': 'R',
 '15958': 'R',
 '15487': 'S',
 '14818': 'R',
 '12981': 'R',
 '17109': 'L',
 '16442': 'L',
 '16524': 'R',
 '15117': 'R',
 '12179': 'R',
 '19339': 'L',
 '11281': 'R',
 '13595': 'R',
 '13769': 'S',
 '17468': 'R',
 '5209': 'L',
 '14735': 'L',
 '13356': 'R',
 '19577': 'L',
 '14221': 'R',
 '15161': 'R',
 '7476': 'L',
 '15608': 'R',
 '12164': 'R',
 '5497': 'S',
 '18568': 'L',
 '8418': 'S',
 '13152': 'S',
 '10416': 'R',
 '5038': 'R',
 '12155': 'L',
 '14477': 'L',
 '14161': 'R',
 '12144': 'L',
 '18126': 'L',
 '2434': 'R',
 '11442': 'R',
 '8267': 'R',
 '14811': 'R',
 '15585': 'L',
 '9874': 'R',
 '16426': 'R',
 '15730': 'L',
 '16997': 'R',
 '13836': 'R',
 '10681': 'R',
 '9927': 'L',
 '15274': 'L',
 '4949': 'R',
 '15983': 'R',
 '15640': 'R',
 '5297': 'S',
 '19352': 'R',
 '14344': 'L',
 '18171': 'L',
 '15191': 'R',
 '15519': 'R',
 '14894': 'L',
 '19635': 'R',
 '12533': 'R',
 '16505': 'R',
 '11445': 'R',
 '5254': 'S',
 '9112': 'R',
 '17128': 'R',
 '13367': 'R',
 '13499': 'R',
 '15941': 'R',
 '15535': 'R',
 '14130': 'L',
 '16542': 'L',
 '8203': 'L',
 '18042': 'R',
 '14137': 'R',
 '15491': 'L',
 '9785': 'L',
 '13608': 'L',
 '17806': 'R',
 '19508': 'R',
 '14274': 'R',
 '13265': 'R',
 '12284': 'R',
 '5452': 'L',
 '16578': 'R',
 '19566': 'L',
 '7996': 'R',
 '19683': 'R',
 '18882': 'L',
 '13853': 'L',
 '14145': 'R',
 '15986': 'R',
 '15672': 'L',
 '14712': 'R',
 '11038': 'L',
 '15464': 'R',
 '3142': 'R',
 '3448': 'R',
 '16725': 'R',
 '785': 'R',
 '14388': 'L',
 '10322': 'S',
 '12282': 'L',
 '19294': 'R',
 '16512': 'R',
 '8709': 'R',
 '11863': 'R',
 '17838': 'L',
 '3174': 'L',
 '14128': 'L',
 '14453': 'R',
 '13620': 'R',
 '5000': 'L',
 '19918': 'L',
 '12547': 'R',
 '13419': 'R',
 '13329': 'L',
 '13324': 'S',
 '16211': 'R',
 '13613': 'S',
 '6153': 'S',
 '19734': 'L',
 '12147': 'R',
 '2136': 'L',
 '5227': 'L',
 '15124': 'R',
 '9241': 'R',
 '11200': 'L',
 '7739': 'R',
 '9134': 'R',
 '5361': 'L',
 '10950': 'R',
 '18360': 'R',
 '9393': 'L',
 '16556': 'S',
 '10459': 'R',
 '14950': 'S',
 '3298': 'R',
 '18314': 'R',
 '5352': 'S',
 '10324': 'R',
 '5930': 'L',
 '12225': 'L',
 '18401': 'R',
 '4922': 'L',
 '11609': 'R',
 '14968': 'S',
 '9308': 'R',
 '3473': 'L',
 '15429': 'R',
 '10542': 'L',
 '9776': 'L',
 '14593': 'R',
 '17919': 'S',
 '8392': 'L',
 '12979': 'R',
 '5751': 'R',
 '16478': 'L',
 '4940': 'L',
 '14109': 'R',
 '10200': 'L',
 '12510': 'R',
 '14813': 'R',
 '4314': 'L',
 '15170': 'L',
 '4892': 'L',
 '19293': 'R',
 '6609': 'S',
 '12552': 'R',
 '13167': 'S',
 '13590': 'L',
 '14738': 'R',
 '15482': 'R',
 '14843': 'R',
 '11737': 'R',
 '11470': 'L',
 '11680': 'R',
 '4316': 'L',
 '15112': 'L',
 '16885': 'R',
 '19262': 'R',
 '10030': 'R',
 '17907': 'R',
 '12564': 'R',
 '9777': 'R',
 '17954': 'L',
 '14350': 'L',
 '13744': 'L',
 '15794': 'R',
 '7859': 'L',
 '6887': 'R',
 '9433': 'R',
 '19198': 'R',
 '15937': 'R',
 '19844': 'S',
 '18345': 'L',
 '5417': 'R',
 '15402': 'R',
 '17620': 'R',
 '14162': 'R',
 '17678': 'R',
 '4106': 'L',
 '3892': 'L',
 '12856': 'R',
 '19556': 'L',
 '19197': 'R',
 '12158': 'R',
 '13301': 'L',
 '17710': 'L',
 '10472': 'R',
 '15998': 'L',
 '13757': 'R',
 '13624': 'L',
 '5235': 'R',
 '9256': 'R',
 '11899': 'L',
 '13611': 'R',
 '13658': 'R',
 '1433': 'R',
 '13755': 'R',
 '19251': 'R',
 '15653': 'L',
 '16219': 'R',
 '3269': 'L',
 '9810': 'R',
 '15362': 'L',
 '6848': 'R',
 '15518': 'R',
 '12927': 'L',
 '16376': 'L',
 '4022': 'S',
 '11339': 'R',
 '15181': 'R',
 '13723': 'R',
 '10346': 'R',
 '13132': 'R',
 '13145': 'S',
 '16909': 'L',
 '12937': 'R',
 '15223': 'L',
 '17696': 'R',
 '13369': 'R',
 '17326': 'S',
 '12950': 'R',
 '17642': 'L',
 '19326': 'S',
 '18721': 'R',
 '4866': 'L',
 '12907': 'L',
 'sa738696': 'R',
 '16403': 'S',
 '12976': 'R',
 '3516': 'L',
 '17982': 'R',
 '18036': 'L',
 '10815': 'S',
 '10951': 'L',
 '19709': 'R',
 '11493': 'R',
 '2967': 'R',
 '19698': 'R',
 '10047': 'R',
 '18839': 'L',
 '18564': 'L',
 '16953': 'R',
 '10264': 'L',
 '5409': 'S',
 '5827': 'R',
 '9929': 'R',
 '18399': 'L',
 '16530': 'R',
 '8623': 'R',
 '11602': 'S',
 '5343': 'L',
 '9368': 'R',
 '12649': 'L',
 '8252': 'R',
 '16153': 'R',
 '14854': 'L',
 '10199': 'S',
 '17719': 'L'}
id_arms_dict = {'19755': 'R',
 'sa737853': 'R',
 'sa828706': 'R',
 '14677': 'R',
 '14120': 'R',
 '15264': 'R',
 '18890': 'R',
 '8302': 'R',
 '15166': 'L',
 'sa876350': 'R',
 'sa873980': 'L',
 'sa915645': 'L',
 'sa3004810': 'L',
 '15507': 'R',
 '17664': 'R',
 '18332': 'R',
 '15256': 'R',
 'sa829037': 'R',
 'sa918414': 'L',
 '17282': 'R',
 '16137': 'L',
 'sa874080': 'L',
 '19261': 'R',
 '14353': 'L',
 'sa3006764': 'R',
 'sa602893': 'R',
 '20151': 'R',
 '14605': 'R',
 'sa738902': 'L',
 '5097': 'L',
 '13218': 'R',
 'sa3007593': 'L',
 'sa917924': 'R',
 'sa3008444': 'R',
 'sa829855': 'R',
 'sa656041': 'R',
 '19457': 'R',
 'sa918036': 'R',
 '9239': 'R',
 'sa659598': 'R',
 'sa874682': 'R',
 'sa874212': 'R',
 'sa3008613': 'R',
 '13287': 'R',
 '16155': 'L',
 'sa828674': 'L',
 '17170': 'L',
 'sa828695': 'R',
 '19459': 'R',
 '16511': 'L',
 'sa916944': 'R',
 '19853': 'R',
 '15100': 'R',
 'sa918091': 'R',
 '19959': 'L',
 '19343': 'L',
 '16164': 'R',
 'sa873274': 'R',
 'sa828669': 'R',
 'sa658943': 'R',
 '15306': 'L',
 'sa738964': 'R',
 '15066': 'R',
 '15514': 'R',
 '11836': 'R',
 '19268': 'R',
 'sa875837': 'R',
 'sa874099': 'L',
 '16507': 'R',
 '19256': 'R',
 'sa922200': 'R',
 '18251': 'R',
 'sa3006700': 'R',
 'sa918372': 'R',
 'sa3006775': 'R',
 '14706': 'L',
 '10343': 'L',
 '16128': 'R',
 '17035': 'L',
 '18325': 'L',
 '11487': 'L',
 'sa875783': 'R',
 '14391': 'L',
 'sa919113': 'R',
 '20387': 'L',
 'sa3005097': 'R',
 '16350': 'L',
 '27460': 'R',
 '4185': 'R',
 '6612': 'L',
 'sa829363': 'R',
 '20218': 'R',
 '9646': 'L',
 '10534': 'R',
 'sa738514': 'R',
 '14332': 'R',
 'sa918271': 'R',
 '3321': 'R',
 'sa829466': 'R',
 '11203': 'L',
 '9895': 'L',
 '19981': 'R',
 '12317': 'R',
 'sa828803': 'R',
 '14672': 'R',
 '9346': 'R',
 '16440': 'R',
 '13654': 'R',
 'sa919117': 'R',
 '15463': 'R',
 'sa874781': 'R',
 '15232': 'L',
 '16208': 'L',
 '4696': 'R',
 '19995': 'R',
 'sa738145': 'R',
 'sa658634': 'R',
 '3374': 'R',
 'sa828689': 'R',
 'sa3006525': 'R',
 '20030': 'L',
 'sa873701': 'L',
 'sa737454': 'R',
 'sa874797': 'R',
 'sa873550': 'R',
 'sa738458': 'R',
 'sa3008070': 'L',
 '8855': 'R',
 'sa829245': 'R',
 'sa917930': 'R',
 'sa915815': 'R',
 'sa874816': 'R',
 '16609': 'R',
 'sa737558': 'L',
 '19614': 'L',
 'sa549399': 'L',
 '16687': 'R',
 'sa873692': 'R',
 '20367': 'R',
 '17878': 'R',
 '18323': 'R',
 'sa880983': 'R',
 '7882': 'R',
 'sa828707': 'R',
 '13942': 'L',
 '15010': 'R',
 'sa658084': 'L',
 'sa829227': 'R',
 '14832': 'R',
 '15538': 'R',
 'sa738901': 'R',
 '11720': 'R',
 'sa918477': 'L',
 '16668': 'R',
 '11243': 'R',
 'sa708446': 'R',
 '12360': 'R',
 '16309': 'R',
 '15492': 'L',
 'sa829588': 'L',
 '10314': 'R',
 '6216': 'R',
 '17755': 'R',
 '13677': 'R',
 'sa830592': 'R',
 '15267': 'R',
 'sa3008225': 'R',
 '18403': 'R',
 'sa3004862': 'L',
 '1157': 'R',
 '8410': 'R',
 '15915': 'R',
 '9492': 'R',
 '8241': 'R',
 'sa827423': 'L',
 '14975': 'R',
 'sa875176': 'R',
 '16074': 'R',
 '17355': 'R',
 'sa875318': 'R',
 '5530': 'L',
 'sa856266': 'R',
 '27458': 'L',
 '16561': 'L',
 '15015': 'R',
 'sa874753': 'R',
 '18496': 'R',
 '2660': 'L',
 'sa3004708': 'R',
 'sa3004011': 'R',
 '9784': 'R',
 'sa3005068': 'L',
 '20039': 'L',
 '13435': 'R',
 '17169': 'L',
 '17292': 'R',
 '19569': 'R',
 'sa857694': 'R',
 'sa885967': 'R',
 '11896': 'R',
 'sa873314': 'R',
 '13533': 'R',
 '13346': 'L',
 '9013': 'R',
 '15130': 'R',
 '23293': 'R',
 '19541': 'R',
 'sa738926': 'R',
 '16840': 'R',
 '19456': 'R',
 '19789': 'R',
 '18873': 'R',
 '25377': 'R',
 '20379': 'R',
 '16432': 'L',
 'sa829552': 'R',
 'sa597862': 'R',
 '8337': 'L',
 '18152': 'L',
 '19444': 'R',
 'sa3004222': 'R',
 'sa3007591': 'L',
 '16647': 'R',
 '12249': 'L',
 'sa602914': 'L',
 '19264': 'R',
 '19266': 'R',
 '10058': 'L',
 '9794': 'L',
 '16233': 'R',
 '10061': 'L',
 '7115': 'R',
 '3164': 'L',
 '18699': 'R',
 '25007': 'L',
 '20631': 'R',
 'sa501597': 'R',
 '17606': 'R',
 '19582': 'R',
 '16418': 'L',
 '20447': 'R',
 '19866': 'R',
 'sa943525': 'R',
 '8350': 'R',
 'sa3005114': 'R',
 '16610': 'R',
 '6832': 'L',
 '15009': 'R',
 '10855': 'L',
 'sa3004923': 'R',
 '20368': 'R',
 '12917': 'R',
 '15423': 'L',
 '6797': 'R',
 '19867': 'R',
 '19447': 'L',
 '12022': 'R',
 '18356': 'R',
 '19911': 'L',
 '18790': 'L',
 '11801': 'R',
 '14719': 'R',
 '13360': 'R',
 '12800': 'R',
 '15451': 'R',
 '9456': 'L',
 '13403': 'R',
 '12323': 'R',
 '14288': 'R',
 '19200': 'L',
 '11964': 'R',
 '6562': 'R',
 '16269': 'L',
 '10836': 'R',
 '5221': 'L',
 '7608': 'L',
 '18361': 'R',
 '10430': 'R',
 '7803': 'L',
 '15684': 'R',
 '15832': 'L',
 '12857': 'R',
 '17586': 'L',
 '18600': 'R',
 '17734': 'R',
 '16130': 'R',
 '17796': 'R',
 '14101': 'R',
 '6902': 'L',
 '9132': 'R',
 '10160': 'R',
 '13164': 'L',
 '18076': 'L',
 '14825': 'R',
 '14862': 'R',
 '14272': 'L',
 '14151': 'R',
 '11428': 'R',
 '12863': 'R',
 '11632': 'R',
 '8004': 'R',
 '5615': 'R',
 '17237': 'R',
 '12997': 'R',
 '17359': 'L',
 '19848': 'L',
 '17677': 'L',
 '15605': 'R',
 '6576': 'R',
 '11528': 'L',
 '19495': 'R',
 '9434': 'L',
 '15474': 'R',
 '18525': 'R',
 '16400': 'R',
 '7448': 'L',
 '2859': 'L',
 '6661': 'R',
 '16258': 'L',
 '6483': 'R',
 '16159': 'L',
 '12297': 'R',
 '15690': 'R',
 'sa738581': 'R',
 '14552': 'R',
 '6033': 'R',
 '11691': 'R',
 '19427': 'R',
 '12808': 'R',
 '6632': 'R',
 '19979': 'R',
 '19479': 'R',
 '18555': 'L',
 '15846': 'R',
 '17018': 'R',
 '9111': 'L',
 '15594': 'R',
 '1514': 'L',
 '15288': 'R',
 '18064': 'R',
 '13763': 'R',
 '15091': 'R',
 '15440': 'L',
 '1994': 'R',
 '16207': 'R',
 '13475': 'L',
 '15821': 'R',
 '4505': 'R',
 '17735': 'L',
 '15761': 'R',
 '14814': 'R',
 '6399': 'R',
 '19677': 'L',
 '20020': 'R',
 '18311': 'R',
 '20430': 'R',
 '19814': 'R',
 '12575': 'L',
 '3542': 'L',
 '15734': 'R',
 '13619': 'R',
 '4869': 'L',
 '11713': 'R',
 '20041': 'R',
 'sa3008128': 'L',
 '14527': 'R',
 '15200': 'R',
 '6986': 'R',
 '14993': 'R',
 '7196': 'R',
 '10745': 'R',
 '15240': 'R',
 '18335': 'R',
 '14169': 'R',
 '19337': 'L',
 '18135': 'R',
 '14508': 'L',
 '15136': 'R',
 '14168': 'R',
 '18498': 'R',
 '4806': 'L',
 '8362': 'R',
 '6397': 'R',
 '5372': 'R',
 '23798': 'R',
 '19487': 'L',
 '13449': 'L',
 '9817': 'R',
 '13758': 'R',
 '6398': 'R',
 '5640': 'R',
 '15823': 'R',
 '18088': 'R',
 '12804': 'R',
 '15906': 'L',
 '7947': 'R',
 '15885': 'R',
 '13125': 'R',
 '11828': 'L',
 '15764': 'R',
 '7410': 'L',
 '19753': 'R',
 '10233': 'L',
 '1247': 'R',
 '3240': 'L',
 '15552': 'R',
 '16588': 'R',
 '13345': 'R',
 '2882': 'L',
 '11384': 'R',
 '9756': 'R',
 '21189': 'R',
 '14309': 'R',
 '7754': 'R',
 '15873': 'L',
 '12304': 'R',
 '16980': 'R',
 '14739': 'R',
 '3548': 'R',
 '4020': 'R',
 '6941': 'R',
 '5003': 'L',
 '15043': 'R',
 '3237': 'L',
 '14506': 'R',
 '13190': 'R',
 '12784': 'R',
 '15467': 'L',
 '20633': 'L',
 '17285': 'L',
 '20395': 'L',
 '16587': 'R',
 '19756': 'R',
 '14899': 'R',
 '8110': 'R',
 '3950': 'R',
 '13801': 'R',
 '13607': 'R',
 '17874': 'L',
 '13527': 'R',
 '18256': 'R',
 '18306': 'L',
 '4676': 'R',
 '13543': 'L',
 '14374': 'R',
 '16401': 'R',
 '16502': 'L',
 '20186': 'L',
 '19309': 'R',
 '17192': 'L',
 '18337': 'R',
 '17496': 'R',
 '9866': 'R',
 '17780': 'L',
 '8823': 'R',
 '17998': 'R',
 '12828': 'R',
 '10021': 'L',
 '2520': 'R',
 '10123': 'R',
 '7593': 'R',
 '18694': 'L',
 '2429': 'R',
 '5448': 'R',
 '13685': 'R',
 '17464': 'R',
 '17369': 'L',
 '3990': 'R',
 '17871': 'R',
 '15101': 'R',
 '17556': 'R',
 '12760': 'R',
 '14524': 'R',
 '7731': 'R',
 '5524': 'L',
 '11486': 'L',
 '19291': 'R',
 '11156': 'R',
 '16918': 'R',
 '18333': 'L',
 '17611': 'R',
 '19546': 'R',
 '12918': 'R',
 '5702': 'R',
 '12988': 'L',
 '17181': 'R',
 '2391': 'R',
 '19876': 'R',
 '16058': 'R',
 '11449': 'L',
 '15118': 'R',
 '18383': 'R',
 '13743': 'L',
 '16943': 'L',
 '19665': 'R',
 '2608': 'R',
 '16929': 'R',
 '4972': 'L',
 '10811': 'R',
 '19990': 'R',
 '4264': 'R',
 '8048': 'L',
 '10756': 'R',
 '11752': 'R',
 '11847': 'R',
 '9388': 'R',
 '18655': 'L',
 '17470': 'R',
 '13074': 'R',
 '4930': 'L',
 '12049': 'R',
 '4338': 'R',
 '11423': 'L',
 '13834': 'R',
 '17859': 'R',
 '6655': 'R',
 '14646': 'R',
 '11753': 'L',
 '9490': 'R',
 '11209': 'R',
 '12237': 'R',
 '16390': 'L',
 '10291': 'R',
 '12703': 'R',
 '15689': 'R',
 '12768': 'R',
 '13050': 'R',
 '8779': 'L',
 '16358': 'R',
 '13470': 'R',
 '17130': 'R',
 '14375': 'L',
 '14843': 'R',
 '4070': 'R',
 '13594': 'R',
 '13528': 'R',
 '10688': 'R',
 '13690': 'R',
 '14916': 'R',
 '15038': 'R',
 '16256': 'L',
 '15488': 'R',
 '17432': 'R',
 '14663': 'R',
 '7441': 'R',
 '13911': 'R',
 '12774': 'R',
 '14542': 'R',
 '13323': 'L',
 '18297': 'R',
 '15068': 'R',
 '16300': 'R',
 '1943': 'R',
 '16794': 'R',
 '17295': 'L',
 '18413': 'R',
 '8700': 'R',
 '15290': 'R',
 '13764': 'R',
 '7005': 'R',
 '12763': 'R',
 '16106': 'R',
 '5401': 'R',
 '13194': 'R',
 '494': 'R',
 '20329': 'R',
 '22449': 'R',
 '2036': 'L',
 '19374': 'R',
 '13273': 'R',
 '14765': 'L',
 '13781': 'L',
 '19716': 'R',
 '19388': 'R',
 '3096': 'R',
 '5420': 'R',
 '9761': 'R',
 '12572': 'R',
 '13394': 'R',
 '19349': 'L',
 '10843': 'L',
 '7550': 'L',
 '10591': 'L',
 '10954': 'R',
 '2717': 'R',
 '13361': 'L',
 '14078': 'R',
 '13431': 'R',
 '20302': 'L',
 '14710': 'R',
 '12447': 'R',
 '4301': 'L',
 '5114': 'R',
 '13696': 'R',
 '13892': 'R',
 '6627': 'R',
 '9325': 'R',
 '17811': 'L',
 '13804': 'R',
 '12970': 'R',
 '16977': 'R',
 '17594': 'R',
 '15291': 'L',
 '4141': 'L',
 '10354': 'L',
 '17578': 'R',
 '13761': 'R',
 '14696': 'R',
 '13182': 'R',
 '13549': 'R',
 '11903': 'R',
 '11604': 'R',
 '19558': 'R',
 '18792': 'L',
 '13649': 'R',
 '16265': 'L',
 '20099': 'R',
 '13183': 'R',
 '17186': 'R',
 '19320': 'L',
 '19312': 'R',
 '9073': 'R',
 '7274': 'R',
 '14771': 'R',
 '11426': 'L',
 '13799': 'L',
 '16814': 'L',
 '14682': 'R',
 '16306': 'R',
 '20100': 'R',
 '6893': 'R',
 '3254': 'R',
 '11760': 'L',
 '14107': 'R',
 '17995': 'R',
 '12880': 'L',
 '18300': 'L',
 '14509': 'R',
 '19199': 'L',
 '3132': 'L',
 '15541': 'R',
 '15046': 'R',
 '14295': 'L',
 '19453': 'R',
 '20023': 'R',
 '17264': 'R',
 '19231': 'L',
 '14966': 'R',
 '13397': 'L',
 '18282': 'L',
 '6249': 'R',
 'sa550615': 'R'}

In [6]:
class Batter:
    """Class that holds information for a Batter.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L, singles_R, 
                 doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R, sb_rate, sb_success, bsr, fld):
        """Initializes values for this class
        
        Args:
            team: team name of batter
            name: batter name
            hand: Batting hand of hitter(L, R, or S)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            hbp_L: fraction of plate appearances ending in a hbp vs. lefties
            so_L: fraction of plate appearances ending in a strikeout vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            hbp_R: fraction of plate appearances ending in a hbp vs. righties
            so_R: fraction of plate appearances ending in a strikeout vs. righties
            sb_rate: approximate percentage of time player will steal base given opportunity
            sb_success: fraction of stolen base attempts ending in success
            bsr: fangraphs stat representing projected runs created by a base runner. Higher number is better
            fld: bsr: fangraphs stat representing projected runs created by a fielder. higher number is better
        """
        self.team = team
        self.name = name
        self.hand = hand
        self.singles_L = singles_L
        self.doubles_L = doubles_L
        self.triples_L = triples_L
        self.home_runs_L = home_runs_L
        self.walks_L = walks_L
        self.hbp_L = hbp_L
        self.so_L = so_L
        self.slg_L = walks_L+singles_L+2*doubles_L+3*triples_L+4*home_runs_L
        self.singles_R = singles_R
        self.doubles_R = doubles_R
        self.triples_R = triples_R
        self.home_runs_R = home_runs_R
        self.walks_R = walks_R
        self.hbp_R = hbp_R
        self.so_R = so_R
        self.slg_R = walks_R+singles_R+2*doubles_R+3*triples_R+4*home_runs_R
        self.sb_rate = sb_rate
        self.sb_success = sb_success
        self.bsr = bsr
        self.fld = fld
        
        
        #self.cum values represent cumulative (season) totals for a batter
        self.cum_singles_L = 0
        self.cum_doubles_L = 0
        self.cum_triples_L = 0
        self.cum_home_runs_L = 0
        self.cum_walks_L = 0
        self.cum_hbp_L = 0
        self.cum_so_L = 0
        self.cum_pa_L = 0
        self.cum_singles_R = 0
        self.cum_doubles_R = 0
        self.cum_triples_R = 0
        self.cum_home_runs_R = 0
        self.cum_walks_R = 0
        self.cum_hbp_R = 0
        self.cum_so_R = 0
        self.cum_pa_R = 0
        self.cum_runs = 0
        self.cum_rbi = 0
        self.cum_sb = 0
        self.cum_cs = 0
        self.cum_sf = 0
        self.cum_sh = 0
        self.cum_gdp = 0
        self.cum_G = 0
        self.cum_GS = 0
     
    def hits(self):
        """cumulative hits of a batter
        
        Returns:
            int: number of cumulative hits
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L+
               self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf+self.cum_sh)>0):
            return(round(float((self.hits())/((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf))), 3))
        else:
            return 0    
        
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)>0):
            return(round((self.hits()+(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R))/(self.cum_pa_L+self.cum_pa_R), 3))
        else:
            return 0
        
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf)>0):
            return(round(((self.cum_singles_L+self.cum_singles_R)+2*(self.cum_doubles_L+self.cum_doubles_R)+
                           3*(self.cum_triples_L+self.cum_triples_R)+4*(self.cum_home_runs_L+self.cum_home_runs_R))/
                           ((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf+self.cum_sh)), 3))
        else:
            return 0
        
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form
            """
        return(self.slg() + self.obp())
    
    def strikeouts(self):
        """Number of strikeouts of a batter
        
        Returns:
            float: total strikeouts
        """
        return(self.cum_so_L+self.cum_so_R)
    
    def print_stats(self):
        """Prints some stats of a player
        """
        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str((self.cum_home_runs_L+self.cum_home_runs_R)))
              
    def hits_L(self):
        """cumulative hits of a batter against lefties
        
        Returns:
            int: number of hits of batter agianst lefties
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L)
    
    def avg_L(self):
        """Calculates avg of a player against lefties
        
        Returns:
            float: batting average in decimal form of batter against lefties
        """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(float((self.hits_L())/(self.cum_pa_L-self.cum_walks_L)), 3))
        else:
            return 0    
              
    def obp_L(self):
        """Calculates on base percentage of a player against lefties
        
        Returns:
            float: opb in decimal form of batter against lefties
        """
        if(self.cum_pa_L>0):
            return(round((self.hits_L()+self.cum_walks_L)/self.cum_pa_L, 3))
        else:
            return 0
              
    def slg_L(self):
        """Calculates slugging percentage of a player against lefties
        
        Returns:
            float: slg in decimal form of batter against lefties
            """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(((self.cum_singles_L+2*self.cum_doubles_L+
                           3*self.cum_triples_L+4*self.cum_home_runs_L))/
                           (self.cum_pa_L-self.cum_walks_L), 3))
        else:
            return 0
              
    def ops_L(self):
        """Calculates on base plus slugging of a player against lefties
        
        Returns:
            float: ops in decimal form of batter against lefties
        """
        return(self.slg_L() + self.obp_L())
              
    def hits_R(self):
        """Cumulative hits of a batter against righties
        
        Returns:
            int: number of hits of batter against righties
        """
        return(self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg_R(self):
        """Calculates avg of a player against righties
        
        Returns:
            float: batting average in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(float((self.hits_R())/(self.cum_pa_R-self.cum_walks_R)), 3))
        else:
            return 0    
              
    def obp_R(self):
        """Calculates on base percentage of a player against righties
        
        Returns:
            float: opb in decimal form of batter against righties
        """
        if(self.cum_pa_R>0):
            return(round((self.hits_R()+self.cum_walks_R)/self.cum_pa_R, 3))
        else:
            return 0
              
    def slg_R(self):
        """Calculates slugging percentage of a player against righties
        
        Returns:
            float: slg in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(((self.cum_singles_R2*self.cum_doubles_R+
                           3*self.cum_triples_R+4*self.cum_home_runs_R))/
                           (self.cum_pa_R-self.cum_walks_R), 3))
        else:
            return 0
              
    def ops_R(self):
        """Calculates on base plus slugging of a player against righties
        
        Returns:
            float: ops in decimal form of batter against righties
        """
        
        return(self.slg_R() + self.obp_R())
              
class Pitcher(Batter):
    """Class that holds information for a pitcher. Many attributes are similar to those from the Batter class.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L, singles_R,
                 doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R, sb_rate, sb_success, bsr, fld):
        """Initializes values for this class
        
        Args:
            team: team name of pitcher
            name: pitcher name
            hand: Pitching arm of pitcher(L, R)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            hbp_L: fraction of plate appearances ending in a hbp vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            hbp_R: fraction of plate appearances ending in a hbp vs. righties
            sb_rate: approximate percentage of time opposing player will steal base given opportunity (set to 0)
            sb_success: fraction of opponent stolen base attempts ending in success (set to 0)
            bsr: not relevant (set to 0)
            fld: not relevant (set to 0)
        """
        
        Batter.__init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L,
                        singles_R, doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R, sb_rate, sb_success, 0, 0)
        self.cum_outs_L = 0
        self.cum_outs_R = 0
        self.cum_wp = 0
        self.current_bf = 0
        self.current_outs = 0
        self.current_runs = 0
        self.finished_half = False
        self.days_off = 100
        self.consecutive_days = 0
    
    def cum_outs(self, hand):
        """Cumulative outs of a pitcher against batters of certain handedness
        
        Args:
            hand: handedness of batter pitcher is facing
        Returns:
            float: value for cumulative outs
        """
        if(hand=='L'):
            return self.cum_outs_L
        elif(hand=='R'):
            return self.cum_outs_R
    
    def ip(self):
        """Calculates innings pitched of a pitcher
        
        Returns:
            float: Innings pitched in normal form(ie. 5.1 for 5 and a third innings pitched)
        """
        ip = (self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3.0
        ip = math.modf(ip)
        ip = round((ip[0]/3.33333 + ip[1]), 2)
        return(ip)  
    
    def era(self):
        """Calculates ERA of a pitcher
        
        Returns:
            float: ERA in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((27*self.cum_runs/(self.cum_outs_L+self.cum_outs_R+self.cum_cs)), 2))
    
    def whip(self):
        """Calculates WHIP of a pitcher
        
        Returns:
            float: WHIP in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((((self.cum_walks_L+self.cum_walks_R)+self.hits())/
                          ((self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3)), 2))
        
class Team:
    """Class that holds information for a team.
    
    Attributes:

    """
    def __init__(self, city, league, batter_df, pitcher_df, batter_stats, pitcher_stats, starters, lineup_dict=None, sp=None):
        """Initializes values for this class
        
        Args:
            city: city of team (ex. 'Atlanta')
            league: league of team ('AL' or 'NL')
            batter_df: dataframe of batters on a team and their 2019 stats
            pitcher_df: dataframe of pitchers on a team and their 2019 stats
            starters: dict with names of batters in projected starting lineup, rotation, and the closer
            relievers_df: dataframe with relievers of a team and their projected innings pitched
        """
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.batter_dict= {}
        self.pitcher_df = pitcher_df
        self.pitcher_dict = {}
        self.starters = starters
        self.lineup_dict = lineup_dict
        print(self.lineup_dict)
        self.sp = sp
        print(self.sp)
        self.pitching_staff = pitcher_df
        self.rotation = [None]*5
        self.closer = None
        self.current_pitcher_list = []
        self.available_pitchers_list = []
        self.batting_pitcher = None
        self.current_pitcher = None
        self.batting_index = 0
        self.rotation_index = 0
        self.runners = [None, None, None, 0]    # first base-home plate. 0 for base empty, 1 for runner on base
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        self.pitchers = []
        self.batters = []
        self.runners_dict  = {}
        self.game_no = 0
        avg_stats = self.batter_df.loc[batter_df['Name']=='Avg_Totals']
        self.avg_batter_stats = avg_stats

        avg_singles_L = avg_stats['1B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_doubles_L = avg_stats['2B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_triples_L = avg_stats['3B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_home_runs_L = avg_stats['HR_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_walks_L = avg_stats['BB_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_hbp_L = avg_stats['HBP_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_so_L = avg_stats['SO_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_singles_R = avg_stats['1B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_doubles_R = avg_stats['2B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_triples_R = avg_stats['3B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_home_runs_R = avg_stats['HR_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_walks_R = avg_stats['BB_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_hbp_R = avg_stats['HBP_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_so_R = avg_stats['SO_R'].values[0]/avg_stats['PA_R'].values[0]
        batter_df = batter_df[batter_df['Name']!='Avg_Totals']

        for index, batter in batter_df.iterrows():   # adding values to each batter and instantiating Batter class
            singles_L = 2*batter['1B_L']/batter['PA_L']-avg_singles_L
            doubles_L = 2*batter['2B_L']/batter['PA_L']-avg_doubles_L
            triples_L = 2*batter['3B_L']/batter['PA_L']-avg_triples_L
            home_runs_L = 2*batter['HR_L']/batter['PA_L']-avg_home_runs_L
            walks_L = 2*batter['BB_L']/batter['PA_L']-avg_walks_L
            hbp_L = 2*batter['HBP_L']/batter['PA_L']-avg_hbp_L
            so_L = 2*batter['SO_L']/batter['PA_L']-avg_so_L
            singles_R = 2*batter['1B_R']/batter['PA_R']-avg_singles_R
            doubles_R = 2*batter['2B_R']/batter['PA_R']-avg_doubles_R
            triples_R = 2*batter['3B_R']/batter['PA_R']-avg_triples_R
            home_runs_R = 2*batter['HR_R']/batter['PA_R']-avg_home_runs_R
            walks_R = 2*batter['BB_R']/batter['PA_R']-avg_walks_R
            hbp_R = 2*batter['HBP_R']/batter['PA_R']-avg_hbp_R
            so_R = 2*batter['SO_R']/batter['PA_R']-avg_so_R
            
            sb_rate = round((batter['SB_L']+batter['SB_R']+batter['CS_L']+batter['CS_R'])/
                            (batter['1B_L']+batter['1B_R']+batter['BB_L']+batter['BB_R']), 3)
            if(batter['CS_L']+batter['CS_R'])==0:
                sb_success = 0
            else:
                sb_success = ((batter['SB_L']+batter['SB_R'])/
                             (batter['SB_L']+batter['SB_R']+batter['CS_L']+batter['CS_R']))

            current_batter = Batter(self.city, batter['Name'], batter['Bats'], singles_L, doubles_L, triples_L, 
                                    home_runs_L, walks_L, hbp_L, so_L, singles_R, doubles_R, triples_R, 
                                    home_runs_R, walks_R, hbp_R, so_R, sb_rate, sb_success, batter['BsR'], batter['Fld'])
            self.batters.append(current_batter)
            self.batter_dict.update({batter['Name']: current_batter})
            
        avg_stats = self.pitcher_df.loc[pitcher_df['Name']=='Avg_Totals']
        self.avg_pitcher_stats = avg_stats
        
        avg_singles_L = avg_stats['1B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_doubles_L = avg_stats['2B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_triples_L = avg_stats['3B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_home_runs_L = avg_stats['HR_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_walks_L = avg_stats['BB_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_hbp_L = avg_stats['HBP_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_so_L = avg_stats['SO_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_singles_R = avg_stats['1B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_doubles_R = avg_stats['2B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_triples_R = avg_stats['3B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_home_runs_R = avg_stats['HR_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_walks_R = avg_stats['BB_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_hbp_R = avg_stats['HBP_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_so_R = avg_stats['SO_R'].values[0]/avg_stats['TBF_R'].values[0]
        pitcher_df = pitcher_df[pitcher_df['Name']!='Avg_Totals']
        
        for index, pitcher in pitcher_df.iterrows(): # adding values to each pitcher and instantiating Pitcher class
            singles_L = 2*pitcher['1B_L']/pitcher['TBF_L']-avg_singles_L
            doubles_L = 2*pitcher['2B_L']/pitcher['TBF_L']-avg_doubles_L
            triples_L = 2*pitcher['3B_L']/pitcher['TBF_L']-avg_triples_L
            home_runs_L = 2*pitcher['HR_L']/pitcher['TBF_L']-avg_home_runs_L
            walks_L = 2*pitcher['BB_L']/pitcher['TBF_L']-avg_walks_L
            hbp_L = 2*pitcher['HBP_L']/pitcher['TBF_L']-avg_hbp_L
            so_L = 2*pitcher['SO_L']/pitcher['TBF_L']-avg_so_L
            singles_R = 2*pitcher['1B_R']/pitcher['TBF_R']-avg_singles_R
            doubles_R = 2*pitcher['2B_R']/pitcher['TBF_R']-avg_doubles_R
            triples_R = 2*pitcher['3B_R']/pitcher['TBF_R']-avg_triples_R
            home_runs_R = 2*pitcher['HR_R']/pitcher['TBF_R']-avg_home_runs_R
            walks_R = 2*pitcher['BB_R']/pitcher['TBF_R']-avg_walks_R
            hbp_R = 2*pitcher['HBP_R']/pitcher['TBF_R']-avg_hbp_R
            so_R = 2*pitcher['SO_R']/pitcher['TBF_R']-avg_so_R

            current_pitcher = Pitcher(self.city, pitcher['Name'], pitcher['Throws'], singles_L, doubles_L, triples_L, 
                                  home_runs_L, walks_L, hbp_L, so_L, singles_R, doubles_R, triples_R, 
                                  home_runs_R, walks_R, hbp_R, so_R, 0, 0, 0, 0)
            
            
            self.pitchers.append(current_pitcher)
            self.pitcher_dict.update({pitcher['Name']: current_pitcher})
           
            if(current_pitcher.name==self.starters['closer']):    # Assigning the team's projected closer to self.closer
                self.closer = current_pitcher

    def make_lineup(self, batters, pitcher, league, lineup=None):
        """Making a lineup of batters for a givevn game based on Fangraphs projected plate appearances

        Args:
            batters: dataframe of batters
            pitcher: opposing teams pitcher
            league: league of team
            lineup: lineup if reading in actual game
            starters: dict with names in projected starting lineup
        Returns:
            lineup: list of Batter instances in order of projected lineup
        """
        
        lineup = []
        if(self.lineup_dict!=None):
            batter_df = self.batter_df[self.batter_df['Name']!='Avg_Totals']
            bench = []
            for name in self.lineup_dict.keys():
                if(name in self.batter_dict.keys()):
                    batter = self.batter_dict[name]
                else:
                    batter = Batter(self.city, name, 'R', .13, .04, .01, .02, 
                                              .03, .25, .13, .04, .01, .02, .03, .25, .02, .01, 0, 0, 0, 0)
                lineup.append(batter)
        else:
            success = False
            while(not success):
                batter_df = self.batter_df[self.batter_df['Name']!='Avg_Totals']
                unordered_lineup = []
                unordered_df = pd.DataFrame(columns = ['Batter', 'obp', 'slg', 'ops'])
                lineup = [None]*9
                bench = []
                if(league=='NL'):
                    pos_list = ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']
                else:
                    pos_list = ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']

                for pos in pos_list:   # Iterating through each position to get a player to be in the lineup
                    num = random.randint(0, self.avg_batter_stats[pos].values[0])
                    addition = False
                    total = 0

                    while(not addition and total<self.avg_batter_stats[pos].values[0]):
                        for index, row in batter_df.iterrows():
                            batter = self.batter_dict[row['Name']]
                            if(row[pos]+total>num and batter not in unordered_lineup):
                                unordered_lineup.append(batter)
                                
                                if(pitcher.hand=='L'):
                                    unordered_df = unordered_df.append(pd.Series([batter, batter.walks_L, batter.slg_L, batter.walks_L+batter.slg_L], index=unordered_df.columns), ignore_index=True)
                                else:
                                    unordered_df = unordered_df.append(pd.Series([batter, batter.walks_R, batter.slg_R, batter.walks_R+batter.slg_R], index=unordered_df.columns), ignore_index=True)

                                addition=True
                                break
                            else:
                                total+=row[pos]
                                continue


                if(len(unordered_lineup)==len(pos_list)):    # If lineup doesn't have enough batters, restart
                    success = True

            for num in range(1,3):
                first_ops = unordered_df.loc[unordered_df['ops'].idxmax()]['Batter']
                lineup[num] = first_ops
                unordered_df = unordered_df[unordered_df['Batter']!=first_ops]

            first_obp = unordered_df.loc[unordered_df['obp'].idxmax()]['Batter']
            lineup[0] = first_obp
            unordered_df = unordered_df[unordered_df['Batter']!=first_obp]

            for num in range(3,len(pos_list)):
                first_ops = unordered_df.loc[unordered_df['ops'].idxmax()]['Batter']
                lineup[num] = first_ops
                unordered_df = unordered_df[unordered_df['Batter']!=first_ops]

            """
            #if(league=='NL'): # Commented out becuase no DH this year in national league
                #if(self.batting_pitcher==None):
                    #self.batting_pitcher = Batter(self.city, 'pitcher', 'R', .06, .01, -.05, -.03, -.03, 0, .5, .06, .01, 
                                                  -.05, -.03, -.03, 0, .5, 0, 0, -5, -5)
                    #lineup[8] = self.batting_pitcher
                    #self.batters.append(self.batting_pitcher)
                #else:
                    #lineup[8] = self.batting_pitcher """
            
        if(league=='AL' or league=='NL'):
            # If not all batters in projected lineup have data, create a batter to be in the lineup
            while(len(lineup)<9):
                lineup.append(Batter(self.city, 'extra' +str(len(lineup)), 'R', .13, .04, .01, .02, 
                                              .03, .25, .13, .04, .01, .02, .03, .25, .02, .01, 0, 0, 0, 0))             
        for batter in lineup:
            batter.cum_G+=1
            batter.cum_GS+=1
        
        for index, row in batter_df.iterrows():
            batter = self.batter_dict[row['Name']]
            if(batter not in lineup):
                bench.append(batter)
                

        return lineup, bench
    
    def choose_pitcher(self, outs, inning, runs, opp_runs, hitting_lineup, hitting_lineup_index, pitchers_lineup, pitchers_lineup_index):
        """Returns a pitcher to pitch for a team given game attributes
        
            Args:
                outs: current number of outs in inning
                runs: number of runs team has
                opp_runs: number of runs opposing team has
                hitting_lineup: batting team's lineup
                hitting_lineup_index: place in lineup of batting team
                pitchers_lineup: fielding team's opposing team's lineup
                pitchers_lineup_index: place in lineup of fielding team
            Returns:
                instance of Pitcher class to pitch in game
            """
        remove_pitcher = False
        self.available_pitchers_list = []
        for pitcher in self.pitchers:  # Adding pitchers who have not already pitched to a list
            if pitcher not in self.current_pitcher_list and pitcher not in self.available_pitchers_list:
                self.available_pitchers_list.append(pitcher)
            
        # If Pitcher hasn't faced the minimum of 3 batters or finished a hlaf inning, they remain in game
        if(self.current_pitcher.current_bf<3 and self.current_pitcher.finished_half==False): 
            return self.current_pitcher
        
        # Creating variable total that uses game factors to determine if pitcher should get taken out
        total = ((self.current_pitcher.current_bf-self.current_pitcher.current_outs)*3 + self.current_pitcher.current_outs+self.current_pitcher.current_runs*3)

        if(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3.0>0):
            total+=(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3)*2

        if(self.current_pitcher.hand==hitting_lineup[hitting_lineup_index].hand):   # If batter and pitcher have same handedness
            total-=3
            
        if(self.current_pitcher.finished_half):  # If pitcher has already finished half inning
            total+=2
        
        if(outs==0 and (self.current_pitcher in self.rotation or self.current_pitcher.name==self.sp)):    # If no outs yet in inning
            total+=5
            
        if(pitchers_lineup_index>4 and pitchers_lineup[8].name=='pitcher'):  # If pitcher is in lineup, spot is almost up
            total+=6
        
        if(total<42):
            num = random.uniform(0, (60-total)*100)
        else:
            num = random.uniform(0, (90-total)*4)
            
        if(num<total):
            remove_pitcher = True
        
        # Leave current potcher in probabilistically
        if(not remove_pitcher and (self.current_pitcher in self.rotation or self.current_pitcher.name==self.sp) or total<=8 and self.current_pitcher not 
           in self.rotation):
            return self.current_pitcher
        else:  # If current pitcher has total value greater than threshold, find new reliever
            
            if(inning==8 and outs>0 and self.closer!=None and self.closer not in self.current_pitcher_list 
               and runs-opp_runs>0 and runs-opp_runs<3): # Put closer in game if game is close and in late innings
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            elif(inning>8 and self.closer!=None and self.closer not in self.current_pitcher_list and runs-opp_runs>0 and runs-opp_runs<4):
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            else:
                # If game not close or game not in late innings, put non-closer reliever in game
                
                left_count = 0    # Number of lefties out of next three batters
                right_count = 0   # Number of righties out of next three batters

                for num in range(3):
                    if(hitting_lineup[(hitting_lineup_index+num)%9].hand=='L'):
                        left_count+=(1+outs-num)
                    elif(hitting_lineup[(hitting_lineup_index+num)%9].hand=='R'):
                        right_count+=(1+outs-num)

                if(left_count>right_count):  # If more of the next 3 batters are lefties, hand_indicator = 0
                    hand_indicator = 'L'
                else:                        # If more or equal of the next 3 batters are righties, hand_indicator = 1
                    hand_indicator = 'R'
                
                
                pitcher_df = self.pitcher_df[self.pitcher_df['Name']!='Avg_Totals']
        
                
                reliever_chosen = False
                wrong_hand = 0
                attempts = 0
                while(attempts<200):    # Trying to find a reliever
                    num = random.randint(0, self.avg_pitcher_stats['RP'].values[0])
                    total = 0
                    if(attempts<50):
                        days_off_threshold = 4
                    elif(attempts<100):
                        if(self.closer in self.available_pitchers_list and self.closer!=None):
                            self.current_pitcher = self.closer
                            self.current_pitcher.cum_G+=1
                            self.current_pitcher_list.append(self.current_pitcher)
                            return self.current_pitcher
                        
                        days_off_threshold = 7
                    else:
                        days_off_threshold = 10
                    
                    for index, row in pitcher_df.iterrows():
                        pitcher = self.pitcher_dict[row['Name']]
                        if(pitcher in self.available_pitchers_list and pitcher!=self.closer):
                            if(row['RP']+total>num and pitcher.consecutive_days<days_off_threshold):
                                if(pitcher.hand==hand_indicator or wrong_hand>1):
                                    self.current_pitcher = pitcher
                                    self.current_pitcher.cum_G+=1
                                    self.current_pitcher_list.append(self.current_pitcher)
                                    return self.current_pitcher
                                if(wrong_hand<2):    # If pitcher chosen doesn't have same hand as most upcoming batters, will try again
                                    wrong_hand+=1
                                    attempts+=1
                                    break
                            else:
                                total+=row['RP']
                                continue
                        else:
                            attempts+=1

            if(self.closer in self.available_pitchers_list and self.closer!=None):
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            else:
                pitcher_chosen = False
                attempts = 0
                while(not pitcher_chosen and len(self.available_pitchers_list)>0 and attempts<100):
                    num = random.randint(0, len(self.available_pitchers_list)-1)
                    pitcher = self.available_pitchers_list[num]
                    if(pitcher in self.rotation and attempts<30 and pitcher.days_off<3):
                        attempts+=1
                    elif(pitcher in self.rotation and attempts<60 and pitcher.days_off<2):
                        attempts+=1
                    elif(pitcher in self.rotation and attempts<100 and pitcher.days_off<1):
                        attempts+=1
                    else:
                        self.current_pitcher = pitcher
                        self.current_pitcher.cum_G+=1
                        self.current_pitcher_list.append(self.current_pitcher)
                        return self.current_pitcher

        return self.current_pitcher
    
    def choose_starter(self):
        """Chooses a starting pitcher for a team
        """
        print(self.sp)
        if(self.sp!=None):
            if(self.sp in self.pitcher_dict.keys()):
                pitcher = self.pitcher_dict[self.sp]
            else:
                pitcher = Pitcher(self.city, self.sp, 'R', .1, .05, .01, 
                                  .03, .1, .01, .15, .1, .05, .01, 
                                  .03, .1, .01, .15, 0, 0, 0, 0)
            print('starter is :' + str(pitcher.name))   
            pitcher.cum_G+=1
            pitcher.cum_GS+=1
            return pitcher
        
        pitcher_df = self.pitcher_df[self.pitcher_df['Name']!='Avg_Totals']
        pitcher_df.sort_values(by=['SP'], ascending=False, inplace=True)
        
        if(self.game_no<6):    # If early in season, pick starter based on projected innings pitched
            for index, row in pitcher_df.iterrows():
                if(index>=self.game_no-1):
                    pitcher = self.pitcher_dict[row['Name']]
                    if(pitcher.days_off>=4):
                        starter_chosen=True
                        pitcher.cum_G+=1
                        pitcher.cum_GS+=1
                        self.rotation[self.rotation_index] = pitcher
                        return pitcher

        starter_chosen = False

        while(not starter_chosen):
            supposed_starter = self.rotation[self.rotation_index]
            starts_projected = int(pitcher_df[pitcher_df['Name']==supposed_starter.name]['SP']/5.5)
            n = random.randint(0, 40)
            if(n<starts_projected):
                    supposed_starter.cum_G+=1
                    supposed_starter.cum_GS+=1
                    self.rotation[self.rotation_index] = supposed_starter
                    return supposed_starter
            else:
                total = 0
                num = random.randint(0, self.avg_pitcher_stats['SP'].values[0])
                for index, row in pitcher_df.iterrows():
                    pitcher = self.pitcher_dict[row['Name']]
                    if(row['SP']+total>num and pitcher.days_off>=4):
                        pitcher.cum_G+=1
                        pitcher.cum_GS+=1
                        self.rotation[self.rotation_index] = pitcher
                        return pitcher
                    else:
                        total+=row['SP']
                        continue

    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started
        """
        for pitcher in self.current_pitcher_list:
            pitcher.consecutive_days+=1
            pitcher.days_off = 0
            pitcher.current_bf = 0
            pitcher.current_outs = 0
            pitcher.current_runs = 0
            pitcher.finished_half = False
        
        for pitcher in self.available_pitchers_list:
            pitcher.days_off+=1
            pitcher.consecutive_days = 0

        self.runners_dict = {}    
        self.current_pitcher_list = []   
        self.batting_index = 0
        self.current_pitcher = None
        self.starter_pitching = True
        self.runners = [None, None, None, 0]
        self.runs = 0
        
    def reset_runners(self):
        """Resets baserunners after each half inning
        """
        self.runners = [None, None, None, 0]
    
    def cumul_stats_df(self):
        """Class that returns dataframes of stats for batters and pitchers
    
        Returns:
            batter_df: dataframe of batter stats
            pitcher_df: dataframe of pitcher stats
        """
        batter_df = pd.DataFrame(columns=['Name', 'G', 'PA', 'AB', 'H', '1B', '2B', '3B', 'HR', 'BB', 'HBP', 'SF', 
                                          'SH', 'K', 'AVG', 'OBP', 'SLG', 'OPS', 'R', 'RBI', 'SB', 'CS'])
        
        for b in self.batters:
            batter_values = [b.name, b.cum_G, b.cum_pa_L+b.cum_pa_R, b.cum_pa_L+b.cum_pa_R-
                             (b.cum_walks_L+b.cum_walks_R+b.cum_hbp_L+b.cum_hbp_R+b.cum_sf), 
                             b.hits(), b.cum_singles_L+b.cum_singles_R, b.cum_doubles_L+b.cum_doubles_R, 
                             b.cum_triples_L+b.cum_triples_R, b.cum_home_runs_L+b.cum_home_runs_R, 
                             b.cum_walks_L+b.cum_walks_R, b.cum_hbp_L+b.cum_hbp_R, b.cum_sf, b.cum_sh, b.strikeouts(), b.avg(), b.obp(), 
                             b.slg(), b.ops(), b.cum_runs, b.cum_rbi, b.cum_sb, b.cum_cs]
            batter_df = batter_df.append(pd.Series(batter_values, index=batter_df.columns), ignore_index=True)
        batter_df.sort_values(by=['PA'], ascending=False, inplace=True)
        
        pitcher_df = pd.DataFrame(columns=['Name', 'G', 'GS', 'IP', 'K', 'H', 'BB', 'HBP', 'ERA', 'WHIP', 'HR',
                                           'AVG', 'OBP', 'SLG', 'OPS', 'R', 'SB', 'CS', 'WP'])
        
        for p in self.pitchers:
            pitcher_values = [p.name, p.cum_G, p.cum_GS, p.ip(), p.strikeouts(), p.hits(), p.cum_walks_L+p.cum_walks_R, 
                              p.cum_hbp_L+p.cum_hbp_R, p.era(), p.whip(), p.cum_home_runs_L+p.cum_home_runs_R, p.avg(), 
                              p.obp(), p.slg(), p.ops(), p.cum_runs, p.cum_sb, p.cum_cs, p.cum_wp]
            pitcher_df = pitcher_df.append(pd.Series(pitcher_values, index=pitcher_df.columns), ignore_index=True)
        pitcher_df.sort_values(by=['IP'], ascending=False, inplace=True) 
        
        return batter_df, pitcher_df
    
class Game:
    """Class that holds information for a Game.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team
                                
    def get_sb_wp_result(self, runners, pitcher, team, outs, num):
        """stolen base result of a batter
        
        Args:
            runners: list of runners on base
            pitcher: pitcher currently pitching
            team: batting team
            outs: number of outs
            num: random number used to determine whether or not batter should steal base
        Returns:
            result of stolen base attempt (or None if no attempt)
        """
        if(runners[0]!=None and runners[1]==None):   # If first base occupied and second base not, steal possible
            if(num>runners[0].sb_rate/1.2):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[0].sb_success):  # Successful stolen base attempt
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[0].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[0] = None
                    return "out"
        elif(runners[0]!=None and runners[1]!=None):   # If first and second base occupied
            if(num>runners[0].sb_rate/5):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[1].sb_success):  # Successful stolen base attempt
                    runners[1].cum_sb+=1
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=2
                    runners[2] = runners[1]
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[1].cum_cs+=1
                    pitcher.cum_cs+=1
                    if(outs<2):
                        runners[0].cum_sb+=1
                        pitcher.cum_sb+=1
                        runners[1] = runners[0]
                        
                    runners[0] = None
                    return "out"
        elif(runners[0]!=None and runners[1]!=None):   # If second base occupied
            if(num>runners[0].sb_rate/5):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[1].sb_success):  # Successful stolen base attempt
                    runners[1].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[2] = runners[1]
                    runners[1] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[1].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[1] = None
                    return "out"
        
        wp_num = random.uniform(0,1)
        if(wp_num<.01):    # Seeing whether at bat featured wild pitch
            pitcher.cum_wp+=1
            if(runners[2]!=None):
                team.runs+=1
                runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = None
        if(wp_num<.0005):    # Seeing whether at bat featured second wild pitch   
            pitcher.cum_wp+=1
            if(runners[2]!=None):
                team.runs+=1
                runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = None          
        
    def get_result(self, batter, pitcher, outs, runners, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            outs: number of outs in inning
            runners: List of where runners are
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """      
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        pitcher.current_bf+=1
        
        sacrifice_num = random.uniform(0,1)
        if(outs<2 and batter.name=='pitcher' and runners[2]==None and (runners[1]!=None or runners[2]==None)):    # Seeing whether a pitcher will try to lay down a sacrifice bunt
            if(sacrifice_num<.3):
                batter.cum_sh+=1
                
                if(batter_against=='L'):
                    batter.cum_pa_L+=1
                else:
                    batter.cum_pa_R+=1
                    
                if(pitcher_against=='L'):
                    pitcher.cum_pa_L+=1
                    pitcher.cum_outs_L+=1
                else:
                    pitcher.cum_pa_R+=1
                    pitcher.cum_outs_R+=1
                
                return 'sh'
            elif(sacrifice_num<.45):
                if(batter_against=='L'):
                    batter.cum_pa_L+=1
                else:
                    batter.cum_pa_R+=1
                    
                if(pitcher_against=='L'):
                    pitcher.cum_pa_L+=1
                    pitcher.cum_outs_L+=1
                else:
                    pitcher.cum_pa_R+=1
                    pitcher.cum_outs_R+=1
                
                return 'out'
            
        if(batter_against=='L' and pitcher_against=='L'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            if(num<batter.singles_L+pitcher.singles_L):
                batter.cum_singles_L+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L):
                batter.cum_triples_L+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L
                 +batter.walks_L+pitcher.walks_L):
                batter.cum_walks_L+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L
                 +batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L
                 +batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L+batter.so_L+pitcher.so_L):
                batter.cum_so_L+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
        elif(batter_against=='R' and pitcher_against=='L'):
            batter.cum_pa_R+=1
            pitcher.cum_pa_L+=1
            if(num<batter.singles_R+pitcher.singles_L):
                batter.cum_singles_R+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L):
                batter.cum_triples_R+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L
                 +batter.walks_R+pitcher.walks_L):
                batter.cum_walks_R+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L
                 +batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L
                 +batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L+batter.so_R+pitcher.so_L):
                batter.cum_so_R+=1
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
        elif(batter_against=='L' and pitcher_against=='R'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_R+=1
            if(num<batter.singles_L+pitcher.singles_R):
                batter.cum_singles_L+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R):
                batter.cum_triples_L+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R
                 +batter.walks_L+pitcher.walks_R):
                batter.cum_walks_L+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R
                 +batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R
                 +batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R+batter.so_L+pitcher.so_R):
                batter.cum_so_L+=1
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
        elif(batter_against=='R' and pitcher_against=='R'):
            batter.cum_pa_R+=1
            pitcher.cum_pa_R+=1
            if(num<batter.singles_R+pitcher.singles_R):
                batter.cum_singles_R+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R):
                batter.cum_triples_R+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R
                 +batter.walks_R+pitcher.walks_R):
                batter.cum_walks_R+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R
                 +batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R
                 +batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R+batter.so_R+pitcher.so_R):
                batter.cum_so_R+=1
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                 else:
                    pitcher.cum_outs_R+=1
                    return "out"
              
    def move_runners(self, team, batter, pitcher, outs, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            batter who is hitting
            pitcher: pitcher pitching to team
            outs: number of outs
            result: str of result for batter (ie "walk")
        """
        made_out = False
        print(team.runners)
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        if(result=='single'):            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                team.runners[2] = None
                
            if(team.runners[1]!=None): 
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically

                if(num<((batter.bsr*2)+45+(outs*10))):
                    new_num = random.uniform(0,100)
                    if(new_num<3):
                        if(pitcher_against=='L'):
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[1].cum_runs+=1
                        team.runners_dict[team.runners[1]].cum_runs+=1
                        team.runners_dict[team.runners[1]].current_runs+=1
                        batter.cum_rbi+=1
                        team.runners[3]+=1
                else:
                    team.runners[2] = team.runners[1]
                    
                team.runners[1] = None
   
            if(team.runners[0]!=None and team.runners[2]==None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically

                if(num<((batter.bsr*3)+30)):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[2] = team.runners[0] 
                else:
                    team.runners[1] = team.runners[0]
            else:
                team.runners[1] = team.runners[0]
                
            team.runners[0] = batter
            team.runners_dict.update({batter: pitcher})

        elif(result=='double'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            
            if(team.runners[0]!=None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                if(num<((batter.bsr*4)+45)):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[0].cum_runs+=1
                        team.runners_dict[team.runners[0]].cum_runs+=1
                        team.runners_dict[team.runners[0]].current_runs+=1
                        batter.cum_rbi+=1
                        team.runners[3]+=1 
                else:
                    team.runners[2] = team.runners[0]
                
            team.runners[1] = batter
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='triple'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = batter
            team.runners[1] = None
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='home_run'):
            batter.cum_runs+=1
            pitcher.cum_runs+=1
            pitcher.current_runs+=1
            batter.cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=='walk' or result=='hbp' or result=='sh'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            team.runners_dict.update({batter: pitcher})
            
            if(temp_0!=None and temp_1!=None and temp_2!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            
            team.runners[0] = batter
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
            if(result=='sh'):   # If sacrifice is hit, batter doesn't reach base
                team.runners[0] = None
                                          
        elif(result=='gdp'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            rand_num = random.uniform(0,1)

            if(temp_1==None and temp_2==None):    # If only runner on first
                team.runners[0] = None
            elif(temp_1!=None and temp_2==None):    # If runners on first and second
                if(rand_num<.02):   # Outs made at first and third
                    team.runners[1] = temp_0
                    team.runners[0] = None
                elif(rand_num<.08):   # Outs made at third and second
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                else:
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1==None and temp_2!=None):    # If runners on first and third
                if(rand_num<.01):   # Batter goes to first
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                elif(rand_num<.02):   # Batter goes to second
                    team.runners[0] = None
                    team.runners[1] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                else: # Runner scores, bases empty
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[2] = None
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1!=None and temp_2!=None):    # If bases loaded
                if(rand_num<.6):   # Outs made at second and first
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[0] = None
                    team.runners[1] = None
                    team.runners[2] = temp_1
                elif(rand_num<.9):   # Outs made at home and first
                    team.runners[1] = temp_0
                    team.runners[2] = temp_1
                    team.runners[0] = None
                else:    # Outs made at third and home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                                        
        elif(result=='out'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            if(temp_0!=None and temp_1==None and temp_2==None):
                rand_num = random.uniform(0,1)
                
                if(rand_num<(.3+batter.bsr/15)):   # If batter hits ground ball and reaches on fielder's choice
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.4):    # If batter hits ground ball and is out but runner on first moves to second
                    team.runners[1] = temp_1
                    team.runners[0] = None
            elif(temp_0==None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.3+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third or grounder moving runner
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                elif(rand_num<.4):   # If batter hits ground ball scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
            elif(temp_0!=None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.1+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third
                    team.runners[2] = temp_1
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, guy on second goes to third, guy on first is out, batter goes to first
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.35):    # If batter hits ball, guy on second goes to third, guy on first goes to second, batter makes out
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = None
            elif(temp_0!=None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.05):
                        team.runners[1] = temp_0
                    
                    team.runners[2] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.35+batter.bsr/15):    # If batter hits ball, guy on third scores, guy on first out, batter ends up on first
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runner on second advances
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.33):    # If batter hits ground ball, out made at home, runner on second stays
                    team.runners[2] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0!=None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    if(rand_num<.06):
                        team.runners[1] = temp_0

                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runners advance
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.4+batter.bsr/15):    # If batter hits ground ball, out made at home, runner on second stays
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0
            
        if(made_out):
            return 'out'

        print("updated runners: " + str(team.runners))
        
    def print_result(self):
        """Printed score of a game
        """ 
        print("final score: " + self.away_team.city + " " + str(self.away_team.runs) + " " 
              + self.home_team.city + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams
        """ 
        print(str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played
        """ 
        self.away_team.restart()
        self.away_team.game_no+=1
        self.away_team.rotation_index = (self.away_team.game_no-1)%5
        self.home_team.restart() 
        self.home_team.game_no+=1
        self.home_team.rotation_index = (self.home_team.game_no-1)%5
        inning = 1
        game_over = False
        
        while(inning<10 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                if(inning==9 and side==self.home_team and self.home_team.runs>self.away_team.runs):
                    game_over = True
                    break
                print('inning: ' + str(inning))    
                outs = 0

                if(inning==1 and side==self.away_team):
                    # Determining starting pitchers
                    home_pitcher = self.home_team.choose_starter()
                    away_pitcher = self.away_team.choose_starter()
                    self.home_team.current_pitcher_list.append(home_pitcher)
                    self.home_team.current_pitcher = home_pitcher
                    self.away_team.current_pitcher_list.append(away_pitcher)
                    self.away_team.current_pitcher = away_pitcher
                    self.away_team.lineup, self.away_team.bench = self.away_team.make_lineup(self.away_team.batters, self.home_team.current_pitcher, self.home_team.league, self.away_team.lineup_dict)
                    self.home_team.lineup, self.home_team.bench = self.home_team.make_lineup(self.home_team.batters, self.away_team.current_pitcher, self.home_team.league, self.home_team.lineup_dict)
                    

                while(outs<3):
                    # Choosing a pitcher for a team based on game factors
                    if(side==self.away_team):
                        current_pitcher = self.home_team.current_pitcher
                        print(current_pitcher.name)
                        pitcher = self.home_team.choose_pitcher(outs, inning, side.runs, self.home_team.runs, side.lineup, 
                                                                side.batting_index, self.home_team.lineup, self.home_team.batting_index)
                        
                        if(current_pitcher!=pitcher and self.home_team.lineup[8].name=='pitcher'):   # If starting pitcher gets taken out of game and lineup, put bench player in to bat
                            self.home_team.lineup[8] = self.home_team.bench[random.randint(0, len(self.home_team.bench)-1)]
                            self.home_team.lineup[8].cum_G+=1
                            
                    elif(side==self.home_team):
                        current_pitcher = self.away_team.current_pitcher
                        print(current_pitcher.name)
                        pitcher = self.away_team.choose_pitcher(outs, inning, side.runs, self.away_team.runs, side.lineup, 
                                                                side.batting_index, self.away_team.lineup, self.away_team.batting_index)
                        
                        if(current_pitcher!=pitcher and self.away_team.lineup[8].name=='pitcher'):   # If starting pitcher gets taken out of game and lineup, put bench player in to bat
                            self.away_team.lineup[8] = self.away_team.bench[random.randint(0, len(self.away_team.bench)-1)]
                            self.away_team.lineup[8].cum_G+=1
                            
                    # Determining whether stolen base should be attempted
                    stolen_base_result = self.get_sb_wp_result(side.runners, pitcher, side, outs, random.uniform(0,1))
                    
                    if(stolen_base_result=='out'):   # If failed stolen base, out is recorded
                        pitcher.current_outs+=1
                        outs+=1
                                
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.runners_dict = {}
                            break
                    
                    # Determining outcome of at-bat
                    result = self.get_result(side.lineup[side.batting_index], pitcher, outs, side.runners, random.uniform(0,2))
                    print(side.lineup[side.batting_index].name)
                    print("result: " + result)
                    if(result=='out' or result=='k' or result=='sf'):    # Batter made an out
                        pitcher.current_outs+=1
                        outs+=1
                        print("outs: " + str(outs))
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                            
                    elif(result=='gdp'):    # Batter made an out
                        pitcher.current_outs+=2
                        outs+=2
                        print("outs: " + str(outs))
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                            
                    else:    # Batter reached, runners are moved
                        running_result = self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                        if(running_result=='out'):    # Baserunner made an out
                            pitcher.current_outs+=1
                            outs+=1
                            print("outs: " + str(outs))
                            
                            if(outs>2):   # If outs = 3, end of inning
                                pitcher.finished_half = True
                                side.batting_index+=1    # Gettting next batter in lineup
                                side.batting_index = side.batting_index%9
                                side.runners_dict = {}
                                break
                        print(side.city + ': '+ str(side.runs))
                        if(inning>9 and self.away_team.runs<self.home_team.runs):
                            game_over = True
                    
                    if(game_over):    # If game is over, escape game loop
                        break
                        
                    side.batting_index+=1    # Gettting next batter in lineup
                    side.batting_index = side.batting_index%9
                    
                if(game_over):    # If game is over, escape game loop
                    break
                        
                side.reset_runners()    # Runners are reset after half inning
            

            if(game_over):    # If game is over, escape game loop
                break   
                
            inning+=1
        
        # After game, updating team records and number of cumulative runs
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
        
class Season(Game):
    """Class that holds information for a Season of games.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team

    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(self.away_team, self.home_team)

            g.play() 
            
    def reset_season(self, teams):
        """Reinitializing teams
        
        Args:
            teams, Team instances to reinitialize
        """
        for team in teams:
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.relievers_df)
 
class FullSeason(Game):
    """Class that holds information for a Season of games
    
    Attributes:

    """
    
    def __init__(self, schedule):
        """Initializes values for this class
        
        Args:
            schedule: dataframe of season schedule
        """
        self.schedule = schedule

    def simulate_season(self):
        """A season of games is simulated
        """
        games = 0
        
        for date in self.schedule['date'].unique():
            for acronym, city in acronym_to_city_dict.items():
                away_team = team_dict[acronym_to_city_dict[acronym]]
                home_team_acronym = self.schedule.loc[self.schedule['date']==date][acronym].values[0]

                if(not pd.isna(home_team_acronym)):
                    home_team = team_dict[acronym_to_city_dict[home_team_acronym]]
                    g = Game(away_team, home_team)
                    g.play()
                    games+=1
                    print(games)
  
    def reset_season(self):
        """Reinitializing teams
        """
        for city_name, team in team_dict.items():
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters)   
            
    def print_record(self):
        """Printing records
        """
        for city_name, team in team_dict.items():
            print(team.city + ': ' + str(team.wins) + "-" + str(team.losses))
            
    def league_stats(self):
        """Accumulating stats for the entire league
        """
        pi = pd.DataFrame()
        ba = pd.DataFrame()
        for k, v in nickname_to_city_dict.items():

            a, b = team_dict[v].cum_stats_df()

            if(pi.empty):
                pi = b
            else:
                pi = pi.append(b)

            if(ba.empty):
                ba = a
            else:
                ba = ba.append(a)
        
        pi = pi.sort_values(by=['K'], ascending=False)
        pi.drop_duplicates(inplace=True)
        ba = ba.sort_values(by=['HR'], ascending=False)
        ba.drop_duplicates(inplace=True)
        
        return ba, pi

In [ ]:
elif(batter_against=='L' and pitcher_against=='L'):
            ##batter.cum_pa_R+=1
            ##pitcher.cum_pa_R+=1
            if(num<comb_singles_R*hand_multiplier + comb_singles_Away*field_multiplier):
               ###pitcher.cum_singles_R+=1
                return "single"
            elif(num<comb_doubles_R*hand_multiplier + comb_doubles_Away*field_multiplier):
                #batter.cum_doubles_R+=1
                ##pitcher.cum_doubles_R+=1
                return "double"
            elif(num<comb_triples_R*hand_multiplier + comb_triples_Away*field_multiplier):
                #batter.cum_triples_R+=1
                ##pitcher.cum_triples_R+=1
                return "triple"
            elif(num<comb_home_runs_R*hand_multiplier + comb_home_runs_Away*field_multiplier):
                #batter.cum_home_runs_R+=1
                ##pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<comb_walks_R*hand_multiplier + comb_walks_Away*field_multiplier):
                #batter.cum_walks_R+=1
                #pitcher.cum_walks_R+=1
                return "walk"
            elif(num<comb_hbp_R*hand_multiplier + comb_hbp_Away*field_multiplier):
                ##batter.cum_hbp_R+=1
                ##pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<comb_so_R*hand_multiplier + comb_so_Away*field_multiplier):
                ##batter.cum_so_R+=1                
                ##pitcher.cum_outs_R+=1
                ##pitcher.cum_so_R+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    ##batter.cum_gdp+=1
                    ##pitcher.cum_outs_R+=2
                    ##pitcher.cum_gdp+=1
                    #return "gdp"
                    return "out"
                else:
                    #pitcher.cum_outs_R+=1
                    return "out"
                
elif(batter_against=='L' and pitcher_against=='L'):
            ##batter.cum_pa_Home+=1
            ##pitcher.cum_pa_Home+=1
            if(num<comb_singles_Home*hand_multiplier + comb_singles_Away*field_multiplier):
               ###pitcher.cum_singles_Home+=1
                return "single"
            elif(num<comb_doubles_Home*hand_multiplier + comb_doubles_Away*field_multiplier):
                #batter.cum_doubles_Home+=1
                ##pitcher.cum_doubles_Home+=1
                return "double"
            elif(num<comb_triples_Home*hand_multiplier + comb_triples_Away*field_multiplier):
                #batter.cum_triples_Home+=1
                ##pitcher.cum_triples_Home+=1
                return "triple"
            elif(num<comb_home_runs_Home*hand_multiplier + comb_home_runs_Away*field_multiplier):
                #batter.cum_home_runs_Home+=1
                ##pitcher.cum_home_runs_Home+=1
                return "home_run"
            elif(num<comb_walks_Home*hand_multiplier + comb_walks_Away*field_multiplier):
                #batter.cum_walks_Home+=1
                #pitcher.cum_walks_Home+=1
                return "walk"
            elif(num<comb_hbp_Home*hand_multiplier + comb_hbp_Away*field_multiplier):
                ##batter.cum_hbp_Home+=1
                ##pitcher.cum_hbp_Home+=1
                return "hbp"
            elif(num<comb_so_Home*hand_multiplier + comb_so_Away*field_multiplier):
                ##batter.cum_so_Home+=1                
                ##pitcher.cum_outs_Home+=1
                ##pitcher.cum_so_Home+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    ##batter.cum_gdp+=1
                    ##pitcher.cum_outs_Home+=2
                    ##pitcher.cum_gdp+=1
                    #return "gdp"
                    return "out"
                else:
                    #pitcher.cum_outs_Home+=1
                    return "out"

elif(batter_against=='L' and pitcher_against=='L'):
            ##batter.cum_pa_Away+=1
            ##pitcher.cum_pa_Away+=1
            if(num<comb_singles_Away*hand_multiplier + comb_singles_Away*field_multiplier):
               ###pitcher.cum_singles_Away+=1
                return "single"
            elif(num<comb_doubles_Away*hand_multiplier + comb_doubles_Away*field_multiplier):
                #batter.cum_doubles_Away+=1
                ##pitcher.cum_doubles_Away+=1
                return "double"
            elif(num<comb_triples_Away*hand_multiplier + comb_triples_Away*field_multiplier):
                #batter.cum_triples_Away+=1
                ##pitcher.cum_triples_Away+=1
                return "triple"
            elif(num<comb_home_runs_Away*hand_multiplier + comb_home_runs_Away*field_multiplier):
                #batter.cum_home_runs_Away+=1
                ##pitcher.cum_home_runs_Away+=1
                return "home_run"
            elif(num<comb_walks_Away*hand_multiplier + comb_walks_Away*field_multiplier):
                #batter.cum_walks_Away+=1
                #pitcher.cum_walks_Away+=1
                return "walk"
            elif(num<comb_hbp_Away*hand_multiplier + comb_hbp_Away*field_multiplier):
                ##batter.cum_hbp_Away+=1
                ##pitcher.cum_hbp_Away+=1
                return "hbp"
            elif(num<comb_so_Away*hand_multiplier + comb_so_Away*field_multiplier):
                ##batter.cum_so_Away+=1                
                ##pitcher.cum_outs_Away+=1
                ##pitcher.cum_so_Away+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    ##batter.cum_gdp+=1
                    ##pitcher.cum_outs_Away+=2
                    ##pitcher.cum_gdp+=1
                    #return "gdp"
                    return "out"
                else:
                    #pitcher.cum_outs_Away+=1
                    return "out"               

In [394]:
class Batter_new:
    """Class that holds information for a Batter.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L, singles_R, 
                 doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R,
                 singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                 doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, sb_rate, sb_success):
        """Initializes values for this class
        
        Args:
            team: team name of batter
            name: batter name
            hand: Batting hand of hitter(L, R, or S)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            hbp_L: fraction of plate appearances ending in a hbp vs. lefties
            so_L: fraction of plate appearances ending in a strikeout vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            hbp_R: fraction of plate appearances ending in a hbp vs. righties
            so_R: fraction of plate appearances ending in a strikeout vs. righties
            sb_rate: approximate percentage of time player will steal base given opportunity
            sb_success: fraction of stolen base attempts ending in success
            bsr: fangraphs stat representing projected runs created by a base runner. Higher number is better
            fld: bsr: fangraphs stat representing projected runs created by a fielder. higher number is better
        """
        self.team = team
        self.name = name
        self.hand = hand
        self.singles_L = singles_L
        self.doubles_L = doubles_L
        self.triples_L = triples_L
        self.home_runs_L = home_runs_L
        self.walks_L = walks_L
        self.hbp_L = hbp_L
        self.so_L = so_L
        self.slg_L = walks_L+singles_L+2*doubles_L+3*triples_L+4*home_runs_L
        self.singles_R = singles_R
        self.doubles_R = doubles_R
        self.triples_R = triples_R
        self.home_runs_R = home_runs_R
        self.walks_R = walks_R
        self.hbp_R = hbp_R
        self.so_R = so_R
        self.slg_R = walks_R+singles_R+2*doubles_R+3*triples_R+4*home_runs_R
        self.singles_Home = singles_Home
        self.doubles_Home = doubles_Home
        self.triples_Home = triples_Home
        self.home_runs_Home = home_runs_Home
        self.walks_Home = walks_Home
        self.hbp_Home = hbp_Home
        self.so_Home = so_Home
        self.slg_Home = walks_Home+singles_Home+2*doubles_Home+3*triples_Home+4*home_runs_Home
        self.singles_Away = singles_Away
        self.doubles_Away = doubles_Away
        self.triples_Away = triples_Away
        self.home_runs_Away = home_runs_Away
        self.walks_Away = walks_Away
        self.hbp_Away = hbp_Away
        self.so_Away = so_Away
        self.slg_Away = walks_Away+singles_Away+2*doubles_Away+3*triples_Away+4*home_runs_Away
        self.sb_rate = sb_rate
        self.sb_success = sb_success
        
     
    def hits(self):
        """cumulative hits of a batter
        
        Returns:
            int: number of cumulative hits
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L+
               self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf+self.cum_sh)>0):
            return(round(float((self.hits())/((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf))), 3))
        else:
            return 0    
        
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)>0):
            return(round((self.hits()+(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R))/(self.cum_pa_L+self.cum_pa_R), 3))
        else:
            return 0
        
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf)>0):
            return(round(((self.cum_singles_L+self.cum_singles_R)+2*(self.cum_doubles_L+self.cum_doubles_R)+
                           3*(self.cum_triples_L+self.cum_triples_R)+4*(self.cum_home_runs_L+self.cum_home_runs_R))/
                           ((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf+self.cum_sh)), 3))
        else:
            return 0
        
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form
            """
        return(self.slg() + self.obp())
    
    def strikeouts(self):
        """Number of strikeouts of a batter
        
        Returns:
            float: total strikeouts
        """
        return(self.cum_so_L+self.cum_so_R)
    
    def print_stats(self):
        """Prints some stats of a player
        """
        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str((self.cum_home_runs_L+self.cum_home_runs_R)))
              
    def hits_L(self):
        """cumulative hits of a batter against lefties
        
        Returns:
            int: number of hits of batter agianst lefties
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L)
    
    def avg_L(self):
        """Calculates avg of a player against lefties
        
        Returns:
            float: batting average in decimal form of batter against lefties
        """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(float((self.hits_L())/(self.cum_pa_L-self.cum_walks_L)), 3))
        else:
            return 0    
              
    def obp_L(self):
        """Calculates on base percentage of a player against lefties
        
        Returns:
            float: opb in decimal form of batter against lefties
        """
        if(self.cum_pa_L>0):
            return(round((self.hits_L()+self.cum_walks_L)/self.cum_pa_L, 3))
        else:
            return 0
              
    def slg_L(self):
        """Calculates slugging percentage of a player against lefties
        
        Returns:
            float: slg in decimal form of batter against lefties
            """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(((self.cum_singles_L+2*self.cum_doubles_L+
                           3*self.cum_triples_L+4*self.cum_home_runs_L))/
                           (self.cum_pa_L-self.cum_walks_L), 3))
        else:
            return 0
              
    def ops_L(self):
        """Calculates on base plus slugging of a player against lefties
        
        Returns:
            float: ops in decimal form of batter against lefties
        """
        return(self.slg_L() + self.obp_L())
              
    def hits_R(self):
        """Cumulative hits of a batter against righties
        
        Returns:
            int: number of hits of batter against righties
        """
        return(self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg_R(self):
        """Calculates avg of a player against righties
        
        Returns:
            float: batting average in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(float((self.hits_R())/(self.cum_pa_R-self.cum_walks_R)), 3))
        else:
            return 0    
              
    def obp_R(self):
        """Calculates on base percentage of a player against righties
        
        Returns:
            float: opb in decimal form of batter against righties
        """
        if(self.cum_pa_R>0):
            return(round((self.hits_R()+self.cum_walks_R)/self.cum_pa_R, 3))
        else:
            return 0
              
    def slg_R(self):
        """Calculates slugging percentage of a player against righties
        
        Returns:
            float: slg in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(((self.cum_singles_R2*self.cum_doubles_R+
                           3*self.cum_triples_R+4*self.cum_home_runs_R))/
                           (self.cum_pa_R-self.cum_walks_R), 3))
        else:
            return 0
              
    def ops_R(self):
        """Calculates on base plus slugging of a player against righties
        
        Returns:
            float: ops in decimal form of batter against righties
        """
        
        return(self.slg_R() + self.obp_R())
              
class Pitcher_new(Batter):
    """Class that holds information for a pitcher. Many attributes are similar to those from the Batter class.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L, singles_R,
                 doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R, sb_rate, 
                 singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                 doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, sb_success):
        """Initializes values for this class
        
        Args:
            team: team name of pitcher
            name: pitcher name
            hand: Pitching arm of pitcher(L, R)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            hbp_L: fraction of plate appearances ending in a hbp vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            hbp_R: fraction of plate appearances ending in a hbp vs. righties
            sb_rate: approximate percentage of time opposing player will steal base given opportunity (set to 0)
            sb_success: fraction of opponent stolen base attempts ending in success (set to 0)
        """
        
        Batter_new.__init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L,
                        singles_R, doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R,
                        singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                        doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)
        self.cum_outs_L = 0
        self.cum_outs_R = 0
        self.cum_wp = 0
        self.current_bf = 0
        self.current_outs = 0
        self.current_runs = 0
        self.finished_half = False
        self.days_off = 100
        self.consecutive_days = 0
    
    def cum_outs(self, hand):
        """Cumulative outs of a pitcher against batters of certain handedness
        
        Args:
            hand: handedness of batter pitcher is facing
        Returns:
            float: value for cumulative outs
        """
        if(hand=='L'):
            return self.cum_outs_L
        elif(hand=='R'):
            return self.cum_outs_R
    
    def ip(self):
        """Calculates innings pitched of a pitcher
        
        Returns:
            float: Innings pitched in normal form(ie. 5.1 for 5 and a third innings pitched)
        """
        ip = (self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3.0
        ip = math.modf(ip)
        ip = round((ip[0]/3.33333 + ip[1]), 2)
        return(ip)  
    
    def era(self):
        """Calculates ERA of a pitcher
        
        Returns:
            float: ERA in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((27*self.cum_runs/(self.cum_outs_L+self.cum_outs_R+self.cum_cs)), 2))
    
    def whip(self):
        """Calculates WHIP of a pitcher
        
        Returns:
            float: WHIP in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((((self.cum_walks_L+self.cum_walks_R)+self.hits())/
                          ((self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3)), 2))

class Team_new:
    """Class that holds information for a team.
    
    Attributes:

    """
    def __init__(self, city, league, batter_df, pitcher_df, roster, lineup_dict=None):
        """Initializes values for this class
        
        Args:
            city: city of team (ex. 'Atlanta')
            league: league of team ('AL' or 'NL')
            batter_df: dataframe of batters on a team and their 2019 stats
            pitcher_df: dataframe of pitchers on a team and their 2019 stats
            starters: dict with names of batters in projected starting lineup, rotation, and the closer
            relievers_df: dataframe with relievers of a team and their projected innings pitched
        """
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.batter_dict= {}
        self.pitcher_df = pitcher_df
        self.pitcher_dict = {}
        self.pitchers = []
        self.batters = []
        
        avg_singles_L = batter_stats['singles_L'].mean()/batter_stats['PA_L'].mean()
        avg_doubles_L = batter_stats['doubles_L'].mean()/batter_stats['PA_L'].mean()
        avg_triples_L = batter_stats['triples_L'].mean()/batter_stats['PA_L'].mean()
        avg_home_runs_L = batter_stats['HR_L'].mean()/batter_stats['PA_L'].mean()
        avg_walks_L = batter_stats['BB_L'].mean()/batter_stats['PA_L'].mean()
        avg_hbp_L = batter_stats['HBP_L'].mean()/batter_stats['PA_L'].mean()
        avg_so_L = batter_stats['SO_L'].mean()/batter_stats['PA_L'].mean()
        
        avg_singles_R = batter_stats['singles_R'].mean()/batter_stats['PA_R'].mean()
        avg_doubles_R = batter_stats['doubles_R'].mean()/batter_stats['PA_R'].mean()
        avg_triples_R = batter_stats['triples_R'].mean()/batter_stats['PA_R'].mean()
        avg_home_runs_R = batter_stats['HR_R'].mean()/batter_stats['PA_R'].mean()
        avg_walks_R = batter_stats['BB_R'].mean()/batter_stats['PA_R'].mean()
        avg_hbp_R = batter_stats['HBP_R'].mean()/batter_stats['PA_R'].mean()
        avg_so_R = batter_stats['SO_R'].mean()/batter_stats['PA_R'].mean()
        
        avg_singles_Home = batter_stats['singles_Home'].mean()/batter_stats['PA_Home'].mean()
        avg_doubles_Home = batter_stats['doubles_Home'].mean()/batter_stats['PA_Home'].mean()
        avg_triples_Home = batter_stats['triples_Home'].mean()/batter_stats['PA_Home'].mean()
        avg_home_runs_Home = batter_stats['HR_Home'].mean()/batter_stats['PA_Home'].mean()
        avg_walks_Home = batter_stats['BB_Home'].mean()/batter_stats['PA_Home'].mean()
        avg_hbp_Home = batter_stats['HBP_Home'].mean()/batter_stats['PA_Home'].mean()
        avg_so_Home = batter_stats['SO_Home'].mean()/batter_stats['PA_Home'].mean()
        
        avg_singles_Away = batter_stats['singles_Away'].mean()/batter_stats['PA_Away'].mean()
        avg_doubles_Away = batter_stats['doubles_Away'].mean()/batter_stats['PA_Away'].mean()
        avg_triples_Away = batter_stats['triples_Away'].mean()/batter_stats['PA_Away'].mean()
        avg_home_runs_Away = batter_stats['HR_Away'].mean()/batter_stats['PA_Away'].mean()
        avg_walks_Away = batter_stats['BB_Away'].mean()/batter_stats['PA_Away'].mean()
        avg_hbp_Away = batter_stats['HBP_Away'].mean()/batter_stats['PA_Away'].mean()
        avg_so_Away = batter_stats['SO_Away'].mean()/batter_stats['PA_Away'].mean()

        for index, batter in batter_df.iterrows():   # adding values to each batter and instantiating Batter class
            singles_L = 2*batter['singles_L']/batter['PA_L']-avg_singles_L
            doubles_L = 2*batter['doubles_L']/batter['PA_L']-avg_doubles_L
            triples_L = 2*batter['triples_L']/batter['PA_L']-avg_triples_L
            home_runs_L = 2*batter['HR_L']/batter['PA_L']-avg_home_runs_L
            walks_L = 2*batter['BB_L']/batter['PA_L']-avg_walks_L
            hbp_L = 2*batter['HBP_L']/batter['PA_L']-avg_hbp_L
            so_L = 2*batter['SO_L']/batter['PA_L']-avg_so_L
            
            singles_R = 2*batter['singles_R']/batter['PA_R']-avg_singles_R
            doubles_R = 2*batter['doubles_R']/batter['PA_R']-avg_doubles_R
            triples_R = 2*batter['triples_R']/batter['PA_R']-avg_triples_R
            home_runs_R = 2*batter['HR_R']/batter['PA_R']-avg_home_runs_R
            walks_R = 2*batter['BB_R']/batter['PA_R']-avg_walks_R
            hbp_R = 2*batter['HBP_R']/batter['PA_R']-avg_hbp_R
            so_R = 2*batter['SO_R']/batter['PA_R']-avg_so_R
            
            singles_Home = 2*batter['singles_Home']/batter['PA_Home']-avg_singles_Home
            doubles_Home = 2*batter['doubles_Home']/batter['PA_Home']-avg_doubles_Home
            triples_Home = 2*batter['triples_Home']/batter['PA_Home']-avg_triples_Home
            home_runs_Home = 2*batter['HR_Home']/batter['PA_Home']-avg_home_runs_Home
            walks_Home = 2*batter['BB_Home']/batter['PA_Home']-avg_walks_Home
            hbp_Home = 2*batter['HBP_Home']/batter['PA_Home']-avg_hbp_Home
            so_Home = 2*batter['SO_Home']/batter['PA_Home']-avg_so_Home
            
            singles_Away = 2*batter['singles_Away']/batter['PA_Away']-avg_singles_Away
            doubles_Away = 2*batter['doubles_Away']/batter['PA_Away']-avg_doubles_Away
            triples_Away = 2*batter['triples_Away']/batter['PA_Away']-avg_triples_Away
            home_runs_Away = 2*batter['HR_Away']/batter['PA_Away']-avg_home_runs_Away
            walks_Away = 2*batter['BB_Away']/batter['PA_Away']-avg_walks_Away
            hbp_Away = 2*batter['HBP_Away']/batter['PA_Away']-avg_hbp_Away
            so_Away = 2*batter['SO_Away']/batter['PA_Away']-avg_so_Away
            
            current_batter = Batter_new(self.city, batter['Name'], batter['Bats'], singles_L, doubles_L, triples_L, 
                                home_runs_L, walks_L, hbp_L, so_L, singles_R, doubles_R, triples_R, 
                                home_runs_R, walks_R, hbp_R, so_R, singles_Home, doubles_Home, triples_Home, 
                                home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, doubles_Away, triples_Away, 
                                home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)

            self.batters.append(current_batter)
            self.batter_dict.update({batter['Name']: current_batter})
            
        avg_singles_L = pitcher_stats['singles_L'].mean()/pitcher_stats['TBF_L'].mean()
        avg_doubles_L = pitcher_stats['doubles_L'].mean()/pitcher_stats['TBF_L'].mean()
        avg_triples_L = pitcher_stats['triples_L'].mean()/pitcher_stats['TBF_L'].mean()
        avg_home_runs_L = pitcher_stats['HR_L'].mean()/pitcher_stats['TBF_L'].mean()
        avg_walks_L = pitcher_stats['BB_L'].mean()/pitcher_stats['TBF_L'].mean()
        avg_hbp_L = pitcher_stats['HBP_L'].mean()/pitcher_stats['TBF_L'].mean()
        avg_so_L = pitcher_stats['SO_L'].mean()/pitcher_stats['TBF_L'].mean()
        
        avg_singles_R = pitcher_stats['singles_R'].mean()/pitcher_stats['TBF_R'].mean()
        avg_doubles_R = pitcher_stats['doubles_R'].mean()/pitcher_stats['TBF_R'].mean()
        avg_triples_R = pitcher_stats['triples_R'].mean()/pitcher_stats['TBF_R'].mean()
        avg_home_runs_R = pitcher_stats['HR_R'].mean()/pitcher_stats['TBF_R'].mean()
        avg_walks_R = pitcher_stats['BB_R'].mean()/pitcher_stats['TBF_R'].mean()
        avg_hbp_R = pitcher_stats['HBP_R'].mean()/pitcher_stats['TBF_R'].mean()
        avg_so_R = pitcher_stats['SO_R'].mean()/pitcher_stats['TBF_R'].mean()  
        
        avg_singles_Home = pitcher_stats['singles_Home'].mean()/pitcher_stats['TBF_Home'].mean()
        avg_doubles_Home = pitcher_stats['doubles_Home'].mean()/pitcher_stats['TBF_Home'].mean()
        avg_triples_Home = pitcher_stats['triples_Home'].mean()/pitcher_stats['TBF_Home'].mean()
        avg_home_runs_Home = pitcher_stats['HR_Home'].mean()/pitcher_stats['TBF_Home'].mean()
        avg_walks_Home = pitcher_stats['BB_Home'].mean()/pitcher_stats['TBF_Home'].mean()
        avg_hbp_Home = pitcher_stats['HBP_Home'].mean()/pitcher_stats['TBF_Home'].mean()
        avg_so_Home = pitcher_stats['SO_Home'].mean()/pitcher_stats['TBF_Home'].mean()
        
        avg_singles_Away = pitcher_stats['singles_Away'].mean()/pitcher_stats['TBF_Away'].mean()
        avg_doubles_Away = pitcher_stats['doubles_Away'].mean()/pitcher_stats['TBF_Away'].mean()
        avg_triples_Away = pitcher_stats['triples_Away'].mean()/pitcher_stats['TBF_Away'].mean()
        avg_home_runs_Away = pitcher_stats['HR_Away'].mean()/pitcher_stats['TBF_Away'].mean()
        avg_walks_Away = pitcher_stats['BB_Away'].mean()/pitcher_stats['TBF_Away'].mean()
        avg_hbp_Away = pitcher_stats['HBP_Away'].mean()/pitcher_stats['TBF_Away'].mean()
        avg_so_Away = pitcher_stats['SO_Away'].mean()/pitcher_stats['TBF_Away'].mean()
            
        for index, pitcher in pitcher_df.iterrows(): # adding values to each pitcher and instantiating Pitcher class
            singles_L = 2*pitcher['singles_L']/pitcher['TBF_L']-avg_singles_L
            doubles_L = 2*pitcher['doubles_L']/pitcher['TBF_L']-avg_doubles_L
            triples_L = 2*pitcher['triples_L']/pitcher['TBF_L']-avg_triples_L
            home_runs_L = 2*pitcher['HR_L']/pitcher['TBF_L']-avg_home_runs_L
            walks_L = 2*pitcher['BB_L']/pitcher['TBF_L']-avg_walks_L
            hbp_L = 2*pitcher['HBP_L']/pitcher['TBF_L']-avg_hbp_L
            so_L = 2*pitcher['SO_L']/pitcher['TBF_L']-avg_so_L
            
            singles_R = 2*pitcher['singles_R']/pitcher['TBF_R']-avg_singles_R
            doubles_R = 2*pitcher['doubles_R']/pitcher['TBF_R']-avg_doubles_R
            triples_R = 2*pitcher['triples_R']/pitcher['TBF_R']-avg_triples_R
            home_runs_R = 2*pitcher['HR_R']/pitcher['TBF_R']-avg_home_runs_R
            walks_R = 2*pitcher['BB_R']/pitcher['TBF_R']-avg_walks_R
            hbp_R = 2*pitcher['HBP_R']/pitcher['TBF_R']-avg_hbp_R
            so_R = 2*pitcher['SO_R']/pitcher['TBF_R']-avg_so_R
            
            singles_Home = 2*pitcher['singles_Home']/pitcher['TBF_Home']-avg_singles_Home
            doubles_Home = 2*pitcher['doubles_Home']/pitcher['TBF_Home']-avg_doubles_Home
            triples_Home = 2*pitcher['triples_Home']/pitcher['TBF_Home']-avg_triples_Home
            home_runs_Home = 2*pitcher['HR_Home']/pitcher['TBF_Home']-avg_home_runs_Home
            walks_Home = 2*pitcher['BB_Home']/pitcher['TBF_Home']-avg_walks_Home
            hbp_Home = 2*pitcher['HBP_Home']/pitcher['TBF_Home']-avg_hbp_Home
            so_Home = 2*pitcher['SO_Home']/pitcher['TBF_Home']-avg_so_Home
            
            singles_Away = 2*pitcher['singles_Away']/pitcher['TBF_Away']-avg_singles_Away
            doubles_Away = 2*pitcher['doubles_Away']/pitcher['TBF_Away']-avg_doubles_Away
            triples_Away = 2*pitcher['triples_Away']/pitcher['TBF_Away']-avg_triples_Away
            home_runs_Away = 2*pitcher['HR_Away']/pitcher['TBF_Away']-avg_home_runs_Away
            walks_Away = 2*pitcher['BB_Away']/pitcher['TBF_Away']-avg_walks_Away
            hbp_Away = 2*pitcher['HBP_Away']/pitcher['TBF_Away']-avg_hbp_Away
            so_Away = 2*pitcher['SO_Away']/pitcher['TBF_Away']-avg_so_Away

            current_pitcher = Pitcher_new(self.city, pitcher['Name'], pitcher['Throws'], singles_L, doubles_L, triples_L, 
                                  home_runs_L, walks_L, hbp_L, so_L, singles_R, doubles_R, triples_R, 
                                  home_runs_R, walks_R, hbp_R, so_R, singles_Home, doubles_Home, triples_Home, 
                                  home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, doubles_Away, triples_Away, 
                                  home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)
            
            self.pitchers.append(current_pitcher)
            self.pitcher_dict.update({pitcher['Name']: current_pitcher})
           
    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started
        """
        for pitcher in self.current_pitcher_list:
            pitcher.consecutive_days+=1
            pitcher.days_off = 0
            pitcher.current_bf = 0
            pitcher.current_outs = 0
            pitcher.current_runs = 0
            pitcher.finished_half = False
        
        for pitcher in self.available_pitchers_list:
            pitcher.days_off+=1
            pitcher.consecutive_days = 0

        self.runners_dict = {}    
        self.current_pitcher_list = []   
        self.batting_index = 0
        self.current_pitcher = None
        self.starter_pitching = True
        self.runners = [None, None, None, 0]
        self.runs = 0
        
    def reset_runners(self):
        """Resets baserunners after each half inning
        """
        self.runners = [None, None, None, 0]
    
    def cumul_stats_df(self):
        """Class that returns dataframes of stats for batters and pitchers
    
        Returns:
            batter_df: dataframe of batter stats
            pitcher_df: dataframe of pitcher stats
        """
        batter_df = pd.DataFrame(columns=['Name', 'G', 'PA', 'AB', 'H', 'singles', 'doubles', 'triples', 'HR', 'BB', 'HBP', 'SF', 
                                          'SH', 'K', 'AVG', 'OBP', 'SLG', 'OPS', 'R', 'RBI', 'SB', 'CS'])
        
        for b in self.batters:
            batter_values = [b.name, b.cum_G, b.cum_pa_L+b.cum_pa_R, b.cum_pa_L+b.cum_pa_R-
                             (b.cum_walks_L+b.cum_walks_R+b.cum_hbp_L+b.cum_hbp_R+b.cum_sf), 
                             b.hits(), b.cum_singles_L+b.cum_singles_R, b.cum_doubles_L+b.cum_doubles_R, 
                             b.cum_triples_L+b.cum_triples_R, b.cum_home_runs_L+b.cum_home_runs_R, 
                             b.cum_walks_L+b.cum_walks_R, b.cum_hbp_L+b.cum_hbp_R, b.cum_sf, b.cum_sh, b.strikeouts(), b.avg(), b.obp(), 
                             b.slg(), b.ops(), b.cum_runs, b.cum_rbi, b.cum_sb, b.cum_cs]
            batter_df = batter_df.append(pd.Series(batter_values, index=batter_df.columns), ignore_index=True)
        batter_df.sort_values(by=['PA'], ascending=False, inplace=True)
        
        pitcher_df = pd.DataFrame(columns=['Name', 'G', 'GS', 'IP', 'K', 'H', 'BB', 'HBP', 'ERA', 'WHIP', 'HR',
                                           'AVG', 'OBP', 'SLG', 'OPS', 'R', 'SB', 'CS', 'WP'])
        
        for p in self.pitchers:
            pitcher_values = [p.name, p.cum_G, p.cum_GS, p.ip(), p.strikeouts(), p.hits(), p.cum_walks_L+p.cum_walks_R, 
                              p.cum_hbp_L+p.cum_hbp_R, p.era(), p.whip(), p.cum_home_runs_L+p.cum_home_runs_R, p.avg(), 
                              p.obp(), p.slg(), p.ops(), p.cum_runs, p.cum_sb, p.cum_cs, p.cum_wp]
            pitcher_df = pitcher_df.append(pd.Series(pitcher_values, index=pitcher_df.columns), ignore_index=True)
        pitcher_df.sort_values(by=['IP'], ascending=False, inplace=True) 
        
        return batter_df, pitcher_df
    
class Game_new:
    """Class that holds information for a Game.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team
                                
    def get_sb_wp_result(self, runners, pitcher, team, outs, num):
        """stolen base result of a batter
        
        Args:
            runners: list of runners on base
            pitcher: pitcher currently pitching
            team: batting team
            outs: number of outs
            num: random number used to determine whether or not batter should steal base
        Returns:
            result of stolen base attempt (or None if no attempt)
        """
        if(runners[0]!=None and runners[1]==None):   # If first base occupied and second base not, steal possible
            if(num>runners[0].sb_rate/1.2):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[0].sb_success):  # Successful stolen base attempt
                    runners[0].cum_sb+=1
                    #pitcher.cum_sb+=1
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[0].cum_cs+=1
                    #pitcher.cum_cs+=1
                    runners[0] = None
                    return "out"
        elif(runners[0]!=None and runners[1]!=None):   # If first and second base occupied
            if(num>runners[0].sb_rate/5):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[1].sb_success):  # Successful stolen base attempt
                    runners[1].cum_sb+=1
                    runners[0].cum_sb+=1
                    #pitcher.cum_sb+=2
                    runners[2] = runners[1]
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[1].cum_cs+=1
                    #pitcher.cum_cs+=1
                    if(outs<2):
                        runners[0].cum_sb+=1
                        #pitcher.cum_sb+=1
                        runners[1] = runners[0]
                        
                    runners[0] = None
                    return "out"
        elif(runners[0]!=None and runners[1]!=None):   # If second base occupied
            if(num>runners[0].sb_rate/5):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[1].sb_success):  # Successful stolen base attempt
                    runners[1].cum_sb+=1
                    #pitcher.cum_sb+=1
                    runners[2] = runners[1]
                    runners[1] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[1].cum_cs+=1
                    #pitcher.cum_cs+=1
                    runners[1] = None
                    return "out"
        
        wp_num = random.uniform(0,1)
        if(wp_num<.01):    # Seeing whether at bat featured wild pitch
            #pitcher.cum_wp+=1
            if(runners[2]!=None):
                team.runs+=1
                runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = None
        if(wp_num<.0005):    # Seeing whether at bat featured second wild pitch   
            #pitcher.cum_wp+=1
            if(runners[2]!=None):
                team.runs+=1
                runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = None          
        
    def get_result(self, batter, pitcher, outs, runners, batting_side, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            outs: number of outs in inning
            runners: List of where runners are
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """      
        hand_multiplier = 3/4
        field_multiplier = 1-hand_multiplier
        
        
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        #pitcher.current_bf+=1
        
        sacrifice_num = random.uniform(0,1)
        if(outs<2 and batter.name=='pitcher' and runners[2]==None and (runners[1]!=None or runners[2]==None)):    # Seeing whether a pitcher will try to lay down a sacrifice bunt
            if(sacrifice_num<.3):
                #batter.cum_sh+=1
                
                if(batter_against=='L'):
                    #batter.cum_pa_L+=1
                else:
                    #batter.cum_pa_R+=1
                    
                if(pitcher_against=='L'):
                    #pitcher.cum_pa_L+=1
                    #pitcher.cum_outs_L+=1
                else:
                    #pitcher.cum_pa_R+=1
                    #pitcher.cum_outs_R+=1
                
                return 'sh'
            elif(sacrifice_num<.45):
                if(batter_against=='L'):
                    #batter.cum_pa_L+=1
                else:
                    #batter.cum_pa_R+=1
                    
                if(pitcher_against=='L'):
                    #pitcher.cum_pa_L+=1
                    #pitcher.cum_outs_L+=1
                else:
                    #pitcher.cum_pa_R+=1
                    #pitcher.cum_outs_R+=1
                
                return 'out'
            
        if(batter_against=='L' and pitcher_against=='L' and batting side=='Home'):
            ##batter.cum_pa_L+=1
            ##pitcher.cum_pa_L+=1
            if(num<(batter.singles_L+pitcher.singles_L)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_muliplier):
               ###pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L)*hand_multiplier+
                     (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier:
                #batter.cum_doubles_L+=1
                ##pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L)
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away):
                #batter.cum_triples_L+=1
                ##pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L)
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.Home_runs_Away)
                #batter.cum_home_runs_L+=1
                ##pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L
                 +batter.walks_L+pitcher.walks_L):
                #batter.cum_walks_L+=1
                ##pitcher.cum_walks_L+=1
                return "walk"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L
                 +batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L):
                ##batter.cum_hbp_L+=1
                ##pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L
                 +batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L+batter.so_L+pitcher.so_L):
                ##batter.cum_so_L+=1                
                ##pitcher.cum_outs_L+=1
                ##pitcher.cum_so_L+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    ##batter.cum_gdp+=1
                    ##pitcher.cum_outs_L+=2
                    ##pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    #pitcher.cum_outs_L+=1
                    return "out"
        elif(batter_against=='R' and pitcher_against=='L'):
            #batter.cum_pa_R+=1
            #pitcher.cum_pa_L+=1
            if(num<batter.singles_R+pitcher.singles_L):
                #batter.cum_singles_R+=1
                #pitcher.cum_singles_L+=1
                return "single"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L):
                #batter.cum_doubles_R+=1
                #pitcher.cum_doubles_L+=1
                return "double"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L):
                #batter.cum_triples_R+=1
                #pitcher.cum_triples_L+=1
                return "triple"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L):
                #batter.cum_home_runs_R+=1
                #pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L
                 +batter.walks_R+pitcher.walks_L):
                #batter.cum_walks_R+=1
                #pitcher.cum_walks_L+=1
                return "walk"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L
                 +batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L):
                #batter.cum_hbp_R+=1
                #pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L
                 +batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L+batter.so_R+pitcher.so_L):
                #batter.cum_so_R+=1
                #pitcher.cum_outs_L+=1
                #pitcher.cum_so_L+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    #batter.cum_gdp+=1
                    #pitcher.cum_outs_L+=2
                    #pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    #pitcher.cum_outs_L+=1
                    return "out"
        elif(batter_against=='L' and pitcher_against=='R'):
            #batter.cum_pa_L+=1
            #pitcher.cum_pa_R+=1
            if(num<batter.singles_L+pitcher.singles_R):
                #batter.cum_singles_L+=1
                #pitcher.cum_singles_R+=1
                return "single"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R):
                #batter.cum_doubles_L+=1
                #pitcher.cum_doubles_R+=1
                return "double"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R):
                #batter.cum_triples_L+=1
                #pitcher.cum_triples_R+=1
                return "triple"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R):
                #batter.cum_home_runs_L+=1
                #pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R
                 +batter.walks_L+pitcher.walks_R):
                #batter.cum_walks_L+=1
                #pitcher.cum_walks_R+=1
                return "walk"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R
                 +batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R):
                #batter.cum_hbp_L+=1
                #pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R
                 +batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R+batter.so_L+pitcher.so_R):
                #batter.cum_so_L+=1
                #pitcher.cum_outs_R+=1
                #pitcher.cum_so_R+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    #batter.cum_gdp+=1
                    #pitcher.cum_outs_R+=2
                    #pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    #pitcher.cum_outs_R+=1
                    return "out"
        elif(batter_against=='R' and pitcher_against=='R'):
            #batter.cum_pa_R+=1
            #pitcher.cum_pa_R+=1
            if(num<batter.singles_R+pitcher.singles_R):
                #batter.cum_singles_R+=1
                #pitcher.cum_singles_R+=1
                return "single"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R):
                #batter.cum_doubles_R+=1
                #pitcher.cum_doubles_R+=1
                return "double"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R):
                #batter.cum_triples_R+=1
                #pitcher.cum_triples_R+=1
                return "triple"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R):
                #batter.cum_home_runs_R+=1
                #pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R
                 +batter.walks_R+pitcher.walks_R):
                #batter.cum_walks_R+=1
                #pitcher.cum_walks_R+=1
                return "walk"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R
                 +batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R):
                #batter.cum_hbp_R+=1
                #pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R
                 +batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R+batter.so_R+pitcher.so_R):
                #batter.cum_so_R+=1
                #pitcher.cum_outs_R+=1
                #pitcher.cum_so_R+=1
                return "k"
            else:
                 if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    #batter.cum_gdp+=1
                    #pitcher.cum_outs_R+=2
                    #pitcher.cum_gdp+=1
                    return "gdp"
                 else:
                    #pitcher.cum_outs_R+=1
                    return "out"
              
    def move_runners(self, team, batter, pitcher, outs, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            batter who is hitting
            pitcher: pitcher pitching to team
            outs: number of outs
            result: str of result for batter (ie "walk")
        """
        made_out = False
        print(team.runners)
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        if(result=='single'):            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                team.runners[2] = None
                
            if(team.runners[1]!=None): 
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically

                if(num<((batter.bsr*2)+45+(outs*10))):
                    new_num = random.uniform(0,100)
                    if(new_num<3):
                        if(pitcher_against=='L'):
                            made_out = True
                        else:
                            #pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[1].cum_runs+=1
                        team.runners_dict[team.runners[1]].cum_runs+=1
                        team.runners_dict[team.runners[1]].current_runs+=1
                        #batter.cum_rbi+=1
                        team.runners[3]+=1
                else:
                    team.runners[2] = team.runners[1]
                    
                team.runners[1] = None
   
            if(team.runners[0]!=None and team.runners[2]==None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically

                if(num<((batter.bsr*3)+30)):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            #pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            #pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[2] = team.runners[0] 
                else:
                    team.runners[1] = team.runners[0]
            else:
                team.runners[1] = team.runners[0]
                
            team.runners[0] = batter
            team.runners_dict.update({batter: pitcher})

        elif(result=='double'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
            
            if(team.runners[0]!=None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                if(num<((batter.bsr*4)+45)):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            #pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            #pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[0].cum_runs+=1
                        team.runners_dict[team.runners[0]].cum_runs+=1
                        team.runners_dict[team.runners[0]].current_runs+=1
                        #batter.cum_rbi+=1
                        team.runners[3]+=1 
                else:
                    team.runners[2] = team.runners[0]
                
            team.runners[1] = batter
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='triple'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = batter
            team.runners[1] = None
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='home_run'):
            #batter.cum_runs+=1
            #pitcher.cum_runs+=1
            pitcher.current_runs+=1
            #batter.cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=='walk' or result=='hbp' or result=='sh'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            team.runners_dict.update({batter: pitcher})
            
            if(temp_0!=None and temp_1!=None and temp_2!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                #batter.cum_rbi+=1
                team.runners[3]+=1
            
            team.runners[0] = batter
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
            if(result=='sh'):   # If sacrifice is hit, batter doesn't reach base
                team.runners[0] = None
                                          
        elif(result=='gdp'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            rand_num = random.uniform(0,1)

            if(temp_1==None and temp_2==None):    # If only runner on first
                team.runners[0] = None
            elif(temp_1!=None and temp_2==None):    # If runners on first and second
                if(rand_num<.02):   # Outs made at first and third
                    team.runners[1] = temp_0
                    team.runners[0] = None
                elif(rand_num<.08):   # Outs made at third and second
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                else:
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1==None and temp_2!=None):    # If runners on first and third
                if(rand_num<.01):   # Batter goes to first
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                elif(rand_num<.02):   # Batter goes to second
                    team.runners[0] = None
                    team.runners[1] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                else: # Runner scores, bases empty
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[2] = None
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1!=None and temp_2!=None):    # If bases loaded
                if(rand_num<.6):   # Outs made at second and first
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[0] = None
                    team.runners[1] = None
                    team.runners[2] = temp_1
                elif(rand_num<.9):   # Outs made at home and first
                    team.runners[1] = temp_0
                    team.runners[2] = temp_1
                    team.runners[0] = None
                else:    # Outs made at third and home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                                        
        elif(result=='out'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            if(temp_0!=None and temp_1==None and temp_2==None):
                rand_num = random.uniform(0,1)
                
                if(rand_num<(.3+batter.bsr/15)):   # If batter hits ground ball and reaches on fielder's choice
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.4):    # If batter hits ground ball and is out but runner on first moves to second
                    team.runners[1] = temp_1
                    team.runners[0] = None
            elif(temp_0==None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.3+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third or grounder moving runner
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    #batter.cum_rbi+=1
                    #batter.cum_sf+=1
                    team.runners[3]+=1
                elif(rand_num<.4):   # If batter hits ground ball scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    #batter.cum_rbi+=1
                    team.runners[3]+=1
            elif(temp_0!=None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.1+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third
                    team.runners[2] = temp_1
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, guy on second goes to third, guy on first is out, batter goes to first
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.35):    # If batter hits ball, guy on second goes to third, guy on first goes to second, batter makes out
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = None
            elif(temp_0!=None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    #batter.cum_rbi+=1
                    #batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.05):
                        team.runners[1] = temp_0
                    
                    team.runners[2] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.35+batter.bsr/15):    # If batter hits ball, guy on third scores, guy on first out, batter ends up on first
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    #batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    #batter.cum_rbi+=1
                    #batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runner on second advances
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.33):    # If batter hits ground ball, out made at home, runner on second stays
                    team.runners[2] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0!=None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    #batter.cum_rbi+=1
                    #batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    if(rand_num<.06):
                        team.runners[1] = temp_0

                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runners advance
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.4+batter.bsr/15):    # If batter hits ground ball, out made at home, runner on second stays
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    #batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0
            
        if(made_out):
            return 'out'

        print("updated runners: " + str(team.runners))
        
    def print_result(self):
        """Printed score of a game
        """ 
        print("final score: " + self.away_team.city + " " + str(self.away_team.runs) + " " 
              + self.home_team.city + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams
        """ 
        print(str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played
        """ 
        self.away_team.restart()
        self.away_team.game_no+=1
        self.away_team.rotation_index = (self.away_team.game_no-1)%5
        self.home_team.restart() 
        self.home_team.game_no+=1
        self.home_team.rotation_index = (self.home_team.game_no-1)%5
        inning = 1
        game_over = False
        
        while(inning<10 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                if(inning==9 and side==self.home_team and self.home_team.runs>self.away_team.runs):
                    game_over = True
                    break
                print('inning: ' + str(inning))    
                outs = 0

                if(inning==1 and side==self.away_team):
                    # Determining starting pitchers
                    home_pitcher = self.home_team.choose_starter()
                    away_pitcher = self.away_team.choose_starter()
                    self.home_team.current_pitcher_list.append(home_pitcher)
                    self.home_team.current_pitcher = home_pitcher
                    self.away_team.current_pitcher_list.append(away_pitcher)
                    self.away_team.current_pitcher = away_pitcher
                    self.away_team.lineup, self.away_team.bench = self.away_team.make_lineup(self.away_team.batters, self.home_team.current_pitcher, self.home_team.league, self.away_team.lineup_dict)
                    self.home_team.lineup, self.home_team.bench = self.home_team.make_lineup(self.home_team.batters, self.away_team.current_pitcher, self.home_team.league, self.home_team.lineup_dict)
                    

                while(outs<3):
                    # Choosing a pitcher for a team based on game factors
                    if(side==self.away_team):
                        current_pitcher = self.home_team.current_pitcher
                        print(current_pitcher.name)
                        pitcher = self.home_team.choose_pitcher(outs, inning, side.runs, self.home_team.runs, side.lineup, 
                                                                side.batting_index, self.home_team.lineup, self.home_team.batting_index)
                        
                        if(current_pitcher!=pitcher and self.home_team.lineup[8].name=='pitcher'):   # If starting pitcher gets taken out of game and lineup, put bench player in to bat
                            self.home_team.lineup[8] = self.home_team.bench[random.randint(0, len(self.home_team.bench)-1)]
                            self.home_team.lineup[8].cum_G+=1
                            
                    elif(side==self.home_team):
                        current_pitcher = self.away_team.current_pitcher
                        print(current_pitcher.name)
                        pitcher = self.away_team.choose_pitcher(outs, inning, side.runs, self.away_team.runs, side.lineup, 
                                                                side.batting_index, self.away_team.lineup, self.away_team.batting_index)
                        
                        if(current_pitcher!=pitcher and self.away_team.lineup[8].name=='pitcher'):   # If starting pitcher gets taken out of game and lineup, put bench player in to bat
                            self.away_team.lineup[8] = self.away_team.bench[random.randint(0, len(self.away_team.bench)-1)]
                            self.away_team.lineup[8].cum_G+=1
                            
                    # Determining whether stolen base should be attempted
                    stolen_base_result = self.get_sb_wp_result(side.runners, pitcher, side, outs, random.uniform(0,1))
                    
                    if(stolen_base_result=='out'):   # If failed stolen base, out is recorded
                        pitcher.current_outs+=1
                        outs+=1
                                
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.runners_dict = {}
                            break
                    
                    # Determining outcome of at-bat
                    result = self.get_result(side.lineup[side.batting_index], pitcher, outs, side.runners, random.uniform(0,2))
                    print(side.lineup[side.batting_index].name)
                    print("result: " + result)
                    if(result=='out' or result=='k' or result=='sf'):    # Batter made an out
                        pitcher.current_outs+=1
                        outs+=1
                        print("outs: " + str(outs))
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                            
                    elif(result=='gdp'):    # Batter made an out
                        pitcher.current_outs+=2
                        outs+=2
                        print("outs: " + str(outs))
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                            
                    else:    # Batter reached, runners are moved
                        running_result = self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                        if(running_result=='out'):    # Baserunner made an out
                            pitcher.current_outs+=1
                            outs+=1
                            print("outs: " + str(outs))
                            
                            if(outs>2):   # If outs = 3, end of inning
                                pitcher.finished_half = True
                                side.batting_index+=1    # Gettting next batter in lineup
                                side.batting_index = side.batting_index%9
                                side.runners_dict = {}
                                break
                        print(side.city + ': '+ str(side.runs))
                        if(inning>9 and self.away_team.runs<self.home_team.runs):
                            game_over = True
                    
                    if(game_over):    # If game is over, escape game loop
                        break
                        
                    side.batting_index+=1    # Gettting next batter in lineup
                    side.batting_index = side.batting_index%9
                    
                if(game_over):    # If game is over, escape game loop
                    break
                        
                side.reset_runners()    # Runners are reset after half inning
            

            if(game_over):    # If game is over, escape game loop
                break   
                
            inning+=1
        
        # After game, updating team records and number of cumulative runs
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
        
class Season_new(Game_new):
    """Class that holds information for a Season of games.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team

    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(self.away_team, self.home_team)

            g.play() 
            
    def reset_season(self, teams):
        """Reinitializing teams
        
        Args:
            teams, Team instances to reinitialize
        """
        for team in teams:
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.relievers_df)
 
class FullSeason_new(Game_new):
    """Class that holds information for a Season of games
    
    Attributes:

    """
    
    def __init__(self, schedule):
        """Initializes values for this class
        
        Args:
            schedule: dataframe of season schedule
        """
        self.schedule = schedule

    def simulate_season(self):
        """A season of games is simulated
        """
        games = 0
        
        for date in self.schedule['date'].unique():
            for acronym, city in acronym_to_city_dict.items():
                away_team = team_dict[acronym_to_city_dict[acronym]]
                home_team_acronym = self.schedule.loc[self.schedule['date']==date][acronym].values[0]

                if(not pd.isna(home_team_acronym)):
                    home_team = team_dict[acronym_to_city_dict[home_team_acronym]]
                    g = Game(away_team, home_team)
                    g.play()
                    games+=1
                    print(games)
  
    def reset_season(self):
        """Reinitializing teams
        """
        for city_name, team in team_dict.items():
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters)   
            
    def print_record(self):
        """Printing records
        """
        for city_name, team in team_dict.items():
            print(team.city + ': ' + str(team.wins) + "-" + str(team.losses))
            
    def league_stats(self):
        """Accumulating stats for the entire league
        """
        pi = pd.DataFrame()
        ba = pd.DataFrame()
        for k, v in nickname_to_city_dict.items():

            a, b = team_dict[v].cum_stats_df()

            if(pi.empty):
                pi = b
            else:
                pi = pi.append(b)

            if(ba.empty):
                ba = a
            else:
                ba = ba.append(a)
        
        pi = pi.sort_values(by=['K'], ascending=False)
        pi.drop_duplicates(inplace=True)
        ba = ba.sort_values(by=['HR'], ascending=False)
        ba.drop_duplicates(inplace=True)
        
        return ba, pi

In [399]:
active_rosters

<function __main__.active_rosters()>

In [412]:
def convert_string(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(str(row[column]))
        else:
            return('0')

def convert_int(row, column):
    
    if(column in row.index):
         if(row[column]!=''):
            return(int(row[column]))
         else:
            return(0)
        
def convert_float(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(float(row[column]))
        else:
            return(0.0)
             
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
  
def writing_data(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)
    
def column_type(stats_df, batter):
    """Fixing column types of read in csv files
    Args:
        stats_df: dataframe of stats
        batter: True if working with batter statistics
    Returns:
        stats_df: Updated_stats
    """
    if(batter):
        string_columns = ['Season', 'Name', 'Bats', 'Tm', 'playerId']
        int_columns = ['G_R', 'PA_R', 'AB_R',
                       'H_R', '1B_R', '2B_R', '3B_R', 'HR_R', 'R_R', 'RBI_R', 'BB_R', 'IBB_R','SO_R', 'HBP_R', 'SF_R',
                       'SH_R', 'GDP_R', 'SB_R', 'CS_R', 'G_L', 'PA_L', 'AB_L', 'H_L', '1B_L', '2B_L', '3B_L', 'HR_L', 
                       'R_L', 'RBI_L', 'BB_L', 'IBB_L', 'SO_L', 'HBP_L', 'SF_L', 'SH_L', 'GDP_L', 'SB_L', 'CS_L']
        float_columns = ['AVG_R', 'AVG_L']
    else:
        string_columns = ['Season', 'Name', 'Throws', 'Team', 'playerId']
        int_columns = ['IP_L', 'TBF_L', 'H_L', '1B_L', '2B_L', '3B_L', 'R_L', 'ER_L', 'HR_L', 'BB_L', 'IBB_L', 
                       'HBP_L','SO_L', 'IP_R', 'TBF_R', 'H_R', '2B_R', '3B_R', 'R_R', 'ER_R', 'HR_R', 'BB_R', 'IBB_R', 
                       'HBP_R', 'SO_R']
        float_columns = ['ERA_L', 'AVG_L', 'OBP_L', 'SLG_L', 'wOBA_L', 'ERA_R', 'AVG_R', 'OBP_R', 'SLG_R', 'wOBA_R']

        for column in stats_df.columns:
            if(column in string_columns):
                stats_df[column] = stats_df.apply(lambda row: convert_string(row, column), axis=1)
            elif(column in int_columns):
                stats_df[column] = stats_df.apply(lambda row: convert_float(row, column), axis=1)
            elif(column in float_columns):
                stats_df[column] = stats_df.apply(lambda row: convert_float(row, column), axis=1)
   
            
    return stats_df
    
def date_formatter(row):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    if row.date == '':
        return(row.date)
    else:
        split_date = row.date.split('-')
        day = split_date[0]
        day = day.zfill(2)
        month = strptime(split_date[1],'%b').tm_mon
        date = str(month) +  '/' + day + '/' + '2020'
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        date = date.strftime('%m/%d/%Y')
        
    return date
    
def unknown_player_id(id_arm_dict, new_player_id, pitchers=False):
    """Finds batting or pitching side of players who do not have data in original stats dataframes
    
    Args:
        id_arm_dict: dict with player ID's as keys and arm side as values
        new_player_id: dict with player names as keys and their ID's as values
        pitchers: True if dict is of pitchers. Default=False
    Returns:    
        id_arm_dict: dict with ID's as keys and arm or batting side as values
    """

    p_dict = {}
    url_dict = {}
    url_start = 'https://www.fangraphs.com/players/'
    pos_modifier = 0
    
    if(pitchers):
        pos_modifier = 2
        
    for k, v in new_player_id.items():
        if(k[-1]=='.'):
            k = k[:-1]
        k = k.replace('\xa0', '')
        k = k.replace(' ', '-')
        p_dict.update({k: v})
        url_dict.update({v: url_start+k+'/'+v})


    for ID, url in url_dict.items():
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        for tag in soup.findAll('div', attrs={'class': 'player-info-box-item'}):

            if(tag.get_text()!='' and tag.get_text().split()[0]=='Bats/Throws:'):
                hand = tag.get_text().split(' ')[1][pos_modifier]
                if(hand=='B'):
                    hand = 'S'
                id_arm_dict.update({ID: hand})

    return id_arm_dict

def batter_stats():

    url = 'http://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2019/start/'
    links = [url+str(i) for i in range(1, 350, 50)]
    headers = {'User-Agent': header_name}

    batter_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            batter_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        batter_stats = pd.concat([batter_stats, df], ignore_index=True)

    return batter_stats

def pitcher_stats():
    url = 'http://www.espn.com/mlb/history/leaders/_/type/pitching/breakdown/season/year/2019/sort/wins/start/'
    links = [url+str(i) for i in range(1, 500, 50)]
    headers = {'User-Agent': header_name}

    pitcher_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            pitcher_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        pitcher_stats = pd.concat([pitcher_stats, df], ignore_index=True)

    return pitcher_stats

def projected_starters():

    url = 'https://www.mlb.com/news/projecting-every-mlb-lineup-rotation'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    starters = {}

    for body in soup.findAll('div', attrs={'class': 'article-item__body'}):
        pitchers = []
        batters = []
        closer = None

        for team in body.findAll('p'):
            if(team.strong and team.strong.get_text().lower() in nickname_to_city_dict.keys()):
                current_team = team.strong.get_text().lower()
                a = team.get_text().split(')')
                b = []
                c = []
                d = []

                for entry in a:
                    b.append(str(entry.split('/')[0]))
                for entry in b:
                    c.append(str(entry.split(',')[0]))
                for entry in c:
                    d.append(entry.split(' or ')[0])
            
                for entry in d[1:]:
                    batters.append(unidecode.unidecode(entry.lstrip()))
                
            elif(team.strong and team.strong.get_text()=='Pitchers'):
                a = team.get_text().split(')')
                b = []
                c = []
                d = []

                for entry in a:
                    b.append(str(entry.split('/')[0]))
                for entry in b:
                    c.append(str(entry.split(',')[0]))
                for entry in c:
                    d.append(entry.split(' or ')[0])
            
                for entry in d[1:]:
                    pitchers.append(unidecode.unidecode(entry.lstrip()))
            elif(team.strong and team.strong.get_text()=='Closer:'):
                closer = unidecode.unidecode(team.find(['forge-entity', 'a']).get_text())
            else:
                continue

            if(closer!=None):
                starters.update({nickname_to_city_dict[current_team]: 
                                {'lineup': batters, 'rotation': pitchers, 'closer': closer}})
                pitchers = []
                batters = []
                closer = None
            
    return starters

def projected_relievers():
    league_relief_dict = {}
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='

    for num in range(1,31):
        url_list.append(url_start+str(num))

    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        relief_df = pd.DataFrame(columns=['Name', 'IP'])


        for a_tag in soup.findAll('a', attrs={'href': '#RP'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')
            relief_table = a_tag.find_next('table')

            for row in relief_table.findAll('tr', attrs={'class': 'depth_reg'}):
                name_tag = row.find_next('td')
                name = name_tag.get_text()
                ip = name_tag.find_next('td').get_text()
                relief_df = relief_df.append({'Name': name, 'IP': ip}, ignore_index=True)

        league_relief_dict.update({nickname_to_city_dict[str(team_name.lower())]: relief_df})
        
    return league_relief_dict

def create_player_id_dict(batter_stats, pitcher_stats):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            batter_stats: df of batter stats
            pitcher_stats: df of pitcher stats
        Returns:
            team_dict: dict with city of team as key and Team instance as value
        """
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='
    team_dict = {}
    new_batter_id = {}
    new_pitcher_id = {}

    for num in range(1,31):
        url_list.append(url_start+str(num))
    url_list = ['https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=16', 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=9']
    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        positional_values_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()

        for a_tag in soup.findAll('a', attrs={'href': '#C'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')

        for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP', '#ALL']}):
            pos = a_tag.get_text()

            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'ALL Batters']):
                column_names = ['Name', 'ID', 'PA', 'AVG', 'OBP', 'SLG', 'wOBA', 'Bat', 'BsR', 'Fld', 'WAR']
            else:
                column_names = ['Name', 'ID', 'IP', 'K/9', 'BB/9', 'HR/9', 'BABIP', 'LOB', 'ERA', 'FIP', 'WAR']

            pos_df = pd.DataFrame(columns=column_names)

            table = a_tag.find_next('table')
            

            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                info = row.a['href']
                elements = info.split('=')
                player_ID = elements[1].split('&')[0]
                values_list = []

                for value in row.findAll('td'):
                    values_list.append(value.get_text())

                    if(len(values_list)==1):
                        values_list.append(player_ID)

                pos_df = pos_df.append(pd.Series(values_list, index=pos_df.columns), ignore_index=True)   

            positional_values_dict.update({pos: pos_df})   
        print(positional_values_dict)
        batter_df = pd.DataFrame()    
        pos_dict = {}

        for pos, df in positional_values_dict.items():
            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']):
                for index, row in df.iterrows():
                    if(row['ID'] in batter_stats['playerId'].values.tolist()):
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            continue
                        else:
                            continue
                    else:
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            continue
                        else:
                            new_batter_id.update({row['Name']: row['ID']})
                            print(row['Name']+ '  ' + pos)

                            
            elif(pos in ['SP', 'RP']):
                for index, row in df.iterrows():
                    if(row['ID'] in pitcher_stats['playerId'].values.tolist()):
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            continue
                        else:
                            continue
                    else:
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            continue
                        else:
                            new_pitcher_id.update({row['Name']: row['ID']})
                            print(row['Name']+ '  ' + pos)

    return new_batter_id, new_pitcher_id         

def daily_lineups():
    url = 'https://www.mlb.com/starting-lineups'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    lineup_dict = {}
    schedule_dict = {}
    schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')] = []
    for matchup in soup.findAll('div', attrs={'class': 'starting-lineups__matchup'}):
        for num, team_names in enumerate(matchup.findAll('a', attrs={'class': 'starting-lineups__team-name--link'})): # Getting team names
            if(num==0):
                away_team = nickname_to_city_dict[team_names.get_text().strip().lower()]
                
                if(away_team in lineup_dict.keys()):
                    away_team = away_team + '2'
            elif(num==1):
                home_team = nickname_to_city_dict[team_names.get_text().strip().lower()]
                
                if(home_team in lineup_dict.keys()):
                    home_team = home_team + '2'

        # Don't get lineup info if game is postponed        
        if(matchup.find('span', attrs={'class': 'starting-lineups__game-state starting-lineups__game-state--postponed'})):
            break

        schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')].append((away_team, home_team))  # Adding matchup to schedule dict

        for num, names in enumerate(matchup.findAll('div', attrs={'class': 'starting-lineups__pitcher-name'})): # Getting starter names
            for pitcher_names in names.findAll('a', attrs={'class': 'starting-lineups__pitcher--link'}):
                if(num==0):
                    away_starter = pitcher_names.get_text()
                elif(num==1):
                    home_starter = pitcher_names.get_text()


        away_lineup = {}
        home_lineup = {}

        # Gathering team lineups and putting them in a dict
        for lineups in matchup.findAll('div', attrs={'class': 'starting-lineups__teams starting-lineups__teams--sm starting-lineups__teams--xl'}):
            for a_lineup in lineups.findAll('ol', attrs={'class': 'starting-lineups__team starting-lineups__team--away'}):
                for batter in a_lineup.findAll('li'):
                    if(batter.get_text()=='TBD'):
                        break
                    away_lineup[batter.a.get_text()] = batter.span.get_text().split()[1]
                
                if(len(away_starter)==0):  # Setting starter to None if no starter found
                    away_starter = None
                if(len(away_lineup)==0):  # Setting lineup to None if no Lineup found
                    away_lineup = None
                    
                lineup_dict[away_team] = {'starter': away_starter, 'lineup': away_lineup}
                
            for h_lineup in lineups.findAll('ol', attrs={'class': 'starting-lineups__team starting-lineups__team--home'}):
                for batter in h_lineup.findAll('li'):
                    if(batter.get_text()=='TBD'):
                        break
                    home_lineup[batter.a.get_text()] = batter.span.get_text().split()[1]

                if(len(home_starter)==0):  # Setting starter to None if no starter found
                    home_starter = None
                if(len(home_lineup)==0):  # Setting lineup to None if no Lineup found
                    home_lineup = None
                    
                lineup_dict[home_team] = {'starter': home_starter, 'lineup': home_lineup}
        
    return lineup_dict, schedule_dict

def fangraphs_daily_lineups(lineup_dict):
    url = 'https://www.fangraphs.com/livescoreboard.aspx?'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    lineup_dict = {}
    schedule_dict = {}
    player_dict = {}
    schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')] = []

    header = soup.find('tr', attrs={'class': 'win-board__header'})

    for games in header.next_siblings:
        for matchup in games.findAll('td', style=True, align=False):
            away_team = None
            home_team = None
            
            for num, team_names in enumerate(matchup.findAll('a')): # Getting team names

                if(num==0):
                    if team_names.get_text().strip().capitalize() in league.keys():
                        away_team = team_names.get_text().strip().capitalize()
                    else:
                        away_team = nickname_to_city_dict[team_names.get_text().strip().lower()]

                    if(away_team in lineup_dict.keys()):
                        away_team = away_team + '2'
                elif(num==1):

                    if team_names.get_text().strip().capitalize() in league.keys():
                        home_team = team_names.get_text().strip().capitalize()
                    else:
                        home_team = nickname_to_city_dict[team_names.get_text().strip().lower()]

                    if(home_team in lineup_dict.keys()):
                        home_team = home_team + '2'

            if(away_team!=None and home_team!=None):   # Adding matchup to schedule dict
                schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')].append((away_team, home_team))  


            away_lineup_dict = {}
            away_lineup = []
            away_starter = None
            away_opener = None
            home_lineup_dict = {}
            home_lineup = []
            home_starter = None
            home_opener = None

            lineup = matchup.find('table', attrs={'class', 'lineup'})
            if(lineup!=None):
                for num, position in enumerate(lineup.findAll('td', attrs={'align': 'left'})):

                    if(num%2==0):
                        if position.get_text()[0:2] == 'SP':
                            away_starter = position.a.get_text().strip()
                            away_starter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            player_dict[away_starter] = away_starter_id
                        elif position.get_text()[0:2] == 'OP':
                            away_opener = position.a.get_text().strip()
                            away_opener_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            player_dict[away_opener] = away_opener_id
                        elif position.get_text()[0:2] not in ('SP', 'PP', 'OP'):
                            for player in position.findAll('a'):
                                away_batter = player.get_text().strip()
                                away_lineup.append(away_batter)
                                away_batter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                                player_dict[away_batter] = away_batter_id

                        away_lineup_dict['starter'] = away_starter
                        away_lineup_dict['opener'] = away_opener
                        away_lineup_dict['lineup'] = away_lineup

                    elif(num%2==1):
                        if position.get_text()[0:2] == 'SP':
                            home_starter = position.a.get_text().strip()
                            home_starter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            player_dict[home_starter] = home_starter_id
                        elif position.get_text()[0:2] == 'OP':
                            home_opener = position.a.get_text().strip()
                            home_opener_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            player_dict[home_opener] = home_opener_id
                        elif position.get_text()[0:2] not in ('SP', 'PP', 'OP'):
                            for player in position.findAll('a'):
                                home_batter = player.get_text().strip()
                                home_lineup.append(home_batter)
                                home_batter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                                player_dict[home_batter] = home_batter_id

                            home_lineup_dict['starter'] = home_starter
                            home_lineup_dict['opener'] = home_opener
                            home_lineup_dict['lineup'] = home_lineup    

                lineup_dict[away_team] = away_lineup_dict
                lineup_dict[home_team] = home_lineup_dict
        
    return lineup_dict, schedule_dict, player_dict

def simulate_slate(lineup_dict, schedule_dict, num_games=400):
    
    #for name, team in team_dict.items():        
        #team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.lineup_list, team.sp)
    
    win_rate = pd.Series()
    for matchup in schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')]:
        away_team = matchup[0]
        home_team = matchup[1]
        
        for num in range(num_games):
            g = Game(team_dict[away_team], team_dict[home_team])
            g.play()
            g.print_result()

        win_rate = win_rate.append(pd.Series([team_dict[away_team].wins/num_games], index=[away_team]))
        win_rate = win_rate.append(pd.Series([team_dict[home_team].wins/num_games], index=[home_team]))
    
    
    return win_rate

def simulate_slate_new(lineup_dict, rosters_dict, schedule_dict, num_games=400):
    
    for team, lineup in lineup_dict.items():        
        team.__init__(team, league[team], lineup, rosters_dict[team])
    
    win_rate = pd.Series()
    for matchup in schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')]:
        away_team = matchup[0]
        home_team = matchup[1]
        
        for num in range(num_games):
            g = Game(team_dict[away_team], team_dict[home_team])
            g.play()
            g.print_result()

        win_rate = win_rate.append(pd.Series([team_dict[away_team].wins/num_games], index=[away_team]))
        win_rate = win_rate.append(pd.Series([team_dict[home_team].wins/num_games], index=[home_team]))
    
    
    return win_rate

def create_teams(batter_projections, batter_stats_2019, pitcher_projections, pitcher_stats_2019, lineup_dict={}):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            batter_stats: df of batter stats
            pitcher_stats: df of pitcher stats
            lineup_dict: dict of team lineups and starting pitchers
        Returns:
            team_dict: dict with city of team as key and Team instance as value
        """
    global id_arms_dict, id_bats_dict, batter_stats_df, pitcher_stats_df
    
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='
    team_dict = {}
    new_player_id = {}
    new_batter_df = pd.DataFrame()
    new_pitcher_df = pd.DataFrame()
    batter_headers = ['Season', 'Name', 'playerId', 'Team', 'Bats', 'G_L', 'PA_L', 'AB_L',
                       'H_L', '1B_L', '2B_L', '3B_L', 'HR_L', 'R_L', 'RBI_L', 'BB_L', 'IBB_L',
                       'SO_L', 'HBP_L', 'SF_L', 'SH_L', 'GDP_L', 'SB_L', 'CS_L', 'AVG_L',
                       'G_R', 'PA_R', 'AB_R', 'H_R', '1B_R', '2B_R', '3B_R', 'HR_R', 'R_R',
                       'RBI_R', 'BB_R', 'IBB_R', 'SO_R', 'HBP_R', 'SF_R', 'SH_R', 'GDP_R',
                       'SB_R', 'CS_R', 'AVG_R']
    
    pitcher_headers = ['Season', 'Name', 'playerId', 'Team', 'Throws', 'IP_L', 'TBF_L',
                       'ERA_L', 'H_L', '1B_L', '2B_L', '3B_L', 'R_L', 'ER_L', 'HR_L', 'BB_L',
                       'IBB_L', 'HBP_L', 'SO_L', 'AVG_L', 'OBP_L', 'SLG_L', 'wOBA_L', 'IP_R',
                       'TBF_R', 'ERA_R', 'H_R', '1B_R', '2B_R', '3B_R', 'R_R', 'ER_R', 'HR_R',
                       'BB_R', 'IBB_R', 'HBP_R', 'SO_R', 'AVG_R', 'OBP_R', 'SLG_R', 'wOBA_R']

        
    for name, num in team_num_dict.items():
        if(name in lineup_dict.keys()):
            url_list.append(url_start+str(num))
    #url_list = ['https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=13', 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=30']
    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        positional_values_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()

        for a_tag in soup.findAll('a', attrs={'href': '#C'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')
    
        if(nickname_to_city_dict[team_name.lower()] not in lineup_dict.keys()):
            continue
        for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP', '#ALL']}):
            pos = a_tag.get_text()

            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'ALL Batters']):
                column_names = ['Name', 'ID', 'PA', 'AVG', 'OBP', 'SLG', 'wOBA', 'Bat', 'BsR', 'Fld', 'WAR']
            else:
                column_names = ['Name', 'ID', 'IP', 'K/9', 'BB/9', 'HR/9', 'BABIP', 'LOB', 'ERA', 'FIP', 'WAR']

            pos_df = pd.DataFrame(columns=column_names)

            table = a_tag.find_next('table')


            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                info = row.a['href']
                elements = info.split('=')
                player_ID = elements[1].split('&')[0]
                values_list = []

                for value in row.findAll('td'):
                    values_list.append(value.get_text())

                    if(len(values_list)==1):
                        values_list.append(player_ID)

                pos_df = pos_df.append(pd.Series(values_list, index=pos_df.columns), ignore_index=True)   

            positional_values_dict.update({pos: pos_df})   

        pos_dict = {}

        for pos, df in positional_values_dict.items():
            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']):
                for index, row in df.iterrows():
                    if(row['ID'] in batter_stats_2019['playerId'].values.tolist() and
                       batter_stats_2019.loc[batter_stats_2019['playerId']==row['ID']]['PA_L'].item()>80 and
                       batter_stats_2019.loc[batter_stats_2019['playerId']==row['ID']]['PA_R'].item()>150):
                        
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            batter_df.loc[batter_df['playerId']==row['ID'], pos] = int(row['PA'])
                        else:
                            stats = batter_stats_2019.loc[batter_stats_2019['playerId']==row['ID']]
                            stats[pos] = int(row['PA'])
                            stats['BsR'] = float(row['BsR'])
                            stats['Fld'] = float(row['Fld'])
                            col_headers = batter_headers

                            if(batter_df.empty):
                                batter_df = pd.DataFrame(columns=batter_headers)

                            batter_df = batter_df.append(stats, ignore_index=True) 
                    elif(row['ID'] in batter_projections['playerId'].values.tolist()):
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            batter_df.loc[batter_df['playerId']==row['ID'], pos] = int(row['PA'])
                        else:
                            stats = batter_projections.loc[batter_projections['playerId']==row['ID']]
                            stats[pos] = int(row['PA'])
                            stats['BsR'] = float(row['BsR'])
                            stats['Fld'] = float(row['Fld'])
                            col_headers = batter_headers

                            if(batter_df.empty):
                                batter_df = pd.DataFrame(columns=batter_headers)

                            batter_df = batter_df.append(stats, ignore_index=True) 
                    else:
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            batter_df.loc[batter_df['playerId']==row['ID'], pos] = int(row['PA'])
                        else:
                            player_df = positional_values_dict['ALL Batters']
                            if(row['ID'] not in id_bats_dict.keys()):
                                id_bats_dict = unknown_player_id(id_bats_dict, {row['Name'].strip(): row['ID']}, False)
                            
                            bats = id_bats_dict[row['ID']]

                            new_player_id.update({row['Name'].strip(): row['ID']})
                            player_values = player_df.loc[player_df['ID']==row['ID']]

                            if(int(player_values['PA'])<20):
                                stats = pd.Series({'Season': 2019, 'Name': row['Name'].strip(), 'playerId': row['ID'], 'Team': nickname_to_city_dict[team_name.lower()], 
                                                   'Bats': bats, 'G_L': 60, 'PA_L': 220, 'AB_L': 200, 'H_L': 50, '1B_L': 34, '2B_L': 10, 
                                                   '3B_L': 1, 'HR_L': 5, 'R_L': 25, 'RBI_L': 25, 'BB_L': 20, 'IBB_L': 0, 
                                                   'SO_L': 50, 'HBP_L': 3, 'SF_L': 2, 'SH_L': 0, 'GDP_L': 3, 'SB_L': 5, 
                                                   'CS_L': 1, 'AVG_L': .250, 'G_L': 60, 'PA_R': 220, 'AB_R': 200, 'H_R': 50, 
                                                   '1B_R': 34, '2B_R': 10, '3B_R': 1,'HR_R': 5, 'R_R': 25, 'RBI_R': 25, 
                                                   'BB_R': 20, 'IBB_R': 0, 'SO_R': 50, 'HBP_R': 3,'SF_R': 2, 'SH_R': 0, 'GDP_R': 3, 
                                                   'SB_R': 5, 'CS_R': 1, 'AVG_R': .250, 'BsR': 0, 'Fld': 0, pos: int(row['PA'])})

                            else:
                                if(bats=='L'):
                                    multiplier = 1.05
                                elif(bats=='R'):
                                    multiplier = 1/1.05
                                else:
                                    multiplier = 1

                                pa = int(player_values['PA'])
                                obp = float(player_values['OBP'])
                                avg = float(player_values['AVG'])
                                slg = float(player_values['SLG'])
                                
                                on_base = round(obp*pa)
                                hits = on_base
                                G_L = pa/(8*multiplier)
                                G_R = pa*multiplier*1/4
                                PA_L = pa/(3*multiplier)
                                PA_R = pa-PA_L
                                R_L = math.floor(PA_L*.15)
                                R_R = math.floor(PA_R*.15)
                                RBI_L = math.floor(PA_L*.15)
                                RBI_R = math.floor(PA_R*.15)
                                HBP_L = PA_L/100
                                HBP_R = PA_R/100
                                SF_L = PA_L/180
                                SF_R = PA_R/180
                                SH_L = PA_L/300
                                SH_R = PA_R/300
                                GDP_L = PA_L/50
                                GDP_R = PA_R/50
                                SB_L = max(0, float(player_values['BsR'])*8) + PA_L/100
                                SB_R = max(0, float(player_values['BsR'])*10) + PA_R/100
                                CS_L = SB_L/4
                                CS_R = SB_R/4

                                for num in range(1, on_base):
                                    attempt = on_base-num
                                    walks = num
                                    average = attempt/(pa-num)
                                    if(average<avg):
                                        hits = attempt
                                        break

                                BB_L = walks*5/18
                                BB_R = walks-BB_L
                                AB_L = PA_L-BB_L
                                AB_R = PA_R-BB_R
                                SO_L = (pa-on_base)*(.24*multiplier+(.25-avg))
                                SO_R = (pa-on_base)*(.24/multiplier+(.25-avg))

                                H_L = hits/(3*multiplier*multiplier)
                                AVG_L = H_L/AB_L
                                singles_L = H_L*.58/multiplier
                                doubles_L = H_L*.22/multiplier
                                triples_L = H_L/(70*multiplier)
                                HR_L = H_L-singles_L-doubles_L-triples_L
                                temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L
                                
                                H_R = hits-H_L
                                AVG_R = H_R/AB_R
                                singles_R = H_R*.58*multiplier
                                doubles_R = H_R*.2*multiplier
                                triples_R = H_R*multiplier/70
                                HR_R = H_R-singles_R-doubles_R-triples_R
                            
                                temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R
                                
                                if(temp_slg_L>slg/multiplier):
                                    while(temp_slg_L>slg/multiplier and doubles_L>.2):
                                        doubles_L-=.2
                                        singles_L+=.2
                                        temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L
                                elif(temp_slg_L<slg/multiplier):
                                    while(temp_slg_L<slg/multiplier and singles_L>.2):
                                        doubles_L+=.2
                                        singles_L-=.2
                                        temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L

                                if(temp_slg_R>slg*multiplier):
                                    while(temp_slg_R>slg/multiplier and doubles_R>.2):
                                        doubles_R-=.2
                                        singles_R+=.2
                                        temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R
                                elif(temp_slg_R<slg*multiplier):
                                    while(temp_slg_R<slg*multiplier and singles_R>.2):
                                        doubles_R+=.2
                                        singles_R-=.2
                                        temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R

                                stats = pd.Series({'Season': 2019, 'Name': row['Name'].strip(), 'playerId': row['ID'], 'Team': nickname_to_city_dict[team_name.lower()], 
                                                   'Bats': bats, 'G_L': G_L, 'PA_L': PA_L, 'AB_L': AB_L, 'H_L': H_L, '1B_L':singles_L, '2B_L': doubles_L, 
                                                   '3B_L': triples_L, 'HR_L': HR_L, 'R_L': R_L, 'RBI_L': RBI_L, 'BB_L': BB_L, 'IBB_L': 0, 
                                                   'SO_L': SO_L, 'HBP_L': HBP_L, 'SF_L': SF_L, 'SH_L': SH_L, 'GDP_L': GDP_L, 'SB_L': SB_L, 
                                                   'CS_L': CS_L, 'AVG_L': AVG_L, 'G_R': G_R, 'PA_R': PA_R, 'AB_R': AB_R, 'H_R': H_R, '1B_R': singles_R, 
                                                   '2B_R': doubles_R, '3B_R': triples_R,'HR_R': HR_R, 'R_R': R_R, 'RBI_R': RBI_R, 'BB_R': BB_R, 'IBB_R': 0, 
                                                   'SO_R': SO_R, 'HBP_R': HBP_R,'SF_R': SF_R, 'SH_R': SH_R, 'GDP_R': GDP_R, 'SB_R': SB_R, 'CS_R': CS_R, 
                                                   'AVG_R': AVG_R, 'BsR': float(player_values['BsR']), 'Fld': float(player_values['Fld']), pos: int(row['PA'])}) 

                            if(batter_df.empty):
                                batter_df = pd.DataFrame(columns=batter_headers)

                            batter_df = batter_df.append(stats, ignore_index=True)
                            new_batter_df = new_batter_df.append(stats, ignore_index=True)
                            
            elif(pos in ['SP', 'RP']):
                for index, row in df.iterrows():
                    if(row['ID'] in pitcher_stats_2019['playerId'].values.tolist()
                       and pitcher_stats_2019.loc[pitcher_stats_2019['playerId']==row['ID']]['IP_L'].item()>40
                       and pitcher_stats_2019.loc[pitcher_stats_2019['playerId']==row['ID']]['IP_R'].item()>60):
                        
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            pitcher_df.loc[pitcher_df['playerId']==row['ID'], pos] = float(row['IP'])
                        else:
                            stats = pitcher_stats_2019.loc[pitcher_stats_2019['playerId']==row['ID']]
                            stats[pos] = float(row['IP'])

                            if(pitcher_df.empty):
                                pitcher_df = pd.DataFrame(columns=pitcher_headers)

                            pitcher_df = pitcher_df.append(stats, ignore_index=True) 
                    elif(row['ID'] in pitcher_projections['playerId'].values.tolist()):
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            pitcher_df.loc[pitcher_df['playerId']==row['ID'], pos] = float(row['IP'])
                        else:
                            stats = pitcher_projections.loc[pitcher_projections['playerId']==row['ID']]
                            stats[pos] = float(row['IP'])

                            if(pitcher_df.empty):
                                pitcher_df = pd.DataFrame(columns=pitcher_headers)

                            pitcher_df = pitcher_df.append(stats, ignore_index=True)
                    else:
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            pitcher_df.loc[pitcher_df['playerId']==row['ID'], pos] = float(row['IP'])
                        else:
                            player_df = positional_values_dict['ALL Pitchers']
                            if(row['ID'] not in id_arms_dict.keys()):
                                id_arms_dict = unknown_player_id(id_arms_dict, {row['Name']: row['ID']}, True)
                            throws = id_arms_dict[row['ID']]

                            new_player_id.update({row['Name'].strip(): row['ID']})
                            player_values = player_df.loc[player_df['ID']==row['ID']]

                            if(float(player_values['IP'])<1):
                                stats = pd.Series({'Season': 2019, 'Name': row['Name'].strip(), 'playerId': row['ID'], 'Team': nickname_to_city_dict[team_name.lower()], 
                                                   'Throws': throws, 'IP_L': 30, 'TBF_L': 127, 'ERA_L': 5.40, 'H_L': 30, '1B_L': 20, 
                                                   '2B_L': 5, '3B_L': 1, 'R_L': 18, 'ER_L': 18, 'HR_L': 4, 'BB_L': 17, 'IBB_L': 0,  
                                                   'HBP_L': 3, 'SO_L': 50, 'AVG_L': .250, 'OBP_L': .385, 'SLG_L': .544, 
                                                   'wOBA_L': .370, 'IP_R': 30, 'TBF_R': 127, 'ERA_R': 5.40, 'H_R': 30, 
                                                   '1B_R': 20, '2B_R': 5, '3B_R': 1, 'R_R': 18, 'ER_R': 18, 'HR_R': 4, 
                                                   'BB_R': 17, 'IBB_R': 0, 'HBP_R': 3, 'SO_R': 50, 'AVG_R': .250, 
                                                   'OBP_R': .385, 'SLG_R': .544, 'wOBA_R': .370, pos: int(float(row['IP']))})


                            else:
                                if(throws=='L'):
                                    multiplier = 1.05
                                elif(throws=='R'):
                                    multiplier = 1/1.05
                                else:
                                    multiplier = 1

                                ip = int(round(float(player_values['IP'])))
                                ip_L = ip*.4
                                ip_R = ip-ip_L
                                outs_L = ip_L*3
                                outs_R = ip_R*3
                                SO_L = float(player_values['K/9'])*ip_L*multiplier/9
                                SO_R = float(player_values['K/9'])*ip_L/(multiplier*9)
                                ER_L = ip_L*float(player_values['ERA'])/(multiplier*9)
                                ER_R = ip_R*float(player_values['ERA'])*multiplier/9
                                R_L = ER_L*1.07
                                R_R = ER_R*1.07
                                ERA_L = ER_L*27/outs_L
                                ERA_R = ER_R*27/outs_R
                                BB_L = float(player_values['BB/9'])*ip_L/(multiplier*9)
                                BB_R = float(player_values['BB/9'])*ip_R*multiplier/9
                                temp_whip_L = ERA_L/(10*multiplier) + .8
                                temp_whip_R = ERA_R*multiplier/10 + .8
                                temp_wh_L = temp_whip_L*ip_L
                                temp_wh_R = temp_whip_R*ip_R
                                HR_L =  float(player_values['HR/9'])*ip_L/(multiplier*9)
                                HR_R = float(player_values['HR/9'])*ip_R*multiplier/9
                                triples_L = outs_L*.005
                                triples_R = outs_R*.005
                                doubles_L = outs_L/(17*multiplier)
                                doubles_R = outs_R*multiplier/17            
                                singles_L = temp_wh_L-HR_L-triples_L-doubles_L-BB_L
                                singles_R = temp_wh_R-HR_R-triples_R-doubles_R-BB_R
                                TBF_L = .98*(outs_L+temp_wh_L)
                                TBF_R = .98*(outs_R+temp_wh_R)
                                H_L = singles_L+doubles_L+triples_L+HR_L
                                H_R = singles_R+doubles_R+triples_R+HR_R
                                HBP_L = TBF_L*.01
                                HBP_R = TBF_R*.01
                                IBB_L = BB_L/200
                                IBB_R = BB_R/200
                                AVG_L = H_L/(TBF_L-HBP_L-BB_L)
                                AVG_R = H_R/(TBF_R-HBP_R-BB_R)
                                OBP_L = (H_L+BB_L+HBP_L)/TBF_L
                                OBP_R = (H_R+BB_R+HBP_R)/TBF_R
                                SLG_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/(TBF_L-HBP_L-BB_L)
                                SLG_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/(TBF_R-HBP_R-BB_R)
                                woba_L = (.69*IBB_L+.719*HBP_L+.87*singles_L+1.217*doubles_L+1.529*triples_L+1.94*HR_L)/(TBF_L-IBB_L+HBP_L+.6*TBF_L)
                                woba_R = (.69*IBB_R+.719*HBP_R+.87*singles_R+1.217*doubles_R+1.529*triples_R+1.94*HR_R)/(TBF_R-IBB_R+HBP_R+.6*TBF_R)
                                
                                stats = pd.Series({'Season': 2019, 'Name': row['Name'].strip(), 'playerId': row['ID'], 'Team': nickname_to_city_dict[team_name.lower()], 
                                                   'Throws': throws, 'IP_L': ip_L, 'TBF_L': TBF_L, 'ERA_L': ERA_L, 'H_L': H_L, '1B_L': singles_L, 
                                                   '2B_L': doubles_L, '3B_L': triples_L, 'R_L': R_L, 'ER_L': ER_L, 'HR_L': HR_L, 'BB_L': BB_L, 'IBB_L': IBB_L,  
                                                   'HBP_L': HBP_L, 'SO_L': SO_L, 'AVG_L': AVG_L, 'OBP_L': OBP_L, 'SLG_L': SLG_L, 
                                                   'wOBA_L': woba_L, 'IP_R': ip_R, 'TBF_R': TBF_R, 'ERA_R': ERA_R, 'H_R': H_R, 
                                                   '1B_R': singles_R, '2B_R': doubles_R, '3B_R': triples_R, 'R_R': R_R, 'ER_R': ER_R, 'HR_R': HR_R, 
                                                   'BB_R': BB_R, 'IBB_R': IBB_R, 'HBP_R': HBP_R, 'SO_R': SO_R, 'AVG_R': AVG_R, 
                                                   'OBP_R': OBP_R, 'SLG_R': SLG_R, 'wOBA_R': woba_R, pos: int(float(row['IP']))})
                   
                            if(pitcher_df.empty):
                                pitcher_df = pd.DataFrame(columns=pitcher_headers)

                            pitcher_df = pitcher_df.append(stats, ignore_index=True)
                            new_pitcher_df = new_pitcher_df.append(stats, ignore_index=True)
                
        batter_df = batter_df.append(batter_stats_2019.loc[batter_stats_2019['Name']=='Avg_Totals'])
        batter_df.fillna(0, inplace=True)
        
        pitcher_df = pitcher_df.append(pitcher_stats_2019.loc[pitcher_stats_2019['Name']=='Avg_Totals'])
        pitcher_df.fillna(0, inplace=True)

        for pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']:
            batter_df.loc[batter_df.Name=='Avg_Totals', pos] = batter_df.sum()[pos]
            
        batter_df.loc[batter_df.Name=='Avg_Totals', 'BsR'] = batter_df.sum()['BsR']
        batter_df.loc[batter_df.Name=='Avg_Totals', 'Fld'] = batter_df.sum()['Fld']
            
        for pos in ['SP', 'RP']:
            pitcher_df.loc[pitcher_df.Name=='Avg_Totals', pos] = pitcher_df.sum()[pos]    

        city = nickname_to_city_dict[team_name.lower()]

        if(city+'2' in lineup_dict.keys()):
            team_dict.update({city+'2': Team(city, league[city], batter_df, pitcher_df, projected_starters()[city], 
                                             lineup_dict[city+'2']['lineup'], lineup_dict[city+'2']['starter'])})

        team_dict.update({city: Team(city, league[city], batter_df, pitcher_df, projected_starters()[city], 
                                             lineup_dict[city]['lineup'], lineup_dict[city]['starter'])})
    
    batter_projections = batter_projections.append(new_batter_df, ignore_index=True)
    if(not new_batter_df.empty):  
        writing_data(batter_projections, 'Batter_Projections_FanGraphs_2020.csv')
        
    pitcher_projections = pitcher_projections.append(new_pitcher_df, ignore_index=True)
    if(not new_pitcher_df.empty):  
        writing_data(pitcher_projections, 'Pitcher_Projections_FanGraphs_2020.csv')
        
    return team_dict

def create_teams_new(batter_stats, pitcher_stats, lineup_dict={}, rosters={}):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            batter_stats: df of batter stats
            pitcher_stats: df of pitcher stats
            lineup_dict: dict of team lineups and starting pitchers
            rosters: active rosters of players for each team
        Returns:
            team_dict: dict with city of team as key and Team instance as value
        """
    global id_arms_dict, id_bats_dict, batter_stats_df, pitcher_stats_df
    
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='
    
    for name, num in team_num_dict.items():
        if(name in lineup_dict.keys()):
            url_list.append(url_start+str(num))
    
    batter_stats = column_type(reading_data('Joined_Batter_Stats.csv'), True)
    batter_stats = batter_stats.replace(np.nan, 0)
    pitcher_stats = column_type(reading_data('Joined_Pitcher_Stats.csv'), True)
    pitcher_stats = pitcher_stats.replace(np.nan, 0)
    #url_list = ['https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=5']
    
    for url in url_list:    # Getting players on each team
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        positional_values_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()
        pos_dict = {}

        for a_tag in soup.findAll('a', attrs={'href': '#C'}):
            team_name = a_tag.find_next('span').get_text().strip()

            print(team_name.capitalize())
            if team_name not in league.keys() and team_name.capitalize() not in league.keys():
                team_name = nickname_to_city_dict[team_name.replace('\xa0', '').lower()]
            
        for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP']}):#, '#ALL']}):
            pos = a_tag.get_text()
            pos_dict[pos] = {}

            table = a_tag.find_next('table')

            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                if not row.a:
                    continue
                    
                info = row.a['href']
                elements = info.split('=')
                player_ID = elements[1].split('&')[0]

                if not player_ID.isdigit():
                    continue

                player_name = row.a.get_text().strip()
                pos_dict[pos][player_ID] = player_name

        for position, player_info in pos_dict.items():
            for player_ID, player_name in player_info.items():
                
                if position not in rosters[team_name].keys() or player_name not in rosters[team_name][position]:
                    print(player_name)
                    continue

                if position in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']:

                    if(not batter_df.empty and player_ID in batter_df['playerID'].values.tolist()):  # player already in batter_df at different position
                        batter_df.loc[batter_df['playerID']==player_ID, position] = 1
                    else:

                        batter_headers = ['Name', 'playerID', 'Team', 'Bats', 'PA_L', 'AB_L',
                                           'singles_L', 'doubles_L', 'triples_L', 'HR_L', 
                                           'BB_L', 'SO_L', 'HBP_L', 'SB_L', 'CS_L', 
                                           'PA_R', 'AB_R', 'singles_R', 'doubles_R', 
                                           'triples_R', 'HR_R', 'BB_R', 'SO_R', 'HBP_R', 
                                           'SB_R', 'CS_R', 'PA_Home', 'AB_Home',
                                           'singles_Home', 'doubles_Home', 'triples_Home', 'HR_Home', 
                                           'BB_Home', 'SO_Home', 'HBP_Home', 'SB_Home', 'CS_Home', 
                                           'PA_Away', 'AB_Away', 'singles_Away', 'doubles_Away', 
                                           'triples_Away', 'HR_Away', 'BB_Away', 'SO_Away', 'HBP_Away', 
                                           'SB_Away', 'CS_Away', 'C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'SP', 'RP']

                        player_stats = batter_stats.loc[batter_stats['playerId']==int(player_ID)]

                        if player_stats.empty:  # Change this after finding handedness of all players
                            continue

                        if(player_stats['Bats'].tolist()[0]=='L'):
                            PA_L = 50
                            AB_L = 45
                            singles_L = 7
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 2
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 12
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 12
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 4
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 18
                            SB_R = 1
                            CS_R = 0
                        elif(player_stats['Bats'].tolist()[0]=='R'):
                            PA_L = 50
                            AB_L = 45
                            singles_L = 8
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 3
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 10
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 10
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 3
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 20
                            SB_R = 1
                            CS_R = 0
                        else:
                            PA_L = 50
                            AB_L = 45
                            singles_L = 8
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 3
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 10
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 12
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 4
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 18
                            SB_R = 1
                            CS_R = 0

                        PA_Home = 80
                        AB_Home = 72
                        singles_Home = 12
                        doubles_Home = 4
                        triples_Home = 0
                        HR_Home = 4
                        BB_Home = 7
                        HBP_Home = 1
                        SO_Home = 18
                        SB_Home = 1
                        CS_Home = 0

                        PA_Away = 80
                        AB_Away = 72
                        singles_Away = 12
                        doubles_Away = 4
                        triples_Away = 0
                        HR_Away = 4
                        BB_Away = 7
                        HBP_Away = 1
                        SO_Away = 18
                        SB_Away = 1
                        CS_Away = 0

                        seasons_list = sorted(player_stats['Season'].to_list())
                        for year in seasons_list:
                            multiplier = (1-(max(seasons_list)-year)/len(seasons_list))

                            PA_L += player_stats.loc[player_stats['Season']==year]['PA_L'].values[0]*multiplier
                            AB_L += player_stats.loc[player_stats['Season']==year]['AB_L'].values[0]*multiplier
                            singles_L += player_stats.loc[player_stats['Season']==year]['singles_L'].values[0]*multiplier
                            doubles_L += player_stats.loc[player_stats['Season']==year]['doubles_L'].values[0]*multiplier
                            triples_L += player_stats.loc[player_stats['Season']==year]['triples_L'].values[0]*multiplier
                            HR_L += player_stats.loc[player_stats['Season']==year]['HR_L'].values[0]*multiplier
                            BB_L += player_stats.loc[player_stats['Season']==year]['BB_L'].values[0]*multiplier
                            HBP_L += player_stats.loc[player_stats['Season']==year]['HBP_L'].values[0]*multiplier
                            SO_L += player_stats.loc[player_stats['Season']==year]['SO_L'].values[0]*multiplier
                            #SB_L += player_stats.loc[player_stats['Season']==year]['SB_L'].values[0]*multiplier
                            #CS_L += player_stats.loc[player_stats['Season']==year]['CS_L'].values[0]*multiplier

                            PA_R += player_stats.loc[player_stats['Season']==year]['PA_R'].values[0]*multiplier
                            AB_R += player_stats.loc[player_stats['Season']==year]['AB_R'].values[0]*multiplier
                            singles_R += player_stats.loc[player_stats['Season']==year]['singles_R'].values[0]*multiplier
                            doubles_R += player_stats.loc[player_stats['Season']==year]['doubles_R'].values[0]*multiplier
                            triples_R += player_stats.loc[player_stats['Season']==year]['triples_R'].values[0]*multiplier
                            HR_R += player_stats.loc[player_stats['Season']==year]['HR_R'].values[0]*multiplier
                            BB_R += player_stats.loc[player_stats['Season']==year]['BB_R'].values[0]*multiplier
                            HBP_R += player_stats.loc[player_stats['Season']==year]['HBP_R'].values[0]*multiplier
                            SO_R += player_stats.loc[player_stats['Season']==year]['SO_R'].values[0]*multiplier
                            #SB_R += player_stats.loc[player_stats['Season']==year]['SB_R'].values[0]*multiplier
                            #CS_R += player_stats.loc[player_stats['Season']==year]['CS_R'].values[0]*multiplier

                            PA_Home += player_stats.loc[player_stats['Season']==year]['PA_Home'].values[0]*multiplier
                            AB_Home += player_stats.loc[player_stats['Season']==year]['AB_Home'].values[0]*multiplier
                            singles_Home += player_stats.loc[player_stats['Season']==year]['singles_Home'].values[0]*multiplier
                            doubles_Home += player_stats.loc[player_stats['Season']==year]['doubles_Home'].values[0]*multiplier
                            triples_Home += player_stats.loc[player_stats['Season']==year]['triples_Home'].values[0]*multiplier
                            HR_Home += player_stats.loc[player_stats['Season']==year]['HR_Home'].values[0]*multiplier
                            BB_Home += player_stats.loc[player_stats['Season']==year]['BB_Home'].values[0]*multiplier
                            HBP_Home += player_stats.loc[player_stats['Season']==year]['HBP_Home'].values[0]*multiplier
                            SO_Home += player_stats.loc[player_stats['Season']==year]['SO_Home'].values[0]*multiplier
                            #SB_Home += player_stats.loc[player_stats['Season']==year]['SB_Home'].values[0]*multiplier
                            #CS_Home += player_stats.loc[player_stats['Season']==year]['CS_Home'].values[0]*multiplier

                            PA_Away += player_stats.loc[player_stats['Season']==year]['PA_Away'].values[0]*multiplier
                            AB_Away += player_stats.loc[player_stats['Season']==year]['AB_Away'].values[0]*multiplier
                            singles_Away += player_stats.loc[player_stats['Season']==year]['singles_Away'].values[0]*multiplier
                            doubles_Away += player_stats.loc[player_stats['Season']==year]['doubles_Away'].values[0]*multiplier
                            triples_Away += player_stats.loc[player_stats['Season']==year]['triples_Away'].values[0]*multiplier
                            HR_Away += player_stats.loc[player_stats['Season']==year]['HR_Away'].values[0]*multiplier
                            BB_Away += player_stats.loc[player_stats['Season']==year]['BB_Away'].values[0]*multiplier
                            HBP_Away += player_stats.loc[player_stats['Season']==year]['HBP_Away'].values[0]*multiplier
                            SO_Away += player_stats.loc[player_stats['Season']==year]['SO_Away'].values[0]*multiplier
                            #SB_Away += player_stats.loc[player_stats['Season']==year]['SB_Away'].values[0]*multiplier
                            #CS_Away += player_stats.loc[player_stats['Season']==year]['CS_Away'].values[0]*multiplier


                        stats = pd.Series({'Name': player_name, 'playerID': player_ID, 'Team': team_name, 
                                           'Bats': player_stats['Bats'].tolist()[0], 'PA_L': PA_L, 'AB_L': AB_L,
                                           'singles_L': singles_L, 'doubles_L': doubles_L, 'triples_L': triples_L, 'HR_L': HR_L, 
                                           'BB_L': BB_L, 'SO_L': SO_L, 'HBP_L': HBP_L, 'SB_L': SB_L, 'CS_L': CS_L, 
                                           'PA_R': PA_R, 'AB_R': AB_R, 'singles_R': singles_R, 'doubles_R': doubles_R, 
                                           'triples_R': triples_R, 'HR_R': HR_R, 'BB_R': BB_R, 'SO_R': SO_R, 'HBP_R': HBP_R, 
                                           'SB_R': SB_R, 'CS_R': CS_R, 'PA_Home': PA_Home, 'AB_Home': AB_Home,
                                           'singles_Home': singles_Home, 'doubles_Home': doubles_Home, 'triples_Home': triples_Home, 'HR_Home': HR_Home, 
                                           'BB_Home': BB_Home, 'SO_Home': SO_Home, 'HBP_Home': HBP_Home, 'SB_Home': SB_Home, 'CS_Home': CS_Home, 
                                           'PA_Away': PA_Away, 'AB_Away': AB_Away, 'singles_Away': singles_Away, 'doubles_Away': doubles_Away, 
                                           'triples_Away': triples_Away, 'HR_Away': HR_Away, 'BB_Away': BB_Away, 'SO_Away': SO_Away, 'HBP_Away': HBP_Away, 
                                           'SB_Away': SB_Away, 'CS_Away': CS_Away, position: 1})



                        if(batter_df.empty):
                            batter_df = pd.DataFrame(columns=batter_headers)

                        batter_df = batter_df.append(stats, ignore_index=True)   # Getting stats from batter stats dataframe
                elif position in ['SP', 'RP']:
                    if(not pitcher_df.empty and player_ID in pitcher_df['playerID'].values.tolist()):  # player already in batter_df at different position
                        pitcher_df.loc[pitcher_df['playerID']==player_ID, position] = 1
                    else:

                        pitcher_headers = ['Name', 'playerID', 'Team', 'Throws', 'TBF_L',
                                           'singles_L', 'doubles_L', 'triples_L', 'HR_L', 'R_L', 
                                           'ER_L', 'BB_L','HBP_L', 'SO_L', 
                                           'TBF_R',  'singles_R', 'doubles_R', 'triples_R', 'HR_R', 'R_R', 
                                           'ER_R', 'BB_R','HBP_R', 'SO_R', 'TBF_Home',
                                           'singles_Home', 'doubles_Home', 'triples_Home', 'HR_Home', 'R_Home', 
                                           'ER_Home', 'BB_Home','HBP_Home', 'SO_Home', 
                                           'TBF_Away',  'singles_Away', 'doubles_Away', 'triples_Away', 'HR_Away', 'R_Away', 
                                           'ER_Away', 'BB_Away','HBP_Away', 'SO_Away']

                        player_stats = pitcher_stats.loc[pitcher_stats['playerId']==int(player_ID)]

                        if player_stats.empty:  # Change this after finding handedness of all players
                            continue

                        if(player_stats['Throws'].tolist()[0]=='L'):
                            TBF_L = 90
                            singles_L = 14
                            doubles_L = 4
                            triples_L = 0
                            HR_L = 2
                            R_L = 10
                            ER_L = 9
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 25

                            TBF_R = 90
                            singles_R = 15
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 3
                            R_R = 11
                            ER_R = 10
                            BB_R = 5
                            HBP_R = 0
                            SO_R = 20
                        elif(player_stats['Throws'].tolist()[0]=='R'):
                            TBF_L = 90
                            singles_L = 15
                            doubles_L = 4
                            triples_L = 0
                            HR_L = 3
                            R_L = 11
                            ER_L = 10
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 20

                            TBF_R = 90
                            singles_R = 14
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 2
                            R_R = 10
                            ER_R = 9
                            BB_R = 5
                            HBP_R = 0
                            SO_R = 25


                        TBF_Home = 90
                        singles_Home = 14
                        doubles_Home = 4
                        triples_Home = 0
                        HR_Home = 2
                        R_Home = 10
                        ER_Home = 9
                        BB_Home = 5
                        HBP_Home = 0
                        SO_Home = 25

                        TBF_Away = 90
                        singles_Away = 15
                        doubles_Away = 4
                        triples_Away = 0
                        HR_Away = 3
                        R_Away = 11
                        ER_Away = 10
                        BB_Away = 5
                        HBP_Away = 0
                        SO_Away = 20


                        seasons_list = sorted(player_stats['Season'].to_list())
                        for year in seasons_list:
                            multiplier = (1-(max(seasons_list)-year)/len(seasons_list))

                            TBF_L += player_stats.loc[player_stats['Season']==year]['TBF_L'].values[0]*multiplier
                            singles_L += player_stats.loc[player_stats['Season']==year]['singles_L'].values[0]*multiplier
                            doubles_L += player_stats.loc[player_stats['Season']==year]['doubles_L'].values[0]*multiplier
                            triples_L += player_stats.loc[player_stats['Season']==year]['triples_L'].values[0]*multiplier
                            HR_L += player_stats.loc[player_stats['Season']==year]['HR_L'].values[0]*multiplier
                            R_L += player_stats.loc[player_stats['Season']==year]['R_L'].values[0]*multiplier
                            ER_L += player_stats.loc[player_stats['Season']==year]['ER_L'].values[0]*multiplier
                            BB_L += player_stats.loc[player_stats['Season']==year]['BB_L'].values[0]*multiplier
                            HBP_L += player_stats.loc[player_stats['Season']==year]['HBP_L'].values[0]*multiplier
                            SO_L += player_stats.loc[player_stats['Season']==year]['SO_L'].values[0]*multiplier


                            TBF_R += player_stats.loc[player_stats['Season']==year]['TBF_R'].values[0]*multiplier
                            singles_R += player_stats.loc[player_stats['Season']==year]['singles_R'].values[0]*multiplier
                            doubles_R += player_stats.loc[player_stats['Season']==year]['doubles_R'].values[0]*multiplier
                            triples_R += player_stats.loc[player_stats['Season']==year]['triples_R'].values[0]*multiplier
                            HR_R += player_stats.loc[player_stats['Season']==year]['HR_R'].values[0]*multiplier
                            R_R += player_stats.loc[player_stats['Season']==year]['R_R'].values[0]*multiplier
                            ER_R += player_stats.loc[player_stats['Season']==year]['ER_R'].values[0]*multiplier
                            BB_R += player_stats.loc[player_stats['Season']==year]['BB_R'].values[0]*multiplier
                            HBP_R += player_stats.loc[player_stats['Season']==year]['HBP_R'].values[0]*multiplier
                            SO_R += player_stats.loc[player_stats['Season']==year]['SO_R'].values[0]*multiplier

                            TBF_Home += player_stats.loc[player_stats['Season']==year]['TBF_Home'].values[0]*multiplier
                            singles_Home += player_stats.loc[player_stats['Season']==year]['singles_Home'].values[0]*multiplier
                            doubles_Home += player_stats.loc[player_stats['Season']==year]['doubles_Home'].values[0]*multiplier
                            triples_Home += player_stats.loc[player_stats['Season']==year]['triples_Home'].values[0]*multiplier
                            HR_Home += player_stats.loc[player_stats['Season']==year]['HR_Home'].values[0]*multiplier
                            R_Home += player_stats.loc[player_stats['Season']==year]['R_Home'].values[0]*multiplier
                            ER_Home += player_stats.loc[player_stats['Season']==year]['ER_Home'].values[0]*multiplier
                            BB_Home += player_stats.loc[player_stats['Season']==year]['BB_Home'].values[0]*multiplier
                            HBP_Home += player_stats.loc[player_stats['Season']==year]['HBP_Home'].values[0]*multiplier
                            SO_Home += player_stats.loc[player_stats['Season']==year]['SO_Home'].values[0]*multiplier


                            TBF_Away += player_stats.loc[player_stats['Season']==year]['TBF_Away'].values[0]*multiplier
                            singles_Away += player_stats.loc[player_stats['Season']==year]['singles_Away'].values[0]*multiplier
                            doubles_Away += player_stats.loc[player_stats['Season']==year]['doubles_Away'].values[0]*multiplier
                            triples_Away += player_stats.loc[player_stats['Season']==year]['triples_Away'].values[0]*multiplier
                            HR_Away += player_stats.loc[player_stats['Season']==year]['HR_Away'].values[0]*multiplier
                            R_Away += player_stats.loc[player_stats['Season']==year]['R_Away'].values[0]*multiplier
                            ER_Away += player_stats.loc[player_stats['Season']==year]['ER_Away'].values[0]*multiplier
                            BB_Away += player_stats.loc[player_stats['Season']==year]['BB_Away'].values[0]*multiplier
                            HBP_Away += player_stats.loc[player_stats['Season']==year]['HBP_Away'].values[0]*multiplier
                            SO_Away += player_stats.loc[player_stats['Season']==year]['SO_Away'].values[0]*multiplier


                        stats = pd.Series({'Name': player_name, 'playerID': player_ID, 'Team': team_name, 
                                           'Throws': player_stats['Throws'].tolist()[0], 
                                           'TBF_L': PA_L,'singles_L': singles_L, 'doubles_L': doubles_L, 'triples_L': triples_L, 
                                           'HR_L': HR_L, 'R_L': R_L, 'ER_L': ER_L, 'BB_L': BB_L, 'HBP_L': HBP_L, 'SO_L': SO_L,  
                                           'TBF_R': PA_R, 'singles_R': singles_R, 'doubles_R': doubles_R, 'triples_R': triples_R, 
                                           'HR_R': HR_R, 'R_R': R_R, 'ER_R': ER_R, 'BB_R': BB_R, 'HBP_R': HBP_R, 'SO_R': SO_R,
                                           'TBF_Home': PA_Home,'singles_Home': singles_Home, 'doubles_Home': doubles_Home, 'triples_Home': triples_Home, 
                                           'HR_Home': HR_Home, 'R_Home': R_Home, 'ER_Home': ER_Home, 'BB_Home': BB_Home, 'HBP_Home': HBP_Home, 'SO_Home': SO_Home,  
                                           'TBF_Away': PA_Away, 'singles_Away': singles_Away, 'doubles_Away': doubles_Away, 'triples_Away': triples_Away, 
                                           'HR_Away': HR_Away, 'R_Away': R_Away, 'ER_Away': ER_Away, 'BB_Away': BB_Away, 
                                           'HBP_Away': HBP_Away, 'SO_Away': SO_Away, position: 1})



                        if(pitcher_df.empty):
                            pitcher_df = pd.DataFrame(columns=pitcher_headers)

                        pitcher_df = pitcher_df.append(stats, ignore_index=True)   # Getting pitcher stats from pitcher stats dataframe
  


        batter_df.fillna(0, inplace=True)
        pitcher_df.fillna(0, inplace=True)

        team_dict.update({team_name: Team_new(team_name, league[team_name], batter_df, pitcher_df, lineup_dict[team_name])})


    return team_dict

def active_rosters():
    
    url = 'https://www.mlb.com/orioles/roster/depth-chart'
    headers = {'User-Agent': header_name}

    url_list = []
    url_start = 'https://www.mlb.com/'
    
    team_roster_dict = {}
    
    for team_name in mlb_teams:
        url = url_start+team_name+'/roster/depth-chart'

        position_player_dict = {}

        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        # Getting player names on each roster
        for table in soup.findAll('table', attrs={'class': 'roster__table'}):
            position = table.td.get_text()

            if position=='Rotation':
                position = 'SP'
            elif position=='Bullpen':
                position = 'RP'
            elif position=='Catcher':
                position = 'C'
            elif position=='First Base':
                position = '1B'
            elif position=='Second Base':
                position = '2B'
            elif position=='Shortstop':
                position = 'SS'
            elif position=='Third Base':
                position = '3B'
            elif position=='Left Field':
                position = 'LF'
            elif position=='Center Field':
                position = 'CF'
            elif position=='Right Field':
                position = 'RF'
            elif position=='Designated Hitter':
                position = 'DH'

            position_player_dict[position] = []

            for row in table.tbody.findAll('td', attrs={'class': 'info'}):
                if row.find('span', attrs={'class': 'status-il'}):
                    continue 
                
                player_name = row.a.get_text().strip()
                position_player_dict[position].append(player_name)
                
                if row.find('span', attrs={'class': 'closer'}):
                    position_player_dict['Closer'] = player_name 
                        
        team_roster_dict[nickname_to_city_dict[team_name]] = position_player_dict

        time.sleep(2)
                        
    return team_roster_dict

def loading_moneylines(df, schedule_dict, year):
    """Loads moneylines to spreadsheet
    Args:
        df: dataframe that odds will be written to
        scedule_dict: dict of games to be played (date as key, matchups as values)
        year: year that will be used to gather the dates of the games
    Returns:
        dataframe of data with moneylines
    """
    names = ['Open', 'ml','Westgate','MGM Mirage', 'betMGM',
             'William Hill', 'CG Technology', 'Circa Sports','Stations']
    book = pd.DataFrame(columns=names)
    temp_away_list = []
    temp_home_list = []
    teams_list = []
    dates_list = []
    URL = 'http://www.vegasinsider.com/mlb/odds/las-vegas/'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
        for num, games in enumerate(gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'})):
            for matchup in games.findAll('tr'):
                for game_info in matchup.findAll('td', attrs={'class': ['viCellBg1 cellTextNorm cellBorderL1 gameCell',
                                                                      'viCellBg2 cellTextNorm cellBorderL1 gameCell']}):
                    
                    no_game = False

                    for date in game_info.findAll('span', attrs={'class': 'cellTextHot'}):   # Finding and formatting dates
                        date = date.get_text().split()[0]
                        month = date.split('/')[0]
                        day = date.split('/')[1]
                        date = str(month) + '/' + str(day) + '/' + str(year)
                        dates_list.extend([date, date])

                    for team_name in game_info.findAll('a', attrs={'class': 'tabletext'}): # Gathering team names
                        if(team_name.get_text().strip()=='N.Y. Yankees'):
                            teams_list.append('New York Yankees')
                        elif(team_name.get_text().strip()=='N.Y. Mets'):
                            teams_list.append('New York Mets')
                        elif(team_name.get_text().strip()=='Chi. Cubs'):
                            teams_list.append('Chicago Cubs')
                        elif(team_name.get_text().strip()=='Chi. White Sox'):
                            teams_list.append('Chicago White Sox')
                        elif(team_name.get_text().strip()=='L.A. Dodgers'):
                            teams_list.append('Los Angeles Dodgers')
                        elif(team_name.get_text().strip()=='L.A. Angels'):
                            teams_list.append('Los Angeles Angels')
                        else:
                            teams_list.append(team_name.get_text().strip())

                    if((teams_list[-2], teams_list[-1]) not in list(schedule_dict.values())[0]):
                        teams_list = teams_list[0:len(teams_list)-2]
                        dates_list = dates_list[0:len(dates_list)-2]
                        no_game = True

                    if(not no_game):
                        for num, spread in enumerate(matchup.findAll('td', attrs={'class': 
                                                                     ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap oddsCell',
                                                                      'viCellBg1 cellTextHot cellBorderL1 center_text nowrap oddsCell',
                                                                      'viCellBg2 cellTextNorm cellBorderL1 center_text nowrap oddsCell',
                                                                      'viCellBg2 cellTextHot cellBorderL1 center_text nowrap oddsCell']})):           
                            spread_text = spread.get_text().strip()

                            spread_away = str(spread).split('<br/>')[1]
                            if(len(spread_away.split())==0 or spread_away.split()[0]=='XX'):
                                away_spread = ''
                            else:   
                                away_spread = spread_away.split()[0]

                            spread_home = str(spread).split('<br/>')[2]
                            if(len(spread_home.split())<2 or spread_home.split()[0]=='XX'):
                                home_spread = ''
                            else:   
                                home_spread = spread_home.split()[0]


                            temp_away_list.append(away_spread)
                            temp_home_list.append(home_spread)

                            if len(temp_home_list) == 9:
                                book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                                book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                                temp_away_list = []
                                temp_home_list = []

    while(len(dates_list) > len(teams_list)):  # In case extra games in odds like All Star game
        dates_list.pop()
        book.drop(book.tail(1).index, inplace=True)

    book['team'] = pd.Series(teams_list)
    
    book['date'] = dates_list
    book = book[book.date==datetime.datetime.now().strftime('%m/%d/%Y')]    # Only getting odds for games in current week

    odds = book[['team', 'ml']]
    odds.set_index('team', inplace=True)
    df = pd.merge(odds, df)[['date', 'team', 'proj win%', 'ml']]
    
    return odds

In [54]:
batter_stats = column_type(reading_data('Joined Batter Stats.csv'), True)
batter_stats_2019 = column_type(reading_data('Splits_Leaderboard_Data_Hitters.csv'), True)
pitcher_projections = column_type(reading_data('Pitcher_Projections_FanGraphs_2020.csv'), False)
pitcher_stats_2019 = column_type(reading_data('Splits_Leaderboard_Data_Pitchers.csv'), False)
schedule = reading_data('2020_mlb_schedule.csv')
lineup_dict, schedule_dict = daily_lineups()
team_dict = create_teams(batter_projections, batter_stats_2019, pitcher_projections, pitcher_stats_2019, lineup_dict)

<ipython-input-12-d2980fdae5e4>:596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats[pos] = int(row['PA'])
<ipython-input-12-d2980fdae5e4>:597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats['BsR'] = float(row['BsR'])
<ipython-input-12-d2980fdae5e4>:598: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

{}


KeyError: 'Atlanta'

In [410]:
lineup_dict, schedule_dict, player_dict  = fangraphs_daily_lineups({})
rosters = active_rosters()

print(lineup_dict)
print(schedule_dict)
print(player_dict)
print(rosters)

{'Cincinnati': {'starter': 'Vladimir Gutierrez', 'opener': None, 'lineup': ['', 'Jesse Winker', 'Nick Castellanos', 'Tyler Stephenson', 'Joey Votto', 'Eugenio Suarez', 'Aristides Aquino', 'Tyler Naquin', 'Mike Freeman']}, 'Kansas City': {'starter': 'Mike Minor', 'opener': None, 'lineup': ['Whit Merrifield', 'Carlos Santana', 'Salvador Perez', "Ryan O'Hearn", 'Jorge Soler', 'Hanser Alberto', 'Hunter Dozier', 'Michael A. Taylor', 'Nicky Lopez']}, 'Detroit': {'starter': 'Wily Peralta', 'opener': None, 'lineup': ['Akil Baddoo', 'Jonathan Schoop', 'Miguel Cabrera', 'Eric Haase', 'Jeimer Candelario', 'Robbie Grossman', 'Nomar Mazara', 'Willi Castro', 'Zack Short']}, 'Texas': {'starter': 'Kolby Allard', 'opener': None, 'lineup': ['Eli White', 'Nate Lowe', 'Adolis Garcia', 'Joey Gallo', 'Jonah Heim', 'Isiah Kiner-Falefa', 'David Dahl', 'Nick Solak', 'Brock Holt']}, 'Boston': {'starter': 'Martin Perez', 'opener': None, 'lineup': ['Enrique Hernandez', 'Rafael Devers', 'J.D. Martinez', 'Xander Bo

In [413]:
batter_stats = column_type(reading_data('Joined_Batter_Stats.csv'), True)
pitcher_stats = column_type(reading_data('Joined_Pitcher_Stats.csv'), True)
team_dict = create_teams_new(batter_stats, pitcher_stats, lineup_dict, rosters)
team_dict

Nationals
Alex Avila
Starlin Castro
Luis García
Starlin Castro
Alcides Escobar
Jordy Mercer
Luis García
Carter Kieboom
Jordy Mercer
Carter Kieboom
Jordy Mercer
Alcides Escobar
Kyle Schwarber
Andrew Stevenson
Yadiel Hernandez
Andrew Stevenson
Josh Harrison
Andrew Stevenson
Yadiel Hernandez
Ryan Zimmerman
Andrew Stevenson
Josh Harrison
Starlin Castro
Kyle Schwarber
Alcides Escobar
Stephen Strasburg
Erick Fedde
Jefry Rodriguez
Austin Voth
Kyle Finnegan
Daniel Hudson
Tanner Rainey
Paolo Espino
Erick Fedde
Javy Guerra
Phillies
Rafael Marchan
J.T. Realmuto
Ronald Torreyes
Nick Maton
Scott Kingery
Brad Miller
Nick Maton
Matt Joyce
Adam Haseley
Matt Vierling
Matt Joyce
Adam Haseley
Mickey Moniak
Matt Vierling
Adam Haseley
Mickey Moniak
Brad Miller
Matt Joyce
Mickey Moniak
J.T. Realmuto
Rhys Hoskins
Adam Haseley
Bryce Harper
Travis Jankowski
Luke Williams
Spencer Howard
Bailey Falter
Chase Anderson
Adonis Medina
Sam Coonrod
Chase Anderson
Spencer Howard
Vince Velasquez
Cristopher Sánchez
Brady 

{'Cleveland': <__main__.Team_new at 0x8a89f10>,
 'Los Angeles Angels': <__main__.Team_new at 0x67575e80>,
 'Washington': <__main__.Team_new at 0x65eb1df0>,
 'Philadelphia': <__main__.Team_new at 0x61312d90>,
 'Chicago Cubs': <__main__.Team_new at 0x66a2c0b8>,
 'Cincinnati': <__main__.Team_new at 0x57c789a0>,
 'San Diego': <__main__.Team_new at 0x66d29d18>,
 'Boston': <__main__.Team_new at 0x63e68d30>,
 'Kansas City': <__main__.Team_new at 0x66d29af0>,
 'Detroit': <__main__.Team_new at 0x65eebaa8>,
 'Texas': <__main__.Team_new at 0x65ef6e80>}

In [327]:
win_rates = simulate_slate_new(lineup_dict, active_rosters, schedule_dict, 1)
win_rates

KeyError: 'Chicago White Sox'

In [406]:
rosters

{'Atlanta': {'SP': ['Max Fried',
   'Ian Anderson',
   'Drew Smyly',
   'Charlie Morton'],
  'RP': ['Shane Greene',
   'Luke Jackson',
   'A.J. Minter',
   'Tyler Matzek',
   'Will Smith',
   'Chris Martin',
   'Josh Tomlin',
   'Jacob Webb',
   'Jesse Chavez'],
  'Catcher': ['William Contreras', 'Kevan Smith'],
  'First Base': ['Freddie Freeman'],
  'Second Base': ['Ozzie Albies', 'Ehire Adrianza', 'Orlando Arcia'],
  'Third Base': ['Austin Riley',
   'Ehire Adrianza',
   'Pablo Sandoval',
   'Orlando Arcia'],
  'Shortstop': ['Dansby Swanson', 'Ehire Adrianza', 'Orlando Arcia'],
  'Left Field': ['Abraham Almonte',
   'Ender Inciarte',
   'Guillermo Heredia',
   'Orlando Arcia'],
  'Center Field': ['Guillermo Heredia', 'Abraham Almonte'],
  'Right Field': ['Ronald Acuña Jr.',
   'Ender Inciarte',
   'Guillermo Heredia',
   'Abraham Almonte']},
 'Washington': {'SP': ['Max Scherzer',
   'Patrick Corbin',
   'Jon Lester',
   'Joe Ross',
   'Paolo Espino'],
  'RP': ['Brad Hand',
   'Wander

In [ ]:
for n in range(1):
    g = Game(team_dict['Texas'], team_dict['San Francisco'])
    g.play()
    g.print_result()

In [ ]:
a, b = team_dict['New York Yankees'].cumul_stats_df()
a

In [287]:
odds_df = loading_moneylines(pd.DataFrame(columns=['date', 'team', 'proj win%', 'ml']), schedule_dict, 2021)
#odds_df.set_index(win_rates.index, inplace=True)
odds_df

,ml
team,
St. Louis,+190
San Francisco,-210
Los Angeles Dodgers,-155
Miami,+145
Atlanta,-190
Pittsburgh,+160
Milwaukee,-130
New York Mets,+110
Philadelphia,+130


In [546]:
pd.concat([odds_df, win_rates], axis=1)

,ml,0
Cleveland,+127,1.0
Cincinatti,-137,0.0
Philadelphia,-125,0.0
New York Yankees,+115,1.0
New York Mets,+190,0.0
Atlanta,-220,1.0
Pittsburg,-105,0.0
Minnesota,-105,1.0
Chicago White Sox,+135,0.0
Milwaukee,-145,1.0


In [99]:
url = 'https://www.fangraphs.com/livescoreboard.aspx?'
headers = {'User-Agent': header_name}
source = requests.get(url, headers=headers)
soup = BeautifulSoup(source.content, 'html.parser')

lineup_dict = {}
schedule_dict = {}
player_dict = {}
schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')] = []

header = soup.find('tr', attrs={'class': 'win-board__header'})

for games in header.next_siblings:

    for matchup in games.findAll('td', style=True, align=False):

        for num, team_names in enumerate(matchup.findAll('a')): # Getting team names
            if(num==0):

                if team_names.get_text().strip().capitalize() in league.keys():
                    away_team = team_names.get_text().strip().capitalize()
                else:
                    away_team = nickname_to_city_dict[team_names.get_text().strip().lower()]
                print(away_team)
                if(away_team in lineup_dict.keys()):
                    away_team = away_team + '2'
            elif(num==1):
                
                if team_names.get_text().strip().capitalize() in league.keys():
                    home_team = team_names.get_text().strip().capitalize()
                else:
                    home_team = nickname_to_city_dict[team_names.get_text().strip().lower()]
                print(home_team)
                if(home_team in lineup_dict.keys()):
                    home_team = home_team + '2'
                    
            schedule_dict[datetime.datetime.now().strftime('%Y-%m-%d')].append((away_team, home_team))  # Adding matchup to schedule dict
    
          
        away_lineup_dict = {}
        away_lineup = []
        away_starter = None
        away_opener = None
        home_lineup_dict = {}
        home_lineup = []
        home_starter = None
        home_opener = None
        
        lineup = matchup.find('table', attrs={'class', 'lineup'})
        if(lineup!=None):
            for num, position in enumerate(lineup.findAll('td', attrs={'align': 'left'})):

                if(num%2==0):
                    if position.get_text()[0:2] == 'SP':
                        away_starter = position.a.get_text().strip()
                        away_starter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                        print(away_starter_id)
                    elif position.get_text()[0:2] == 'OP':
                        away_opener = position.a.get_text().strip()
                    elif position.get_text()[0:2] not in ('SP', 'PP', 'OP'):
                        for player in position.findAll('a'):
                            away_lineup.append(player.get_text().strip())

                    away_lineup_dict['starter'] = away_starter
                    away_lineup_dict['opener'] = away_opener
                    away_lineup_dict['lineup'] = away_lineup

                elif(num%2==1):
                    if position.get_text()[0:2] == 'SP':
                        home_starter = position.a.get_text().strip()
                    elif position.get_text()[0:2] == 'OP':
                        home_opener = position.a.get_text().strip()
                    elif position.get_text()[0:2] not in ('SP', 'PP', 'OP'):
                        for player in position.findAll('a'):
                            home_lineup.append(player.get_text().strip())

                        home_lineup_dict['starter'] = home_starter
                        home_lineup_dict['opener'] = home_opener
                        home_lineup_dict['lineup'] = home_lineup    

            lineup_dict[away_team] = away_lineup_dict
            lineup_dict[home_team] = home_lineup_dict

    

Miami
Philadelphia
17085
Los Angeles Dodgers
Washington
19388
San Diego
Cincinnati
23796
Los Angeles Angels
New York Yankees
19867
New York Mets
Atlanta
10954
Texas
Oakland
19409
Milwaukee
Pittsburgh
19361
Houston
Cleveland
17295
St. Louis
Colorado
2233
San Francisco
Arizona
6893


In [114]:
url = 'https://www.fangraphs.com/players/'
headers = {'User-Agent': header_name}

players_stats_dict = {}

for name, id in player_dict.items():
    print(name)
    print(id)
    adjusted_name = name.replace(' ', '-').replace('.', '')  # Fagraphs format
    
    start_year = 2021
    end_year = 2021
    
    for season in range (start_year, end_year+1):   # Going through several season worth of stats
    
        player_url = url + adjusted_name + '/' + str(id) + '/splits?&season=' + str(season)
        print(player_url)
        player_url = 'https://www.fangraphs.com/players/pablo-lopez/17085/splits?position=P&season=2021'
        source = requests.get(player_url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        attr_list_hand = []
        values_list_left = []
        values_list_right = []
        attr_list_location = []
        values_list_home = []
        values_list_away = []
        print(soup)
        for table in soup.findAll('div', attrs={'class': 'table-scroll'}):
            print("hi")
            for num, row in enumerate(table.findAll('tr', attrs={'class': 'row-mlb-season'})):
                if(num==2):
                    for attr in row.findAll('th'):
                        attr_list_hand.append(attr['data-stat'])
                if(num in (3, 4)):
                    temp_list = []
                    for attr in row.findAll('th'):
                        temp_list.append(attr.get_text().strip())

                    if 'vs L' in temp_list:
                        values_list_left = temp_list
                    elif 'vs R' in temp_list:
                        values_list_right = temp_list
                        
                if(num==5):
                    for attr in row.findAll('th'):
                        attr_list_location.append(attr['data-stat'])
                if(num in (6, 7)):
                    temp_list = []
                    for attr in row.findAll('th'):
                        temp_list.append(attr.get_text().strip())

                    if 'Home' in temp_list:
                        values_list_home = temp_list
                    elif 'Away' in temp_list:
                        values_list_away = temp_list
            
            attr_dict_left = {k: v for k, v in zip(attr_list_hand, values_list_left)}
            attr_dict_right = {k: v for k, v in zip(attr_list_hand, values_list_right)}
            attr_dict_home = {k: v for k, v in zip(attr_list_location, values_list_home)}
            attr_dict_away = {k: v for k, v in zip(attr_list_location, values_list_away)}
            
            
            player_stats_dict[name] = {}
            player_stats_dict[name][season] = {}
            player_stats_dict[name][season]['Left'] = attr_dict_left
            player_stats_dict[name][season]['Right'] = attr_dict_right
            player_stats_dict[name][season]['Home'] = attr_dict_home
            player_stats_dict[name][season]['Away'] = attr_dict_away
            
            print(player_stats_dict)

    
    
    
    


Pablo Lopez
17085
https://www.fangraphs.com/players/Pablo-Lopez/17085/splits?&season=2021

<!DOCTYPE html>

<html>
<head id="Head1"><title>
	Pablo López - Splits - 2021 - Pitching | FanGraphs Baseball
</title><meta content="Pablo López 2021 pitching splits statistics for Major League, Minor League, and postseason baseball" data-react-helmet="true" id="description" name="description"/><link href="//cdn.fangraphs.com/css/telerik" rel="stylesheet"/>
<script src="//ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js" type="text/javascript"></script>
<link href="https://cdn.fangraphs.com/dist/css/head_section-style.css?v=ec817083430befe5d7ee" rel="stylesheet">
<script src="https://cdn.fangraphs.com/dist/common-bundle.js?v=ec817083430befe5d7ee"></script>
<script src="https://cdn.fangraphs.com/dist/head_section-bundle.js?v=ec817083430befe5d7ee"></script>
<link href="https://cdn.fangraphs.com/dist/css/mobile-style.css?v=ec817083430befe5d7ee" rel="stylesheet"/>
<meta content="width=device

KeyboardInterrupt: 